### Import Packages

In [1]:
import optuna
import os
import numpy as np
import pandas as pd
import scipy
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
import matplotlib.pyplot as plt
import seaborn as sns 
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
# from sklearn.tree import ExtraTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score,StratifiedKFold,train_test_split
from sklearn.decomposition import PCA
import xgboost as XGB
from sklearn.feature_selection import SelectKBest,f_regression, mutual_info_regression
from sklearn.feature_selection import VarianceThreshold
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 100)

print(os.listdir('../input/engraulis-encrasicolus-prediction/'))


['sample_submission.csv', 'train.csv', 'test.csv']


In [2]:
train = pd.read_csv('../input/engraulis-encrasicolus-prediction/train.csv')
test = pd.read_csv('../input/engraulis-encrasicolus-prediction/test.csv')
print("Train has {} samples and {} variables.".format(train.shape[0],train.shape[1]))
print("Test has {} samples and {} variables.".format(test.shape[0],test.shape[1]))

Train has 527 samples and 6831 variables.
Test has 202 samples and 6830 variables.


In [3]:
train.head()

,obs_id,Center Lat,Center Long,Overall Probability,distanceToCoast,majorRiverDistance,majorRiversScale,bathymetry,substrateType,temperatureSurface_mean,temperatureSurface_median,temperatureSurface_std,temperatureSurface_max,temperatureSurface_min,temperatureSurface_max_to_min_diff,temperatureSurface_skew,temperatureSurface_kurt,temperatureSurface_mad,temperatureSurface_quantile_1,temperatureSurface_quantile_5,temperatureSurface_quantile_10,temperatureSurface_quantile_90,temperatureSurface_quantile_95,temperatureSurface_quantile_99,temperature100_300_mean,temperature100_300_median,temperature100_300_std,temperature100_300_max,temperature100_300_min,temperature100_300_max_to_min_diff,temperature100_300_skew,temperature100_300_kurt,temperature100_300_mad,temperature100_300_quantile_1,temperature100_300_quantile_5,temperature100_300_quantile_10,temperature100_300_quantile_90,temperature100_300_quantile_95,temperature100_300_quantile_99,salinitySurface_mean,salinitySurface_median,salinitySurface_std,salinitySurface_max,salinitySurface_min,salinitySurface_max_to_min_diff,salinitySurface_skew,salinitySurface_kurt,salinitySurface_mad,salinitySurface_quantile_1,salinitySurface_quantile_5,salinitySurface_quantile_10,salinitySurface_quantile_90,salinitySurface_quantile_95,salinitySurface_quantile_99,salinity100_300_mean,salinity100_300_median,salinity100_300_std,salinity100_300_max,salinity100_300_min,salinity100_300_max_to_min_diff,salinity100_300_skew,salinity100_300_kurt,salinity100_300_mad,salinity100_300_quantile_1,salinity100_300_quantile_5,salinity100_300_quantile_10,salinity100_300_quantile_90,salinity100_300_quantile_95,salinity100_300_quantile_99,dissolvedOxygenSurface_mean,dissolvedOxygenSurface_median,dissolvedOxygenSurface_std,dissolvedOxygenSurface_max,dissolvedOxygenSurface_min,dissolvedOxygenSurface_max_to_min_diff,dissolvedOxygenSurface_skew,dissolvedOxygenSurface_kurt,dissolvedOxygenSurface_mad,dissolvedOxygenSurface_quantile_1,dissolvedOxygenSurface_quantile_5,dissolvedOxygenSurface_quantile_10,dissolvedOxygenSurface_quantile_90,dissolvedOxygenSurface_quantile_95,dissolvedOxygenSurface_quantile_99,dissolvedOxygen100_300_mean,dissolvedOxygen100_300_median,dissolvedOxygen100_300_std,dissolvedOxygen100_300_max,dissolvedOxygen100_300_min,dissolvedOxygen100_300_max_to_min_diff,dissolvedOxygen100_300_skew,dissolvedOxygen100_300_kurt,dissolvedOxygen100_300_mad,dissolvedOxygen100_300_quantile_1,dissolvedOxygen100_300_quantile_5,dissolvedOxygen100_300_quantile_10,dissolvedOxygen100_300_quantile_90,dissolvedOxygen100_300_quantile_95,dissolvedOxygen100_300_quantile_99,meridionalCurrentSurface_mean,...,phosphate100_300_20wiener_mean,phosphate100_300_50wiener_mean,phosphate100_300_5savgol_mean,phosphate100_300_11savgol_mean,phosphate100_300_25savgol_mean,phosphate100_300_51savgol_mean,phosphate100_300_symiirorder1_mean,phosphate100_300_symiirorder2_mean,phosphate100_300_decimate_mean,phosphate100_300_detrend_mean,phosphate100_300_argrelmin_count,phosphate100_300_argrelmax_count,phosphate100_300_argrelminmax_count,phosphate100_300_mean_change_argrelmin,phosphate100_300_mean_change_argrelmax,phosphate100_300_mean_change_argrelmin_rate,phosphate100_300_mean_change_argrelmax_rate,phosphate100_300_peak_count,phosphate100_300_mean_change_peak,phosphate100_300_mean_change_peak_rate,phosphate100_300_peak_min,phosphate100_300_peak_max,phosphate100_300_peak_mean,phosphate100_300_peak_width_min,phosphate100_300_peak_width_max,phosphate100_300_peak_width_mean,phosphate100_300_peak_max_to_m in,phosphate100_300_peak_width_max_to_min,temperatureSurface_neighbor_mean,temperatureSurface_neighbor_max,temperatureSurface_neighbor_min,temperatureSurface_neighbor_std,temperature100_300_neighbor_mean,temperature100_300_neighbor_max,temperature100_300_neighbor_min,temperature100_300_neighbor_std,salinitySurface_neighbor_mean,salinitySurface_neighbor_max,salinitySurface_neighbor_min,salinitySurface_neighbor_std,salinity100_300_neighbor_mean,salinity100_300

**Variables**

In [4]:
train.columns.values

array(['obs_id', 'Center Lat', 'Center Long', ...,
       'phosphate100_300_neighbor_max', 'phosphate100_300_neighbor_min',
       'phosphate100_300_neighbor_std'], dtype=object)

### Encode Categorical Features

In [5]:
for df in [train,test]:
    for c in df:
        if (df[c].dtype=='object'):
            lbl = preprocessing.LabelEncoder() 
            lbl.fit(list(df[c].values))
            df[c] = lbl.transform(list(df[c].values))

In [6]:
train.head()

,obs_id,Center Lat,Center Long,Overall Probability,distanceToCoast,majorRiverDistance,majorRiversScale,bathymetry,substrateType,temperatureSurface_mean,temperatureSurface_median,temperatureSurface_std,temperatureSurface_max,temperatureSurface_min,temperatureSurface_max_to_min_diff,temperatureSurface_skew,temperatureSurface_kurt,temperatureSurface_mad,temperatureSurface_quantile_1,temperatureSurface_quantile_5,temperatureSurface_quantile_10,temperatureSurface_quantile_90,temperatureSurface_quantile_95,temperatureSurface_quantile_99,temperature100_300_mean,temperature100_300_median,temperature100_300_std,temperature100_300_max,temperature100_300_min,temperature100_300_max_to_min_diff,temperature100_300_skew,temperature100_300_kurt,temperature100_300_mad,temperature100_300_quantile_1,temperature100_300_quantile_5,temperature100_300_quantile_10,temperature100_300_quantile_90,temperature100_300_quantile_95,temperature100_300_quantile_99,salinitySurface_mean,salinitySurface_median,salinitySurface_std,salinitySurface_max,salinitySurface_min,salinitySurface_max_to_min_diff,salinitySurface_skew,salinitySurface_kurt,salinitySurface_mad,salinitySurface_quantile_1,salinitySurface_quantile_5,salinitySurface_quantile_10,salinitySurface_quantile_90,salinitySurface_quantile_95,salinitySurface_quantile_99,salinity100_300_mean,salinity100_300_median,salinity100_300_std,salinity100_300_max,salinity100_300_min,salinity100_300_max_to_min_diff,salinity100_300_skew,salinity100_300_kurt,salinity100_300_mad,salinity100_300_quantile_1,salinity100_300_quantile_5,salinity100_300_quantile_10,salinity100_300_quantile_90,salinity100_300_quantile_95,salinity100_300_quantile_99,dissolvedOxygenSurface_mean,dissolvedOxygenSurface_median,dissolvedOxygenSurface_std,dissolvedOxygenSurface_max,dissolvedOxygenSurface_min,dissolvedOxygenSurface_max_to_min_diff,dissolvedOxygenSurface_skew,dissolvedOxygenSurface_kurt,dissolvedOxygenSurface_mad,dissolvedOxygenSurface_quantile_1,dissolvedOxygenSurface_quantile_5,dissolvedOxygenSurface_quantile_10,dissolvedOxygenSurface_quantile_90,dissolvedOxygenSurface_quantile_95,dissolvedOxygenSurface_quantile_99,dissolvedOxygen100_300_mean,dissolvedOxygen100_300_median,dissolvedOxygen100_300_std,dissolvedOxygen100_300_max,dissolvedOxygen100_300_min,dissolvedOxygen100_300_max_to_min_diff,dissolvedOxygen100_300_skew,dissolvedOxygen100_300_kurt,dissolvedOxygen100_300_mad,dissolvedOxygen100_300_quantile_1,dissolvedOxygen100_300_quantile_5,dissolvedOxygen100_300_quantile_10,dissolvedOxygen100_300_quantile_90,dissolvedOxygen100_300_quantile_95,dissolvedOxygen100_300_quantile_99,meridionalCurrentSurface_mean,...,phosphate100_300_20wiener_mean,phosphate100_300_50wiener_mean,phosphate100_300_5savgol_mean,phosphate100_300_11savgol_mean,phosphate100_300_25savgol_mean,phosphate100_300_51savgol_mean,phosphate100_300_symiirorder1_mean,phosphate100_300_symiirorder2_mean,phosphate100_300_decimate_mean,phosphate100_300_detrend_mean,phosphate100_300_argrelmin_count,phosphate100_300_argrelmax_count,phosphate100_300_argrelminmax_count,phosphate100_300_mean_change_argrelmin,phosphate100_300_mean_change_argrelmax,phosphate100_300_mean_change_argrelmin_rate,phosphate100_300_mean_change_argrelmax_rate,phosphate100_300_peak_count,phosphate100_300_mean_change_peak,phosphate100_300_mean_change_peak_rate,phosphate100_300_peak_min,phosphate100_300_peak_max,phosphate100_300_peak_mean,phosphate100_300_peak_width_min,phosphate100_300_peak_width_max,phosphate100_300_peak_width_mean,phosphate100_300_peak_max_to_m in,phosphate100_300_peak_width_max_to_min,temperatureSurface_neighbor_mean,temperatureSurface_neighbor_max,temperatureSurface_neighbor_min,temperatureSurface_neighbor_std,temperature100_300_neighbor_mean,temperature100_300_neighbor_max,temperature100_300_neighbor_min,temperature100_300_neighbor_std,salinitySurface_neighbor_mean,salinitySurface_neighbor_max,salinitySurface_neighbor_min,salinitySurface_neighbor_std,salinity100_300_neighbor_mean,salinity100_300

# Methods for outlier Detection

In [7]:
def outlier_replacement(dataframe,method="quantiles"):
    data = dataframe.copy()
    if method=="quantiles":
        down_quantiles = data.quantile(0.05)
        up_quantiles = data.quantile(0.95)
        outliers_low = (data<down_quantiles)
        outliers_high = (data>up_quantiles) 
        data[outliers_low] = np.nan
        data.fillna(down_quantiles, inplace=True)
        data[outliers_high] = np.nan
        data.fillna(up_quantiles, inplace=True)
    elif method == "mad":
        threshold = 3
        mad = scipy.stats.median_abs_deviation(data,scale=0.67449) #median absolute deviation
        median = data.median()
        score = (data-median)/mad
        outliers = abs(score)>3
        outliers_low = (score<-3)
        outliers_high = (score>3)
        data[outliers_low] = np.nan
        data.fillna(data[~outliers].min(), inplace=True)
        data[outliers_high] = np.nan
        data.fillna(data[~outliers].max(), inplace=True)
    elif method == "iqr":
        q1 = data.quantile(0.25)
        q3 = data.quantile(0.75)
        iqr = q3-q1
        thresh_low  = q1-1.5*iqr
        thresh_high = q3+1.5*iqr
        data[data<thresh_low] = np.nan
        data.fillna(thresh_low,inplace=True)
        data[data>thresh_high] = np.nan
        data.fillna(thresh_high,inplace=True)
    else:
        print("Method not valid")
    return data

# Standarization Normalization

In [8]:
def stand_norm(train_dataframe,test_dataframe,method = None ):
    x = train_dataframe.values #returns a numpy array
    x_test = test_dataframe.values
    if method == None:
        return dataframe
    elif method == "MinMaxScale":
        scale_norm = preprocessing.MinMaxScaler()
    elif method == "StandardScale":
        scale_norm = preprocessing.StandardScaler()
    elif method == "Normalize":
        scale_norm = preprocessing.Normalizer()
    else:
        print("Method not valid")
    x_scaled = scale_norm.fit_transform(x)
    x_test_scaled = scale_norm.transform(x_test)
    x_data = pd.DataFrame(x_scaled)
    x_test_data = pd.DataFrame(x_test_scaled)
    return x_data,x_test_data

# Feature Selection
Sources:
1. https://heartbeat.fritz.ai/hands-on-with-feature-selection-techniques-an-introduction-1d8dc6d86c16
2. https://towardsdatascience.com/feature-selection-with-pandas-e3690ad8504b

## quasi_constant_features

In [9]:
def remove_const():
    threshold = 0.98
    # create empty list
    quasi_constant_features = []
    # loop over all the columns
    for feature in train.columns:
        # calculate the ratio.
        predominant = (train[feature].value_counts() / np.float(len(train))).sort_values(ascending=False).values[0]
        # append the column name if it is bigger than the threshold
        if predominant >= threshold:
            quasi_constant_features.append(feature)   
    print(quasi_constant_features)
    qc_features = train[quasi_constant_features].copy()
    # Checking for colleration
    # qc_features['Overall Probability'] = train['Overall Probability'].copy()
    # qc_features
    # corr_mat1 = qc_features.corr().abs()
    # # Find most important features relative to target
    # cor1_target = corr_mat1['Overall Probability'].sort_values(kind="quicksort", ascending=False)
    # print(cor1_target)
    train.drop(labels = quasi_constant_features,axis=1,inplace = True)
    test.drop(labels = quasi_constant_features,axis=1,inplace = True)
    print("There are ",len(quasi_constant_features),"quasi_constant_features that have been removed")

In [10]:
remove_const()

['temperatureSurface_2spline_mean', 'temperatureSurface_3spline_mean', 'temperatureSurface_4spline_mean', 'temperatureSurface_2gauss_spline_mean', 'temperatureSurface_3gauss_spline_mean', 'temperatureSurface_4gauss_spline_mean', 'temperatureSurface_detrend_mean', 'temperature100_300_2spline_mean', 'temperature100_300_3spline_mean', 'temperature100_300_4spline_mean', 'temperature100_300_2gauss_spline_mean', 'temperature100_300_3gauss_spline_mean', 'temperature100_300_4gauss_spline_mean', 'temperature100_300_detrend_mean', 'salinitySurface_2spline_mean', 'salinitySurface_3spline_mean', 'salinitySurface_4spline_mean', 'salinitySurface_2gauss_spline_mean', 'salinitySurface_3gauss_spline_mean', 'salinitySurface_4gauss_spline_mean', 'salinitySurface_detrend_mean', 'salinity100_300_2spline_mean', 'salinity100_300_3spline_mean', 'salinity100_300_4spline_mean', 'salinity100_300_2gauss_spline_mean', 'salinity100_300_3gauss_spline_mean', 'salinity100_300_4gauss_spline_mean', 'salinity100_300_detr

## Dublicates

In [11]:
def remove_duplicates():
    train_features_T = train.T
    # select the duplicated features columns names
    duplicated_columns = train_features_T[train_features_T.duplicated()].index.values
    print(duplicated_columns)
    train.drop(labels=duplicated_columns, axis=1, inplace=True)
    test.drop(labels=duplicated_columns, axis=1, inplace=True)
    print("There are ",train_features_T.duplicated().sum(),"Dublicates that have been removed")

In [12]:
remove_duplicates()

['meridionalCurrentSurface_years_mean'
 'meridionalCurrent100_300_years_mean' 'zonalCurrent100_300_years_mean'
 'chlorophyll_years_mean' 'wave_Height_years_mean'
 'nitrateSurface_years_mean' 'phosphateSurface_years_mean'
 'phosphate100_300_years_mean' 'temperatureSurface_Moving_median_2_mean'
 'temperature100_300_Moving_median_2_mean'
 'salinitySurface_Moving_median_2_mean'
 'salinity100_300_Moving_median_2_mean'
 'dissolvedOxygenSurface_Moving_median_2_mean'
 'dissolvedOxygen100_300_Moving_median_2_mean'
 'meridionalCurrentSurface_Moving_median_2_mean'
 'meridionalCurrent100_300_Moving_median_2_mean'
 'zonalCurrent100_300_Moving_median_2_mean'
 'chlorophyll_Moving_median_2_mean' 'euphoticDepth_Moving_median_2_mean'
 'secchiDiskDepth_Moving_median_2_mean' 'wave_Height_Moving_median_2_mean'
 'nitrate100_300_Moving_median_2_mean'
 'phosphateSurface_Moving_median_2_mean'
 'phosphate100_300_Moving_median_2_mean' 'temperatureSurface_abs_trend'
 'temperatureSurface_peak_count' 'temperatureSu

### Correlated Features

In [13]:
 def remove_high_correl_features(method = "pearson"):
    # creating set to hold the correlated features
    corr_features = set()
    # create the correlation matrix (default to pearson)
    if method=="pearson":
        corr_matrix = train.corr()
        for i in range(len(corr_matrix .columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > 0.9:
                    colname = corr_matrix.columns[i]
                    corr_features.add(colname)
        print("There are ",len( corr_features)," correlated features using",method," colleration")
        train.drop(labels=corr_features, axis=1, inplace=True)
        test.drop(labels=corr_features, axis=1, inplace=True)
        print(corr_features)
    elif method=="kendall":
        corr_matrix = train.corr("kendall")
        for i in range(len(corr_matrix .columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > 0.85:
                    colname = corr_matrix.columns[i]
                    corr_features.add(colname)
        print("There are ",len( corr_features)," correlated features using",method," colleration")
        train.drop(labels=corr_features, axis=1, inplace=True)
        test.drop(labels=corr_features, axis=1, inplace=True)
        print(corr_features)
    elif method=="spearman":
        corr_matrix = train.corr("spearman")
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > 0.95:
                    colname = corr_matrix.columns[i]
                    corr_features.add(colname)
        print("There are ",len( corr_features)," correlated features using",method," colleration")
        print(corr_features)
        train.drop(labels=corr_features, axis=1, inplace=True)
        test.drop(labels=corr_features, axis=1, inplace=True)
    else:
        print("Method not valid") 
    # optional: display a heatmap of the correlation matrix
    # plt.figure(figsize=(11,11))
    # sns.heatmap(corr_matrix)

In [14]:
#remove_high_correl_features("kendall")

In [15]:
kendall_hick_corr_feat = ['zonalCurrent100_300_Moving_quantile_10_12', 'nitrate100_300_September_min', 'temperature100_300_Moving_quantile_10_8', 'nitrate100_300_Moving_average_6_mean', 'meridionalCurrentSurface_25medfilt_mean', 'secchiDiskDepth_Moving_skew_9_mean', 'nitrateSurface_August_quantile_1', 'nitrateSurface_years_quantile_95', 'nitrate100_300_March_quantile_1', 'salinitySurface_September_quantile_99', 'temperatureSurface_Moving_quantile_90_12', 'salinity100_300_Moving_quantile_90_12', 'phosphateSurface_Moving_quantile_95_12', 'salinity100_300_Moving_skew_11_mean', 'meridionalCurrentSurface_July_quantile_95', 'dissolvedOxygen100_300trim_mean_20%', 'salinitySurface_Moving_std_3_mean', 'meridionalCurrentSurface_Moving_quantile_90_8', 'meridionalCurrent100_300_Moving_quantile_1_8', 'salinitySurface_5savgol_mean', 'meridionalCurrent100_300_4spline_mean', 'meridionalCurrentSurface_July_median', 'zonalCurrent100_300_2spline_mean', 'chlorophyll_May_quantile_1', 'temperatureSurface_Moving_quantile_5_8', 'meridionalCurrentSurface_Moving_quantile_1_6', 'dissolvedOxygenSurface_Moving_min_7_mean', 'nitrate100_300trim_mean_10%', 'phosphateSurface_Moving_quantile_1_9', 'temperature100_300_May_median', 'meridionalCurrent100_300_February_quantile_1', 'temperatureSurface_September_min', 'zonalCurrent100_300_May_quantile_95', 'nitrate100_300_Moving_max_7_mean', 'meridionalCurrent100_300_June_quantile_95', 'zonalCurrentSurface_Moving_average_9_mean', 'dissolvedOxygenSurface_January_quantile_90', 'nitrate100_300_years_quantile_1', 'temperatureSurface_June_median', 'temperatureSurface_January_quantile_95', 'salinitySurface_December_median', 'wave_Height_Moving_quantile_90_2', 'phosphateSurface_April_quantile_99', 'meridionalCurrentSurface_Moving_max_6_mean', 'zonalCurrent100_300_Moving_max_3_mean', 'salinity100_300_Moving_max_5_mean', 'euphoticDepth_June_quantile_99', 'meridionalCurrentSurface_May_quantile_1', 'salinitySurface_Moving_quantile_5_8', 'dissolvedOxygen100_300_October_mean', 'nitrateSurface_Moving_max_6_mean', 'zonalCurrent100_300_Moving_skew_9_mean', 'wave_Height_Moving_average_12_mean', 'chlorophyll_March_quantile_99', 'temperature100_300_July_min', 'nitrate100_300_July_median', 'nitrateSurface_Moving_quantile_90_5', 'euphoticDepth_June_quantile_10', 'nitrate100_300_Moving_quantile_99_2', 'temperatureSurface_March_quantile_95', 'meridionalCurrent100_300_December_median', 'meridionalCurrentSurface_Moving_max_4_mean', 'salinitySurface_Moving_quantile_10_7', 'zonalCurrent100_300_Expanding_quantile_99', 'nitrate100_300_symiirorder2_mean', 'salinity100_300_Expanding_average_mean', 'dissolvedOxygenSurface_20wiener_mean', 'chlorophyll_April_quantile_99', 'dissolvedOxygenSurface_max_to_min', 'chlorophyll_Moving_quantile_99_2', 'secchiDiskDepth_November_quantile_1', 'chlorophyll_September_max', 'zonalCurrentSurface_Moving_median_7_mean', 'nitrate100_300_4gauss_spline_mean', 'phosphateSurface_Moving_quantile_1_12', 'chlorophyll_July_quantile_10', 'wave_Height_October_quantile_95', 'meridionalCurrent100_300_Moving_max_11_mean', 'dissolvedOxygen100_300_Moving_median_11_mean', 'nitrateSurface_Moving_quantile_10_9', 'nitrateSurface_June_mean', 'zonalCurrentSurface_Moving_quantile_10_3', 'temperature100_300_Expanding_min_mean', 'temperature100_300_August_min', 'temperature100_300_August_quantile_1', 'wave_Height_April_quantile_10', 'phosphate100_300_October_median', 'secchiDiskDepth_Moving_quantile_1_11', 'nitrateSurface_Expanding_quantile_1', 'dissolvedOxygenSurface_Moving_std_6_mean', 'nitrateSurface_Moving_quantile_90_7', 'salinitySurface_November_median', 'temperature100_300_Moving_quantile_10_7', 'nitrate100_300_Moving_min_9_mean', 'salinity100_300_Moving_quantile_95_3', 'temperatureSurface_Moving_quantile_5_7', 'dissolvedOxygenSurface_May_quantile_10', 'chlorophylltrim_mean_10%', 'secchiDiskDepth_Moving_std_7_mean', 'salinity100_300_Moving_quantile_10_5', 'chlorophyll_Moving_median_11_mean', 'phosphateSurface_March_median', 'dissolvedOxygenSurface_Moving_skew_10_mean', 'euphoticDepth_August_quantile_90', 'zonalCurrent100_300_Moving_std_9_mean', 'dissolvedOxygen100_300_Moving_quantile_90_3', 'meridionalCurrentSurface_Moving_median_5_mean', 'dissolvedOxygen100_300_September_quantile_95', 'secchiDiskDepth_December_quantile_90', 'dissolvedOxygen100_300_Moving_average_11_mean', 'zonalCurrentSurface_November_quantile_95', 'euphoticDepth_years_std', 'temperature100_300_November_mean', 'dissolvedOxygen100_300_July_quantile_10', 'zonalCurrentSurface_September_quantile_10', 'secchiDiskDepth_October_quantile_1', 'salinitySurface_September_quantile_5', 'nitrate100_300_Moving_median_9_mean', 'nitrate100_300_4spline_mean', 'temperatureSurface_Moving_quantile_90_6', 'salinity100_300_January_quantile_95', 'meridionalCurrent100_300_Moving_quantile_90_5', 'euphoticDepth_Moving_quantile_99_4', 'salinitySurface_January_quantile_10', 'temperatureSurface_Hann_window_mean', 'zonalCurrent100_300_years_max', 'dissolvedOxygen100_300_mean_change_argrelmax', 'salinity100_300_Expanding_std_mean', 'euphoticDepth_July_max', 'dissolvedOxygen100_300_December_quantile_5', 'chlorophyll_March_quantile_10', 'nitrate100_300_years_min', 'phosphate100_300_March_quantile_90', 'dissolvedOxygen100_300_Expanding_max_mean', 'wave_Heighttrim_mean_25%', 'meridionalCurrent100_300_Moving_quantile_5_9', 'phosphateSurface_Moving_quantile_1_4', 'zonalCurrent100_300_Moving_median_11_mean', 'zonalCurrentSurface_October_quantile_99', 'temperature100_300_Moving_quantile_1_6', 'zonalCurrent100_300_Expanding_min_mean', 'euphoticDepth_August_quantile_5', 'euphoticDepth_years_quantile_1', 'wave_Height_July_quantile_10', 'nitrateSurface_Moving_quantile_90_10', 'zonalCurrentSurface_October_median', 'nitrateSurface_mean_change_argrelmax', 'nitrateSurface_Moving_quantile_1_5', 'nitrateSurface_February_quantile_5', 'chlorophyll_Moving_average_10_mean', 'wave_Height_years_std', 'wave_Height_Moving_quantile_1_2', 'secchiDiskDepth_Moving_quantile_95_6', 'secchiDiskDepth_June_quantile_95', 'nitrate100_300_50medfilt_mean', 'nitrateSurface_Moving_quantile_95_7', 'zonalCurrentSurface_Expanding_quantile_10', 'phosphate100_300_Moving_average_5_mean', 'euphoticDepth_March_quantile_95', 'temperature100_300_Expanding_max_mean', 'phosphateSurface_November_median', 'temperatureSurface_Moving_quantile_95_9', 'secchiDiskDepth_Moving_max_10_mean', 'zonalCurrentSurface_Moving_quantile_5_7', 'dissolvedOxygenSurface_June_quantile_5', 'secchiDiskDepth_December_quantile_10', 'nitrate100_300_February_max', 'wave_Height_Moving_std_10_mean', 'phosphate100_300_Moving_std_10_mean', 'meridionalCurrentSurface_quantile_10', 'temperature100_300trim_mean_10%', 'nitrateSurface_May_quantile_10', 'phosphate100_300_May_median', 'salinity100_300_Moving_min_2_mean', 'salinitySurface_years_min', 'salinitySurface_July_median', 'chlorophyll_Moving_quantile_10_4', 'wave_Height_Moving_quantile_1_10', 'nitrateSurface_Moving_quantile_10_5', 'nitrate100_300_January_mean', 'nitrateSurface_years_min', 'wave_Height_Moving_quantile_5_6', 'salinity100_300_Moving_average_5_mean', 'wave_Height_July_quantile_99', 'dissolvedOxygen100_300_Moving_std_10_mean', 'temperatureSurface_February_min', 'wave_Height_December_quantile_5', 'nitrate100_300trim_mean_25%', 'chlorophyll_Moving_max_9_mean', 'nitrateSurface_peak_max', 'temperatureSurface_June_quantile_10', 'nitrateSurface_February_quantile_99', 'phosphate100_300_March_mean', 'nitrateSurfacetrim_mean_20%', 'dissolvedOxygen100_300_December_quantile_10', 'temperatureSurface_Moving_quantile_10_10', 'phosphate100_300_11savgol_mean', 'secchiDiskDepth_Expanding_quantile_90', 'nitrate100_300_March_quantile_10', 'dissolvedOxygenSurface_Moving_min_9_mean', 'meridionalCurrentSurface_Moving_min_12_mean', 'temperature100_300_Expanding_quantile_5', 'salinity100_300_years_median', 'euphoticDepth_mean_change_argrelmax', 'salinity100_300_May_mean', 'dissolvedOxygenSurface_years_std', 'phosphate100_300_October_min', 'salinity100_300_Moving_median_7_mean', 'nitrateSurface_Moving_quantile_90_9', 'salinity100_300_December_quantile_1', 'phosphate100_300_Moving_median_8_mean', 'nitrate100_300iqr1', 'nitrateSurface_Moving_std_7_mean', 'dissolvedOxygenSurface_10wiener_mean', 'phosphateSurface_February_quantile_1', 'meridionalCurrentSurface_Moving_quantile_95_2', 'salinity100_300_quantile_95', 'phosphate100_300_October_quantile_5', 'secchiDiskDepth_January_quantile_99', 'chlorophyll_June_median', 'zonalCurrent100_300_Moving_skew_10_mean', 'nitrate100_300_April_quantile_5', 'euphoticDepth_Moving_quantile_95_9', 'salinitySurface_Moving_max_12_mean', 'zonalCurrent100_300_May_quantile_10', 'nitrate100_300_December_quantile_95', 'wave_Height_Moving_quantile_90_4', 'zonalCurrentSurface_Moving_quantile_99_3', 'dissolvedOxygen100_300_Moving_quantile_1_8', 'nitrate100_300_mean_change_argrelmax', 'phosphate100_300_Moving_min_6_mean', 'zonalCurrent100_300_peak_mean', 'chlorophyll_Moving_average_2_mean', 'secchiDiskDepth_Moving_quantile_5_4', 'dissolvedOxygen100_300_11savgol_mean', 'phosphate100_300_April_quantile_5', 'dissolvedOxygen100_300_March_quantile_90', 'phosphateSurface_Moving_quantile_90_4', 'phosphateSurface_Moving_quantile_10_6', 'nitrate100_300_Moving_median_7_mean', 'zonalCurrent100_300_Expanding_quantile_10', 'euphoticDepth_Moving_quantile_90_8', 'nitrateSurface_Moving_quantile_1_6', 'salinity100_300_February_mean', 'salinity100_300_Expanding_quantile_5', 'meridionalCurrent100_300_2spline_mean', 'phosphateSurface_November_quantile_90', 'temperature100_300_March_quantile_1', 'temperatureSurface_Moving_std_6_mean', 'temperature100_300_Moving_quantile_10_11', 'phosphate100_300_Moving_max_9_mean', 'meridionalCurrent100_300iqr1', 'dissolvedOxygenSurface_Moving_std_10_mean', 'salinity100_300_mean_change_argrelmin', 'dissolvedOxygenSurface_Moving_quantile_99_5', 'salinity100_300_September_quantile_99', 'meridionalCurrentSurface_Moving_skew_8_mean', 'phosphate100_300_Moving_quantile_10_9', 'wave_Height_Moving_median_6_mean', 'salinity100_300_June_median', 'nitrate100_300_Moving_average_7_mean', 'secchiDiskDepth_August_quantile_10', 'salinity100_300_March_median', 'dissolvedOxygen100_300_April_median', 'dissolvedOxygen100_300_Moving_quantile_10_6', 'zonalCurrent100_300_Moving_min_8_mean', 'dissolvedOxygenSurface_October_min', 'meridionalCurrentSurface_February_quantile_10', 'meridionalCurrent100_300_September_median', 'nitrate100_300_Moving_skew_12_mean', 'temperatureSurface_August_quantile_5', 'nitrateSurface_Moving_quantile_10_2', 'meridionalCurrentSurface_symiirorder1_mean', 'phosphate100_300trim_mean_15%', 'nitrateSurface_Moving_max_7_mean', 'temperature100_300_Moving_quantile_95_12', 'chlorophyll_June_quantile_99', 'phosphate100_300trim_mean_10%', 'euphoticDepth_Moving_quantile_1_2', 'dissolvedOxygen100_300_January_median', 'dissolvedOxygenSurface_Moving_max_10_mean', 'meridionalCurrentSurface_Moving_median_8_mean', 'chlorophyll_quantile_10', 'nitrate100_300_September_quantile_90', 'wave_Height_March_quantile_1', 'dissolvedOxygenSurface_Moving_min_5_mean', 'zonalCurrent100_300_October_median', 'dissolvedOxygen100_300_Moving_quantile_99_5', 'meridionalCurrentSurface_March_quantile_10', 'salinity100_300_Moving_min_11_mean', 'chlorophyll_Moving_median_8_mean', 'chlorophyll_Moving_quantile_99_5', 'temperature100_300_June_quantile_10', 'dissolvedOxygen100_300_Moving_std_2_mean', 'chlorophyll_November_quantile_1', 'nitrate100_300_October_quantile_90', 'zonalCurrentSurface_Moving_quantile_95_12', 'salinitySurface_December_min', 'dissolvedOxygenSurface_October_quantile_90', 'dissolvedOxygen100_300_Moving_min_8_mean', 'zonalCurrentSurface_Moving_max_12_mean', 'salinitySurface_November_quantile_1', 'meridionalCurrentSurface_Moving_quantile_99_3', 'nitrateSurface_September_quantile_1', 'chlorophyll_Moving_max_8_mean', 'zonalCurrent100_300_51savgol_mean', 'nitrate100_300_Moving_quantile_99_4', 'meridionalCurrent100_300_Moving_max_6_mean', 'nitrateSurface_August_std', 'meridionalCurrentSurface_peak_mean', 'zonalCurrentSurface_Moving_std_8_mean', 'wave_Height_Expanding_quantile_5', 'meridionalCurrent100_300_July_median', 'phosphate100_300_September_median', 'dissolvedOxygen100_300_Moving_quantile_5_7', 'secchiDiskDepth_Moving_quantile_99_4', 'nitrate100_300_Moving_min_12_mean', 'phosphateSurface_May_quantile_1', 'secchiDiskDepth_Moving_quantile_95_5', 'salinitySurface_Moving_quantile_95_7', 'salinitySurface_years_max', 'phosphate100_300_April_quantile_10', 'nitrate100_300_Moving_std_3_mean', 'meridionalCurrentSurface_June_quantile_95', 'nitrate100_300_October_min', 'temperature100_300_Moving_quantile_1_10', 'secchiDiskDepth_Moving_quantile_1_8', 'chlorophyll_Expanding_average_mean', 'phosphateSurface_Moving_quantile_95_5', 'phosphate100_300_Moving_std_4_mean', 'temperatureSurface_Moving_quantile_90_5', 'zonalCurrent100_300_Moving_std_7_mean', 'phosphate100_300_April_max', 'nitrate100_300_Moving_quantile_10_3', 'wave_Height_Expanding_quantile_95', 'zonalCurrentSurface_Moving_quantile_10_12', 'zonalCurrent100_300_Expanding_max_mean', 'phosphate100_300_February_quantile_99', 'salinity100_300_symiirorder2_mean', 'secchiDiskDepth_Moving_quantile_10_8', 'temperature100_300_December_quantile_1', 'dissolvedOxygen100_300_August_mean', 'nitrate100_300_Moving_quantile_5_6', 'chlorophyll_4gauss_spline_mean', 'wave_Height_years_median', 'secchiDiskDepth_Moving_min_10_mean', 'temperature100_300_Moving_quantile_99_7', 'zonalCurrent100_300_Moving_quantile_99_12', 'salinitySurface_Moving_quantile_5_10', 'zonalCurrentSurface_Moving_quantile_99_2', 'zonalCurrent100_300_4gauss_spline_mean', 'wave_Height_10wiener_mean', 'meridionalCurrentSurface_July_quantile_10', 'nitrate100_300_April_quantile_1', 'zonalCurrent100_300_3spline_mean', 'phosphateSurface_September_median', 'phosphate100_300_December_quantile_1', 'nitrate100_300_August_quantile_5', 'meridionalCurrentSurface_years_quantile_1', 'secchiDiskDepth_Moving_std_12_mean', 'dissolvedOxygen100_300_February_median', 'nitrate100_300_July_quantile_10', 'salinitySurface_years_median', 'secchiDiskDepth_November_max', 'temperature100_300_Expanding_median_mean', 'temperature100_300_November_median', 'meridionalCurrent100_300_Moving_quantile_99_3', 'dissolvedOxygenSurface_Moving_quantile_95_5', 'nitrate100_300_August_quantile_1', 'euphoticDepth_Moving_quantile_1_9', 'secchiDiskDepth_mean_change_argrelmin', 'phosphate100_300_Moving_max_7_mean', 'nitrate100_300_Moving_quantile_99_3', 'salinitySurface_neighbor_min', 'nitrateSurface_Moving_min_6_mean', 'phosphate100_300_Moving_std_8_mean', 'salinitySurface_Moving_median_5_mean', 'euphoticDepth_Moving_quantile_99_2', 'salinity100_300_years_quantile_1', 'temperatureSurface_July_quantile_99', 'nitrate100_300_January_quantile_90', 'euphoticDepth_Moving_max_5_mean', 'phosphateSurface_Moving_quantile_90_12', 'dissolvedOxygenSurface_October_median', 'meridionalCurrentSurface_October_quantile_1', 'nitrateSurface_April_quantile_95', 'secchiDiskDepth_Moving_quantile_10_4', 'temperature100_300_Expanding_std_mean', 'temperature100_300_Moving_average_10_mean', 'dissolvedOxygen100_300_quantile_95', 'euphoticDepth_April_quantile_5', 'phosphateSurface_March_quantile_99', 'nitrate100_300_October_quantile_95', 'phosphateSurface_November_quantile_10', 'nitrate100_300_Moving_quantile_1_4', 'dissolvedOxygenSurface_Moving_average_5_mean', 'dissolvedOxygenSurface_peak_max_to_m in', 'nitrateSurface_February_quantile_95', 'dissolvedOxygenSurface_January_quantile_95', 'temperatureSurface_Moving_max_8_mean', 'phosphate100_300_August_quantile_1', 'temperature100_300_Moving_skew_10_mean', 'temperatureSurface_November_quantile_10', 'zonalCurrentSurface_Moving_quantile_5_6', 'dissolvedOxygen100_300_February_quantile_10', 'secchiDiskDepth_3spline_mean', 'meridionalCurrentSurface_November_quantile_95', 'zonalCurrentSurface_Moving_quantile_10_6', 'dissolvedOxygen100_300_years_quantile_1', 'secchiDiskDepth_January_quantile_95', 'phosphateSurface_Moving_quantile_5_11', 'secchiDiskDepth_January_quantile_90', 'dissolvedOxygen100_300_10medfilt_mean', 'zonalCurrentSurface_December_quantile_5', 'phosphate100_300_March_median', 'phosphateSurface_May_max', 'phosphate100_300_April_quantile_95', 'nitrateSurfacetrim_mean_15%', 'chlorophyll_Moving_quantile_5_6', 'euphoticDepth_Moving_quantile_1_6', 'zonalCurrent100_300_November_quantile_1', 'dissolvedOxygenSurface_Expanding_quantile_10', 'chlorophyll_Expanding_min_mean', 'dissolvedOxygen100_300_August_quantile_95', 'euphoticDepth_years_quantile_95', 'secchiDiskDepth_51savgol_mean', 'salinity100_300_Moving_quantile_90_10', 'euphoticDepth_years_min', 'nitrate100_300_August_median', 'dissolvedOxygen100_300_January_quantile_1', 'wave_Height_September_quantile_1', 'salinitySurface_February_quantile_5', 'dissolvedOxygen100_300_September_max', 'temperature100_300_Moving_quantile_90_3', 'nitrate100_300_July_min', 'temperature100_300_February_quantile_10', 'temperature100_300_January_median', 'zonalCurrent100_300_November_quantile_99', 'euphoticDepth_Moving_quantile_90_12', 'secchiDiskDepth_Moving_max_11_mean', 'dissolvedOxygenSurfacetrim_mean_5%', 'temperatureSurface_Moving_quantile_10_11', 'euphoticDepth_argrelminmax_count', 'dissolvedOxygenSurface_Moving_quantile_1_11', 'nitrate100_300_January_median', 'phosphateSurface_Moving_quantile_99_6', 'nitrate100_300_Moving_quantile_95_7', 'salinitySurface_Moving_median_11_mean', 'zonalCurrent100_300_May_quantile_1', 'nitrate100_300_June_min', 'meridionalCurrent100_300_Moving_median_6_mean', 'nitrateSurface_Moving_std_12_mean', 'euphoticDepth_Moving_quantile_90_11', 'salinity100_300_Moving_std_5_mean', 'dissolvedOxygenSurface_Moving_quantile_5_6', 'phosphateSurface_July_quantile_10', 'phosphate100_300_Moving_quantile_1_5', 'secchiDiskDepth_Moving_quantile_5_11', 'meridionalCurrentSurface_Moving_min_9_mean', 'temperature100_300_Moving_min_11_mean', 'temperatureSurface_May_quantile_95', 'phosphateSurface_Moving_quantile_99_8', 'dissolvedOxygen100_300_Hilbert_mean', 'wave_Height_years_quantile_99', 'zonalCurrentSurface_median', 'temperatureSurface_January_quantile_99', 'zonalCurrentSurface_4wiener_mean', 'zonalCurrentSurface_September_quantile_5', 'zonalCurrent100_300_quantile_99', 'phosphateSurface_January_quantile_90', 'euphoticDepth_Moving_quantile_1_11', 'meridionalCurrentSurface_quantile_99', 'phosphate100_300_November_quantile_1', 'zonalCurrent100_300_March_quantile_99', 'phosphateSurface_September_quantile_99', 'wave_Height_Moving_min_3_mean', 'temperatureSurface_Moving_average_7_mean', 'temperature100_300_February_median', 'wave_Height_Moving_quantile_5_7', 'temperature100_300_Moving_quantile_95_2', 'temperature100_300_December_median', 'zonalCurrentSurface_Expanding_quantile_1', 'euphoticDepth_Moving_skew_8_mean', 'secchiDiskDepth_June_min', 'phosphateSurface_5medfilt_mean', 'phosphateSurface_May_median', 'salinity100_300_20wiener_mean', 'phosphate100_300_Moving_std_3_mean', 'phosphate100_300_May_quantile_99', 'nitrate100_300_September_quantile_1', 'temperatureSurface_symiirorder1_mean', 'salinitySurface_Moving_min_6_mean', 'chlorophyll_peak_max', 'euphoticDepth_August_max', 'zonalCurrent100_300_Moving_quantile_1_5', 'chlorophyll_Moving_quantile_5_5', 'temperatureSurface_Moving_quantile_95_11', 'euphoticDepth_Moving_min_10_mean', 'nitrate100_300_Expanding_min_mean', 'wave_Height_June_quantile_95', 'temperature100_300_peak_max', 'salinitySurface_May_mean', 'euphoticDepth_January_quantile_95', 'temperature100_300_May_max', 'phosphate100_300_Moving_quantile_99_11', 'nitrateSurface_3medfilt_mean', 'euphoticDepth_Moving_quantile_95_6', 'phosphateSurface_January_quantile_1', 'temperature100_300_May_quantile_1', 'secchiDiskDepth_symiirorder1_mean', 'dissolvedOxygenSurface_April_quantile_99', 'chlorophyll_Moving_min_4_mean', 'nitrate100_300_Moving_quantile_90_8', 'meridionalCurrentSurface_4gauss_spline_mean', 'zonalCurrentSurface_Expanding_quantile_5', 'salinitySurface_Moving_quantile_5_5', 'wave_Height_December_quantile_99', 'chlorophyll_August_quantile_5', 'meridionalCurrent100_300trim_mean_10%', 'wave_Height_February_quantile_1', 'meridionalCurrentSurface_years_max', 'dissolvedOxygen100_300_quantile_10', 'wave_Height_April_quantile_5', 'zonalCurrent100_300_10medfilt_mean', 'phosphate100_300_June_max', 'euphoticDepth_September_quantile_5', 'meridionalCurrent100_300_symiirorder1_mean', 'meridionalCurrent100_300_March_quantile_10', 'temperature100_300_Moving_skew_7_mean', 'phosphate100_300_February_median', 'euphoticDepth_years_quantile_5', 'secchiDiskDepth_Moving_max_12_mean', 'zonalCurrent100_300_Moving_quantile_90_8', 'salinitySurface_quantile_10', 'zonalCurrent100_300_March_quantile_1', 'nitrateSurface_September_max', 'salinity100_300_June_quantile_10', 'wave_Height_September_std', 'euphoticDepth_3spline_mean', 'secchiDiskDepth_Moving_average_3_mean', 'temperatureSurface_Moving_min_12_mean', 'chlorophyll_years_median', 'nitrateSurface_neighbor_max', 'phosphate100_300_years_quantile_1', 'nitrate100_300_Moving_max_12_mean', 'zonalCurrentSurface_November_quantile_5', 'dissolvedOxygenSurface_August_quantile_5', 'phosphate100_300_February_quantile_10', 'dissolvedOxygen100_300_Moving_std_3_mean', 'temperatureSurface_Moving_min_11_mean', 'meridionalCurrentSurface_2spline_mean', 'salinity100_300_August_median', 'nitrateSurface_October_quantile_99', 'dissolvedOxygen100_300_January_quantile_5', 'nitrate100_300_Moving_quantile_5_3', 'secchiDiskDepth_November_median', 'dissolvedOxygen100_300_Moving_min_10_mean', 'temperatureSurface_Moving_quantile_5_4', 'temperatureSurface_December_quantile_90', 'euphoticDepth_Moving_skew_12_mean', 'salinitySurface_Moving_max_8_mean', 'nitrateSurface_September_quantile_95', 'dissolvedOxygenSurface_Moving_median_4_mean', 'temperature100_300_June_quantile_5', 'wave_Height_Moving_skew_11_mean', 'meridionalCurrentSurface_Moving_quantile_1_9', 'temperatureSurface_Moving_average_3_mean', 'phosphateSurface_Expanding_median_mean', 'euphoticDepth_Moving_median_11_mean', 'nitrate100_300_Hilbert_mean', 'meridionalCurrentSurface_Moving_min_7_mean', 'salinity100_300_February_quantile_95', 'chlorophyll_2gauss_spline_mean', 'salinitySurface_Moving_min_11_mean', 'temperature100_300_Moving_quantile_10_12', 'nitrateSurface_May_max', 'chlorophyll_years_quantile_99', 'meridionalCurrent100_300_August_quantile_10', 'wave_Height_February_quantile_95', 'nitrateSurface_Moving_max_5_mean', 'dissolvedOxygen100_300_November_quantile_10', 'temperatureSurface_Moving_quantile_90_3', 'dissolvedOxygen100_300_June_quantile_99', 'zonalCurrent100_300_December_quantile_99', 'chlorophyll_Moving_average_12_mean', 'nitrate100_300_Moving_median_5_mean', 'temperature100_300_September_median', 'temperature100_300iqr', 'salinity100_300_Moving_min_3_mean', 'nitrateSurface_September_std', 'meridionalCurrentSurfacetrim_mean_15%', 'nitrate100_300_Moving_quantile_10_5', 'phosphate100_300_Moving_quantile_10_8', 'salinity100_300_October_quantile_1', 'meridionalCurrent100_300_Moving_std_3_mean', 'zonalCurrentSurface_Moving_min_4_mean', 'meridionalCurrentSurface_Moving_median_10_mean', 'euphoticDepth_September_min', 'meridionalCurrent100_300_Moving_quantile_10_8', 'salinitySurface_Moving_quantile_90_10', 'dissolvedOxygen100_300_Moving_quantile_5_6', 'salinitySurface_Moving_quantile_5_11', 'temperatureSurface_Moving_skew_8_mean', 'nitrateSurface_Moving_min_10_mean', 'zonalCurrentSurface_Moving_median_8_mean', 'phosphate100_300_Expanding_std_mean', 'salinity100_300_Moving_quantile_1_7', 'temperatureSurface_September_quantile_90', 'salinitySurface_Moving_median_6_mean', 'zonalCurrent100_300_December_quantile_5', 'euphoticDepth_May_quantile_95', 'salinity100_300_July_mean', 'dissolvedOxygen100_300_Moving_quantile_5_9', 'wave_Height_Moving_std_6_mean', 'nitrateSurface_May_quantile_95', 'meridionalCurrentSurface_25savgol_mean', 'phosphate100_300_Moving_quantile_99_7', 'chlorophyll_October_quantile_5', 'temperature100_300_Moving_quantile_95_10', 'meridionalCurrentSurface_January_median', 'secchiDiskDepth_March_median', 'salinity100_300_July_max', 'dissolvedOxygenSurface_mad', 'wave_Height_Moving_quantile_95_8', 'euphoticDepth_Moving_quantile_95_12', 'nitrateSurface_Moving_quantile_10_4', 'phosphateSurface_Expanding_quantile_90', 'temperatureSurface_July_quantile_90', 'phosphateSurface_Moving_quantile_90_9', 'phosphateSurface_argrelmax_count', 'salinitySurface_February_quantile_90', 'salinity100_300_September_max', 'nitrate100_300_November_quantile_90', 'chlorophyll_December_median', 'zonalCurrentSurface_Moving_min_5_mean', 'meridionalCurrent100_300_October_quantile_10', 'wave_Height_Moving_average_3_mean', 'euphoticDepth_Moving_quantile_10_7', 'phosphateSurface_2gauss_spline_mean', 'temperature100_300_Moving_average_6_mean', 'wave_Height_Moving_skew_8_mean', 'nitrate100_300_November_quantile_5', 'salinitySurface_October_quantile_10', 'temperatureSurface_Moving_quantile_5_3', 'salinitySurface_December_max', 'chlorophyll_argrelmax_count', 'phosphateSurface_August_quantile_90', 'temperatureSurface_Expanding_min_mean', 'meridionalCurrentSurface_Moving_quantile_1_11', 'meridionalCurrent100_300_Moving_quantile_10_7', 'dissolvedOxygenSurface_Moving_quantile_95_11', 'meridionalCurrent100_300_median', 'dissolvedOxygen100_300_Moving_quantile_95_6', 'nitrate100_300_quantile_99', 'meridionalCurrentSurface_January_quantile_99', 'nitrate100_300_September_quantile_95', 'euphoticDepth_4wiener_mean', 'nitrateSurface_November_std', 'salinity100_300_Moving_max_6_mean', 'phosphateSurface_February_median', 'dissolvedOxygenSurface_Moving_quantile_5_8', 'zonalCurrent100_300_median', 'secchiDiskDepth_years_std', 'secchiDiskDepth_mad', 'salinity100_300_Moving_max_11_mean', 'zonalCurrentSurface_peak_max', 'zonalCurrent100_300_January_quantile_1', 'dissolvedOxygenSurface_Moving_std_3_mean', 'salinitySurface_April_quantile_99', 'dissolvedOxygenSurface_March_quantile_5', 'euphoticDepth_Moving_quantile_10_10', 'euphoticDepth_Expanding_quantile_1', 'nitrateSurface_Moving_average_9_mean', 'phosphateSurface_4gauss_spline_mean', 'secchiDiskDepth_March_mean', 'secchiDiskDepth_Moving_quantile_1_5', 'meridionalCurrent100_300_Moving_average_8_mean', 'zonalCurrentSurface_25medfilt_mean', 'temperature100_300_February_quantile_90', 'meridionalCurrent100_300_quantile_10', 'zonalCurrent100_300_Moving_average_10_mean', 'dissolvedOxygen100_300_Moving_quantile_1_10', 'salinitySurface_January_max', 'secchiDiskDepth_April_quantile_1', 'euphoticDepth_years_mean', 'wave_Height_Moving_quantile_90_11', 'chlorophyll_Moving_quantile_90_4', 'salinity100_300_February_quantile_90', 'phosphate100_300_Moving_quantile_10_4', 'temperatureSurface_August_min', 'dissolvedOxygen100_300_November_quantile_1', 'chlorophyll_Moving_quantile_95_10', 'euphoticDepth_Moving_std_12_mean', 'euphoticDepth_quantile_95', 'phosphate100_300_Moving_quantile_10_6', 'zonalCurrentSurface_years_quantile_1', 'dissolvedOxygenSurface_Moving_std_7_mean', 'salinitySurface_March_median', 'temperature100_300_May_quantile_99', 'phosphateSurface_Moving_quantile_99_9', 'wave_Height_February_std', 'dissolvedOxygenSurface_Moving_quantile_10_12', 'euphoticDepth_Moving_min_9_mean', 'meridionalCurrentSurface_November_quantile_10', 'wave_Height_Moving_quantile_1_4', 'dissolvedOxygenSurface_June_quantile_90', 'dissolvedOxygenSurface_Moving_quantile_99_9', 'meridionalCurrentSurface_Moving_quantile_99_5', 'temperatureSurface_Moving_max_10_mean', 'salinity100_300_Moving_average_7_mean', 'meridionalCurrent100_300_Moving_quantile_95_2', 'salinity100_300_March_mean', 'wave_Height_Moving_average_6_mean', 'phosphate100_300_4spline_mean', 'temperature100_300_April_max', 'wave_Height_Moving_average_2_mean', 'phosphateSurface_Moving_quantile_95_7', 'chlorophyll_January_median', 'temperature100_300_Moving_quantile_10_9', 'dissolvedOxygenSurface_February_median', 'secchiDiskDepth_4gauss_spline_mean', 'wave_Height_Moving_quantile_99_5', 'meridionalCurrent100_300_max_to_min_diff', 'phosphateSurface_years_min', 'phosphateSurface_January_quantile_5', 'temperatureSurface_October_quantile_1', 'dissolvedOxygen100_300_quantile_5', 'euphoticDepth_Moving_skew_9_mean', 'phosphateSurface_Moving_std_9_mean', 'salinity100_300_July_quantile_99', 'salinitySurface_Moving_min_9_mean', 'salinitySurface_December_quantile_99', 'dissolvedOxygen100_300_25savgol_mean', 'meridionalCurrent100_300_years_quantile_95', 'salinity100_300_quantile_90', 'salinity100_300_October_max', 'phosphateSurface_November_quantile_99', 'dissolvedOxygen100_300_Moving_skew_8_mean', 'meridionalCurrentSurface_mean_change_argrelmin', 'nitrateSurface_Moving_quantile_10_10', 'salinity100_300_Moving_min_7_mean', 'phosphate100_300_Moving_skew_5_mean', 'secchiDiskDepth_Moving_quantile_10_3', 'temperature100_300_Moving_max_11_mean', 'euphoticDepth_Moving_std_8_mean', 'temperatureSurfacetrim_mean_15%', 'salinity100_300_Moving_skew_6_mean', 'salinity100_300_Moving_skew_12_mean', 'salinity100_300_August_quantile_1', 'zonalCurrentSurface_October_quantile_95', 'nitrate100_300_August_quantile_90', 'nitrateSurface_Moving_quantile_99_7', 'dissolvedOxygenSurface_November_quantile_10', 'zonalCurrent100_300_Moving_min_6_mean', 'phosphateSurface_March_quantile_95', 'meridionalCurrent100_300_January_quantile_10', 'salinity100_300_Moving_max_4_mean', 'temperature100_300_March_quantile_10', 'salinitySurface_Moving_quantile_99_9', 'phosphate100_300_Moving_quantile_1_10', 'secchiDiskDepth_Moving_max_6_mean', 'phosphate100_300_Moving_quantile_10_5', 'euphoticDepth_Moving_quantile_5_6', 'nitrateSurface_July_std', 'salinitySurface_Moving_quantile_1_9', 'nitrateSurface_Moving_std_4_mean', 'phosphateSurface_Moving_max_8_mean', 'secchiDiskDepth_Moving_min_9_mean', 'dissolvedOxygenSurface_Moving_max_2_mean', 'meridionalCurrentSurface_May_quantile_95', 'salinitySurfacetrim_mean_5%', 'dissolvedOxygen100_300_Moving_quantile_99_11', 'nitrate100_300_Moving_quantile_90_11', 'temperature100_300_5medfilt_mean', 'temperature100_300_years_std', 'salinity100_300_Moving_quantile_95_5', 'euphoticDepth_March_quantile_5', 'temperature100_300_Expanding_quantile_90', 'nitrate100_300_June_quantile_99', 'meridionalCurrentSurface_Moving_quantile_95_6', 'euphoticDepth_Moving_std_11_mean', 'temperatureSurface_Moving_quantile_90_7', 'nitrateSurfaceiqr1', 'temperature100_300_July_quantile_5', 'nitrate100_300_June_quantile_95', 'temperatureSurface_Moving_average_12_mean', 'euphoticDepth_February_median', 'zonalCurrentSurface_Moving_quantile_99_10', 'phosphate100_300_25savgol_mean', 'salinity100_300_Moving_max_3_mean', 'nitrateSurface_4gauss_spline_mean', 'nitrate100_300_Moving_max_10_mean', 'salinity100_300_Hann_window_mean', 'temperature100_300_Moving_min_7_mean', 'zonalCurrent100_300_Moving_min_4_mean', 'nitrate100_300_February_quantile_90', 'salinitySurface_April_quantile_10', 'phosphateSurface_4spline_mean', 'meridionalCurrent100_300_Moving_median_7_mean', 'wave_Height_Moving_min_11_mean', 'euphoticDepth_February_quantile_10', 'phosphateSurface_April_quantile_1', 'phosphateSurface_June_quantile_1', 'salinitySurface_Expanding_median_mean', 'zonalCurrent100_300_November_quantile_95', 'temperatureSurface_May_quantile_90', 'nitrateSurface_years_max', 'meridionalCurrent100_300_25medfilt_mean', 'salinity100_300_mean_change_peak', 'dissolvedOxygenSurface_Moving_quantile_5_10', 'nitrate100_300_Moving_quantile_10_4', 'zonalCurrentSurface_Moving_quantile_95_3', 'zonalCurrentSurface_10medfilt_mean', 'salinitySurface_September_quantile_1', 'zonalCurrentSurface_years_median', 'phosphateSurface_Moving_average_4_mean', 'zonalCurrent100_300_Moving_quantile_1_9', 'phosphate100_300_years_std', 'temperature100_300_Moving_quantile_95_4', 'secchiDiskDepth_Moving_quantile_90_5', 'zonalCurrent100_300_Moving_quantile_10_7', 'chlorophyll_Moving_quantile_1_5', 'euphoticDepth_October_quantile_1', 'phosphate100_300_Moving_quantile_5_11', 'zonalCurrentSurface_Moving_min_3_mean', 'dissolvedOxygen100_300_Moving_quantile_1_5', 'chlorophyll_October_mean', 'phosphateSurface_November_quantile_5', 'temperature100_300_Moving_quantile_99_3', 'chlorophyll_Moving_average_4_mean', 'meridionalCurrent100_300_September_quantile_10', 'temperatureSurface_July_quantile_5', 'nitrate100_300_Moving_min_5_mean', 'salinitySurface_Moving_quantile_90_8', 'salinitySurface_Moving_skew_12_mean', 'salinitySurface_Moving_std_8_mean', 'salinitySurface_max_to_min', 'secchiDiskDepth_Expanding_quantile_1', 'phosphate100_300_Expanding_quantile_95', 'zonalCurrent100_300_July_quantile_95', 'meridionalCurrentSurface_Hilbert_mean', 'euphoticDepth_March_quantile_10', 'wave_Height_4gauss_spline_mean', 'phosphate100_300_July_quantile_90', 'phosphateSurface_Moving_quantile_99_3', 'zonalCurrent100_300_Moving_average_3_mean', 'euphoticDepth_Moving_quantile_95_8', 'phosphate100_300_October_quantile_1', 'nitrate100_300_Moving_min_7_mean', 'chlorophyll_Moving_quantile_1_6', 'dissolvedOxygen100_300_September_quantile_1', 'chlorophyll_max_to_min_diff', 'meridionalCurrentSurface_Moving_quantile_99_12', 'zonalCurrent100_300_Moving_max_6_mean', 'dissolvedOxygen100_300_Moving_quantile_5_11', 'dissolvedOxygenSurface_years_quantile_1', 'zonalCurrent100_300_peak_max', 'wave_Height_Expanding_quantile_1', 'zonalCurrentSurface_3spline_mean', 'nitrateSurface_Expanding_quantile_10', 'salinitySurface_June_median', 'salinity100_300_Moving_std_2_mean', 'chlorophyll_Moving_max_11_mean', 'euphoticDepth_December_mean', 'wave_Height_Moving_skew_9_mean', 'meridionalCurrent100_300_November_quantile_10', 'temperature100_300_5savgol_mean', 'temperatureSurface_February_quantile_95', 'phosphateSurface_Moving_quantile_10_9', 'dissolvedOxygen100_300_Expanding_median_mean', 'nitrate100_300_November_quantile_99', 'phosphateSurface_25savgol_mean', 'nitrateSurface_June_quantile_95', 'salinity100_300_years_mean', 'euphoticDepth_July_quantile_90', 'zonalCurrentSurface_January_quantile_95', 'nitrateSurface_August_quantile_95', 'wave_Height_5medfilt_mean', 'salinitySurface_February_mean', 'euphoticDepth_argrelmax_count', 'wave_Height_January_quantile_1', 'phosphateSurface_February_quantile_5', 'phosphate100_300_June_quantile_5', 'dissolvedOxygenSurface_quantile_95', 'salinitySurface_Moving_max_2_mean', 'chlorophyll_Expanding_quantile_10', 'phosphateSurface_Hilbert_mean', 'temperature100_300_Moving_std_6_mean', 'temperatureSurface_February_median', 'temperatureSurface_5medfilt_mean', 'phosphate100_300_July_quantile_10', 'phosphate100_300_August_quantile_95', 'nitrateSurface_2gauss_spline_mean', 'temperatureSurface_Expanding_quantile_1', 'wave_Height_Moving_median_3_mean', 'temperature100_300_Moving_quantile_95_6', 'nitrateSurface_Moving_average_12_mean', 'phosphateSurface_Moving_std_5_mean', 'nitrate100_300_Moving_quantile_10_6', 'zonalCurrentSurface_Moving_quantile_99_9', 'salinitySurface_May_quantile_99', 'phosphateSurface_Moving_quantile_99_11', 'nitrateSurface_November_quantile_1', 'chlorophyll_August_mean', 'zonalCurrent100_300_July_quantile_5', 'dissolvedOxygen100_300_Moving_average_3_mean', 'temperature100_300trim_mean_5%', 'nitrateSurface_Moving_min_7_mean', 'zonalCurrent100_300_Moving_quantile_99_8', 'salinitySurface_Moving_std_12_mean', 'temperature100_300_median', 'dissolvedOxygenSurface_April_quantile_90', 'phosphate100_300_Expanding_quantile_5', 'temperatureSurface_quantile_10', 'wave_Height_kurt', 'chlorophyll_December_quantile_95', 'nitrateSurface_Moving_quantile_10_8', 'temperatureSurface_Moving_min_7_mean', 'zonalCurrentSurface_years_max', 'dissolvedOxygenSurface_years_quantile_10', 'euphoticDepth_August_min', 'dissolvedOxygenSurface_December_quantile_90', 'salinitySurface_Moving_min_3_mean', 'zonalCurrentSurface_August_quantile_95', 'chlorophyll_kurt', 'dissolvedOxygenSurface_Moving_min_2_mean', 'chlorophyll_Moving_average_9_mean', 'nitrate100_300_5medfilt_mean', 'nitrateSurface_Moving_min_8_mean', 'dissolvedOxygen100_300_Moving_quantile_90_11', 'meridionalCurrent100_300_Moving_quantile_5_8', 'salinitySurface_Moving_quantile_95_8', 'salinity100_300_Moving_quantile_90_3', 'nitrate100_300_April_quantile_90', 'meridionalCurrentSurface_50wiener_mean', 'wave_Height_June_quantile_1', 'phosphateSurface_December_quantile_5', 'wave_Height_Moving_std_7_mean', 'chlorophyll_50medfilt_mean', 'salinity100_300_Moving_quantile_5_10', 'secchiDiskDepth_Moving_std_8_mean', 'phosphateSurface_Moving_std_7_mean', 'wave_Height_Moving_std_4_mean', 'dissolvedOxygen100_300_Moving_max_2_mean', 'meridionalCurrent100_300_Moving_min_3_mean', 'secchiDiskDepth_Moving_median_3_mean', 'zonalCurrentSurface_Moving_quantile_99_11', 'dissolvedOxygenSurface_February_quantile_1', 'phosphateSurface_December_max', 'meridionalCurrent100_300_January_quantile_99', 'meridionalCurrent100_300_Moving_min_2_mean', 'meridionalCurrent100_300_November_quantile_95', 'meridionalCurrentSurface_mean_change_argrelmax', 'phosphateSurface_Moving_median_4_mean', 'meridionalCurrent100_300_September_quantile_99', 'salinitySurface_April_quantile_5', 'zonalCurrent100_300_Moving_quantile_1_2', 'temperatureSurface_Moving_median_6_mean', 'dissolvedOxygen100_300_years_quantile_5', 'secchiDiskDepth_May_mean', 'chlorophyll_Moving_average_6_mean', 'nitrateSurface_January_quantile_95', 'euphoticDepth_January_mean', 'dissolvedOxygenSurface_April_quantile_5', 'secchiDiskDepth_quantile_95', 'meridionalCurrent100_300_50wiener_mean', 'phosphate100_300_Moving_average_9_mean', 'phosphateSurface_July_quantile_95', 'temperatureSurface_Moving_min_2_mean', 'zonalCurrent100_300_Moving_skew_12_mean', 'nitrate100_300_Moving_min_3_mean', 'temperature100_300_Moving_quantile_90_7', 'nitrate100_300_years_median', 'phosphate100_300_Moving_skew_6_mean', 'salinity100_300_Moving_average_8_mean', 'temperature100_300_December_quantile_5', 'wave_Heighttrim_mean_15%', 'salinitySurface_June_quantile_90', 'phosphateSurface_3medfilt_mean', 'nitrateSurface_Expanding_quantile_95', 'temperature100_300_Moving_std_11_mean', 'temperature100_300_Moving_average_2_mean', 'nitrateSurface_December_quantile_1', 'secchiDiskDepth_October_quantile_95', 'temperatureSurface_years_median', 'phosphate100_300_Moving_quantile_90_7', 'nitrate100_300_Expanding_median_mean', 'meridionalCurrent100_300_Moving_std_2_mean', 'nitrate100_300_August_quantile_99', 'phosphate100_300_Moving_quantile_1_12', 'secchiDiskDepth_11savgol_mean', 'salinity100_300_July_quantile_90', 'nitrateSurface_Moving_quantile_10_3', 'zonalCurrent100_300_Moving_quantile_90_4', 'nitrate100_300_May_mean', 'temperatureSurface_Moving_quantile_95_12', 'phosphateSurface_March_quantile_90', 'salinity100_300_April_quantile_1', 'zonalCurrentSurface_May_quantile_1', 'euphoticDepth_Moving_quantile_90_10', 'secchiDiskDepth_Moving_median_12_mean', 'temperature100_300_Moving_min_4_mean', 'salinitySurface_50wiener_mean', 'zonalCurrentSurface_June_median', 'wave_Height_Hilbert_mean', 'dissolvedOxygen100_300_Moving_quantile_95_11', 'salinity100_300_Moving_quantile_5_4', 'dissolvedOxygen100_300_March_min', 'temperature100_300_Moving_quantile_10_3', 'salinitySurface_Moving_quantile_95_9', 'salinitySurface_July_max', 'meridionalCurrentSurface_August_median', 'temperature100_300_December_quantile_10', 'dissolvedOxygen100_300_Moving_max_11_mean', 'meridionalCurrent100_300_Moving_average_3_mean', 'phosphate100_300_median', 'salinity100_300_Moving_quantile_1_8', 'euphoticDepth_quantile_99', 'nitrate100_300_March_quantile_5', 'wave_Height_August_quantile_5', 'phosphateSurface_Moving_median_12_mean', 'phosphateSurface_June_max', 'euphoticDepth_3gauss_spline_mean', 'temperatureSurface_April_mean', 'salinitySurface_September_min', 'temperatureSurface_June_quantile_5', 'zonalCurrentSurface_Expanding_median_mean', 'phosphate100_300_Moving_median_3_mean', 'zonalCurrentSurface_Moving_std_2_mean', 'salinitySurface_November_mean', 'chlorophyll_March_quantile_95', 'dissolvedOxygenSurface_quantile_10', 'meridionalCurrentSurface_Moving_quantile_10_9', 'chlorophyll_median', 'salinity100_300_Moving_quantile_5_5', 'euphoticDepthtrim_mean_20%', 'chlorophyll_September_quantile_1', 'salinity100_300_Moving_quantile_5_2', 'salinity100_300_September_quantile_90', 'zonalCurrent100_300_Hilbert_mean', 'salinity100_300_Moving_quantile_99_8', 'temperature100_300_4wiener_mean', 'wave_Height_50medfilt_mean', 'dissolvedOxygenSurface_mean_change_argrelmax', 'meridionalCurrent100_300_Moving_min_12_mean', 'phosphate100_300_July_quantile_99', 'salinity100_300_November_max', 'nitrate100_300trim_mean_5%', 'chlorophyll_Moving_quantile_5_3', 'zonalCurrent100_300_peak_count', 'wave_Height_Expanding_average_mean', 'meridionalCurrentSurface_Moving_std_4_mean', 'temperature100_300_October_quantile_1', 'secchiDiskDepth_January_quantile_10', 'secchiDiskDepth_January_quantile_5', 'zonalCurrentSurface_Moving_max_3_mean', 'temperatureSurface_Moving_skew_12_mean', 'euphoticDepth_Moving_quantile_99_9', 'phosphateSurface_Moving_quantile_99_2', 'wave_Height_Expanding_median_mean', 'zonalCurrent100_300_March_quantile_5', 'meridionalCurrentSurface_Moving_skew_10_mean', 'dissolvedOxygenSurface_February_quantile_90', 'salinity100_300_December_mean', 'salinitySurface_Moving_skew_9_mean', 'dissolvedOxygen100_300_October_quantile_10', 'salinitySurface_Expanding_max_mean', 'meridionalCurrent100_300_September_quantile_1', 'temperatureSurface_years_mean', 'phosphate100_300_December_quantile_10', 'zonalCurrent100_300_August_quantile_95', 'nitrate100_300_10medfilt_mean', 'nitrate100_300_years_quantile_10', 'wave_Height_Moving_max_10_mean', 'salinitySurface_July_min', 'nitrate100_300_August_max', 'dissolvedOxygenSurface_October_quantile_95', 'dissolvedOxygen100_300_February_mean', 'salinity100_300_October_median', 'wave_Height_Moving_average_4_mean', 'zonalCurrentSurface_Moving_median_10_mean', 'temperature100_300_June_quantile_90', 'nitrate100_300_mean_change_argrelmin', 'chlorophyll_Moving_quantile_1_7', 'dissolvedOxygen100_300_peak_max', 'meridionalCurrentSurface_April_quantile_10', 'temperatureSurface_3medfilt_mean', 'nitrate100_300_Moving_average_4_mean', 'salinity100_300_Expanding_min_mean', 'salinitySurface_October_max', 'dissolvedOxygen100_300_Moving_quantile_90_12', 'zonalCurrentSurface_years_quantile_5', 'zonalCurrentSurface_January_quantile_10', 'nitrate100_300_decimate_mean', 'phosphate100_300_2gauss_spline_mean', 'temperature100_300_Moving_quantile_90_4', 'meridionalCurrent100_300_10medfilt_mean', 'salinitySurface_August_quantile_99', 'meridionalCurrent100_300_January_quantile_5', 'secchiDiskDepthtrim_mean_10%', 'dissolvedOxygenSurface_September_min', 'temperature100_300_July_quantile_90', 'phosphate100_300_Expanding_average_mean', 'salinity100_300_Moving_quantile_99_10', 'dissolvedOxygenSurface_July_quantile_10', 'secchiDiskDepth_Moving_average_11_mean', 'dissolvedOxygenSurface_Moving_quantile_10_6', 'dissolvedOxygenSurface_11savgol_mean', 'temperature100_300_Moving_quantile_90_10', 'meridionalCurrent100_300_Moving_skew_12_mean', 'dissolvedOxygen100_300_Moving_average_10_mean', 'wave_Height_Moving_quantile_5_10', 'zonalCurrent100_300_August_quantile_10', 'salinitySurface_Moving_quantile_95_6', 'dissolvedOxygenSurface_Moving_min_12_mean', 'wave_Height_Moving_skew_5_mean', 'meridionalCurrentSurface_May_quantile_5', 'phosphate100_300_December_mean', 'nitrate100_300_years_max', 'temperatureSurface_September_quantile_10', 'temperatureSurface_August_max', 'dissolvedOxygenSurface_Moving_quantile_95_6', 'temperature100_300_Moving_std_5_mean', 'salinity100_300_February_quantile_99', 'meridionalCurrentSurface_Expanding_average_mean', 'temperature100_300_Moving_quantile_1_4', 'nitrate100_300_January_quantile_10', 'zonalCurrentSurface_June_quantile_1', 'salinitySurface_July_quantile_10', 'nitrate100_300_July_quantile_95', 'phosphate100_300iqr1', 'temperatureSurface_August_quantile_95', 'zonalCurrentSurface_August_quantile_5', 'phosphate100_300_Moving_min_7_mean', 'temperature100_300_November_min', 'zonalCurrentSurface_Moving_quantile_1_12', 'meridionalCurrentSurface_Moving_quantile_95_4', 'phosphate100_300_Moving_min_8_mean', 'salinitySurfaceiqr', 'zonalCurrent100_300_mean_change_peak', 'nitrate100_300_December_quantile_99', 'nitrate100_300_May_median', 'temperature100_300_Moving_quantile_99_8', 'dissolvedOxygen100_300_Moving_min_3_mean', 'salinitySurface_September_mean', 'temperatureSurface_March_quantile_90', 'zonalCurrent100_300iqr', 'secchiDiskDepth_Moving_quantile_5_7', 'nitrateSurface_years_std', 'zonalCurrentSurface_November_quantile_99', 'nitrateSurface_Moving_std_8_mean', 'euphoticDepth_51savgol_mean', 'nitrate100_300_Moving_skew_6_mean', 'secchiDiskDepth_December_quantile_5', 'meridionalCurrentSurface_Moving_quantile_90_4', 'temperatureSurface_Moving_std_11_mean', 'salinitySurface_10medfilt_mean', 'nitrate100_300_Moving_quantile_5_10', 'zonalCurrentSurface_Moving_skew_10_mean', 'salinity100_300_Expanding_quantile_10', 'temperature100_300_Moving_median_6_mean', 'meridionalCurrent100_300_Moving_quantile_10_5', 'zonalCurrent100_300_Moving_average_4_mean', 'chlorophyll_July_quantile_99', 'temperature100_300_Moving_quantile_1_8', 'zonalCurrentSurface_Moving_min_10_mean', 'secchiDiskDepth_Moving_quantile_1_9', 'nitrate100_300_Moving_median_3_mean', 'dissolvedOxygen100_300_years_max', 'zonalCurrentSurface_March_quantile_1', 'nitrateSurface_Moving_skew_7_mean', 'zonalCurrent100_300_Moving_quantile_10_8', 'nitrateSurface_November_quantile_99', 'salinitySurface_December_mean', 'euphoticDepth_Moving_average_8_mean', 'salinity100_300_Moving_quantile_95_4', 'meridionalCurrent100_300_peak_count', 'zonalCurrent100_300iqr1', 'dissolvedOxygen100_300_February_max', 'phosphateSurface_June_quantile_99', 'phosphateSurface_May_quantile_90', 'temperature100_300_Moving_quantile_10_6', 'nitrate100_300_years_quantile_5', 'secchiDiskDepth_February_quantile_95', 'nitrate100_300_May_quantile_99', 'meridionalCurrent100_300_Moving_median_9_mean', 'temperatureSurface_April_max', 'meridionalCurrent100_300_Moving_max_9_mean', 'salinity100_300_January_mean', 'salinitySurface_October_mean', 'chlorophyll_March_quantile_1', 'meridionalCurrent100_300_Moving_average_10_mean', 'secchiDiskDepth_Moving_quantile_99_11', 'salinity100_300_Moving_quantile_5_3', 'dissolvedOxygenSurface_October_quantile_99', 'temperatureSurface_Moving_quantile_1_12', 'temperature100_300_25savgol_mean', 'temperature100_300_Moving_std_2_mean', 'wave_Height_Moving_min_12_mean', 'meridionalCurrent100_300_Moving_median_3_mean', 'secchiDiskDepth_Moving_quantile_95_12', 'temperatureSurface_Moving_max_4_mean', 'chlorophyll_August_quantile_1', 'phosphateSurface_Moving_min_6_mean', 'temperature100_300_March_mean', 'temperatureSurface_November_median', 'secchiDiskDepth_February_quantile_90', 'phosphate100_300_mean_change_argrelmin', 'nitrateSurface_Moving_std_10_mean', 'salinity100_300_years_quantile_10', 'zonalCurrentSurface_quantile_10', 'secchiDiskDepth_Moving_quantile_5_9', 'secchiDiskDepth_May_quantile_10', 'temperatureSurface_Expanding_max_mean', 'phosphate100_300_quantile_1', 'phosphateSurface_Moving_average_7_mean', 'salinitySurface_Moving_quantile_10_11', 'nitrateSurface_Moving_quantile_5_8', 'temperatureSurfacetrim_mean_20%', 'temperature100_300_November_quantile_95', 'nitrateSurface_Moving_std_9_mean', 'nitrate100_300_3gauss_spline_mean', 'dissolvedOxygen100_300_January_min', 'dissolvedOxygenSurface_quantile_99', 'nitrateSurface_March_quantile_1', 'dissolvedOxygen100_300_years_quantile_95', 'dissolvedOxygenSurface_years_min', 'temperatureSurface_Moving_median_3_mean', 'wave_Height_argrelminmax_count', 'nitrateSurface_June_quantile_99', 'zonalCurrentSurface_Moving_quantile_95_6', 'dissolvedOxygenSurface_median', 'salinity100_300_October_quantile_95', 'meridionalCurrent100_300_March_median', 'nitrateSurface_March_quantile_90', 'wave_Height_Moving_quantile_95_4', 'dissolvedOxygen100_300_April_quantile_10', 'phosphate100_300_Moving_min_2_mean', 'phosphateSurface_Moving_max_11_mean', 'dissolvedOxygenSurface_neighbor_max', 'salinitySurface_September_quantile_90', 'zonalCurrentSurfaceiqr', 'temperatureSurface_Moving_average_6_mean', 'zonalCurrent100_300_Moving_skew_8_mean', 'dissolvedOxygen100_300_Hann_window_mean', 'zonalCurrentSurface_April_quantile_10', 'temperature100_300_Moving_std_10_mean', 'dissolvedOxygenSurface_May_quantile_5', 'secchiDiskDepth_September_quantile_95', 'meridionalCurrent100_300_Moving_quantile_95_12', 'wave_Height_Moving_average_8_mean', 'wave_Height_Moving_max_3_mean', 'phosphateSurface_January_max', 'dissolvedOxygen100_300_Moving_quantile_10_8', 'nitrate100_300_Moving_max_5_mean', 'zonalCurrent100_300_50medfilt_mean', 'salinitySurface_quantile_95', 'meridionalCurrentSurface_Moving_max_3_mean', 'secchiDiskDepth_Moving_quantile_10_5', 'zonalCurrentSurface_November_median', 'nitrateSurface_Moving_skew_8_mean', 'wave_Height_Moving_quantile_95_10', 'phosphateSurface_January_mean', 'zonalCurrent100_300_Moving_quantile_99_10', 'secchiDiskDepth_Moving_quantile_10_7', 'phosphate100_300_Moving_quantile_5_7', 'dissolvedOxygen100_300_Moving_quantile_5_4', 'chlorophyll_Moving_quantile_5_10', 'zonalCurrent100_300_Moving_std_10_mean', 'meridionalCurrentSurface_June_quantile_10', 'temperatureSurface_April_median', 'temperature100_300_mean_change_argrelmin', 'phosphate100_300_Moving_std_6_mean', 'nitrate100_300_February_quantile_99', 'salinitySurface_Moving_median_12_mean', 'chlorophyll_Moving_quantile_95_2', 'salinitySurface_August_median', 'salinitySurface_August_quantile_5', 'phosphateSurface_Moving_quantile_10_3', 'wave_Height_Moving_quantile_10_10', 'euphoticDepth_Moving_min_12_mean', 'temperature100_300_September_mean', 'nitrate100_300_Expanding_max_mean', 'euphoticDepth_25medfilt_mean', 'salinitySurface_April_quantile_95', 'phosphateSurfacetrim_mean_10%', 'phosphate100_300_Moving_skew_7_mean', 'meridionalCurrentSurface_years_quantile_5', 'wave_Height_Moving_std_5_mean', 'zonalCurrent100_300_Moving_quantile_95_5', 'zonalCurrentSurface_Moving_quantile_99_7', 'salinity100_300_Moving_median_4_mean', 'chlorophyll_September_quantile_5', 'chlorophyll_3gauss_spline_mean', 'temperatureSurface_Moving_std_8_mean', 'zonalCurrent100_300_Moving_quantile_90_12', 'temperatureSurface_October_quantile_90', 'wave_Height_51savgol_mean', 'dissolvedOxygen100_300_Moving_max_12_mean', 'chlorophyll_April_quantile_10', 'euphoticDepth_Moving_average_11_mean', 'zonalCurrent100_300_argrelminmax_count', 'dissolvedOxygen100_300_argrelminmax_count', 'dissolvedOxygen100_300_February_quantile_90', 'secchiDiskDepth_Moving_median_4_mean', 'phosphateSurface_years_quantile_5', 'nitrate100_300_Moving_std_8_mean', 'salinitySurface_Moving_quantile_99_10', 'salinitySurface_median', 'nitrateSurface_November_max', 'phosphate100_300_Moving_quantile_95_8', 'chlorophyll_April_std', 'euphoticDepth_Expanding_quantile_90', 'nitrate100_300_Moving_quantile_1_2', 'nitrate100_300_Moving_quantile_1_9', 'temperature100_300_March_min', 'temperature100_300_years_quantile_1', 'temperatureSurface_25medfilt_mean', 'meridionalCurrent100_300_Moving_median_12_mean', 'chlorophylltrim_mean_25%', 'secchiDiskDepth_July_quantile_10', 'temperature100_300_Moving_quantile_90_6', 'meridionalCurrentSurface_51savgol_mean', 'chlorophyll_July_median', 'temperatureSurface_Moving_std_2_mean', 'phosphateSurface_Expanding_quantile_5', 'nitrateSurface_Moving_median_2_mean', 'phosphateSurface_Moving_quantile_5_4', 'zonalCurrentSurface_Moving_quantile_10_10', 'dissolvedOxygenSurface_Moving_max_5_mean', 'zonalCurrent100_300_Moving_quantile_10_11', 'dissolvedOxygenSurface_August_mean', 'dissolvedOxygen100_300_Moving_median_5_mean', 'phosphateSurface_Moving_quantile_1_3', 'temperature100_300_Moving_quantile_5_2', 'zonalCurrent100_300_years_quantile_10', 'meridionalCurrentSurface_Moving_quantile_99_11', 'chlorophyll_Moving_quantile_90_5', 'temperatureSurface_Moving_average_5_mean', 'zonalCurrentSurface_Moving_average_6_mean', 'dissolvedOxygen100_300_April_quantile_90', 'nitrate100_300_November_max', 'phosphate100_300_3spline_mean', 'secchiDiskDepth_May_max', 'temperatureSurface_September_quantile_99', 'dissolvedOxygen100_300_Expanding_quantile_95', 'euphoticDepthtrim_mean_15%', 'secchiDiskDepth_Hilbert_mean', 'secchiDiskDepth_Moving_quantile_1_2', 'zonalCurrent100_300_max_to_min_diff', 'temperature100_300_Moving_skew_8_mean', 'nitrateSurface_Moving_max_2_mean', 'zonalCurrent100_300_mad', 'nitrateSurface_October_quantile_10', 'dissolvedOxygenSurface_argrelmax_count', 'chlorophyll_Moving_quantile_1_4', 'dissolvedOxygen100_300_March_quantile_5', 'euphoticDepth_Moving_std_9_mean', 'temperature100_300_Moving_quantile_10_2', 'nitrateSurface_August_quantile_10', 'zonalCurrentSurface_4spline_mean', 'secchiDiskDepth_Moving_skew_12_mean', 'euphoticDepth_July_quantile_5', 'zonalCurrentSurfacetrim_mean_10%', 'salinitySurface_Moving_std_6_mean', 'meridionalCurrentSurface_July_quantile_5', 'chlorophyll_April_max', 'phosphateSurface_Expanding_quantile_10', 'zonalCurrent100_300_Moving_quantile_95_8', 'meridionalCurrentSurfacetrim_mean_5%', 'dissolvedOxygenSurface_Moving_quantile_1_4', 'wave_Height_January_std', 'zonalCurrentSurface_Moving_quantile_99_4', 'meridionalCurrent100_300_Moving_median_10_mean', 'phosphate100_300_10medfilt_mean', 'salinity100_300_Moving_quantile_5_9', 'temperatureSurfacetrim_mean_5%', 'salinitySurface_Moving_max_6_mean', 'salinity100_300_median', 'salinitySurface_June_quantile_10', 'temperatureSurface_Moving_skew_11_mean', 'salinity100_300_Moving_quantile_99_12', 'euphoticDepth_June_quantile_90', 'salinity100_300_Moving_quantile_10_9', 'meridionalCurrent100_300_Moving_quantile_5_5', 'secchiDiskDepth_June_quantile_10', 'secchiDiskDepth_Expanding_std_mean', 'zonalCurrent100_300_Moving_std_4_mean', 'meridionalCurrent100_300_July_quantile_1', 'wave_Height_November_quantile_1', 'nitrate100_300_March_min', 'salinity100_300_Moving_quantile_99_11', 'nitrate100_300_Moving_quantile_95_4', 'salinitySurface_quantile_90', 'zonalCurrentSurface_5medfilt_mean', 'zonalCurrentSurface_February_quantile_90', 'dissolvedOxygen100_300_Moving_skew_9_mean', 'phosphateSurface_Moving_skew_7_mean', 'temperature100_300_September_min', 'chlorophyll_January_quantile_1', 'zonalCurrentSurface_January_quantile_5', 'meridionalCurrent100_300_August_quantile_95', 'zonalCurrent100_300_Moving_quantile_90_9', 'phosphateSurface_Expanding_quantile_95', 'phosphateSurface_Moving_average_6_mean', 'chlorophyll_mean_change_peak_rate', 'temperatureSurface_March_median', 'dissolvedOxygenSurface_Moving_quantile_10_4', 'phosphateSurface_Moving_quantile_10_2', 'salinity100_300_max_to_min_diff', 'euphoticDepth_Moving_quantile_95_5', 'temperatureSurface_October_quantile_10', 'dissolvedOxygen100_300_June_median', 'phosphate100_300_July_max', 'dissolvedOxygenSurface_Moving_std_4_mean', 'chlorophyll_Moving_quantile_1_12', 'temperatureSurface_May_median', 'temperatureSurface_November_mean', 'euphoticDepth_years_quantile_10', 'dissolvedOxygen100_300trim_mean_10%', 'wave_Height_January_quantile_95', 'secchiDiskDepth_Moving_quantile_10_2', 'chlorophyll_Moving_median_4_mean', 'dissolvedOxygen100_300_Expanding_average_mean', 'phosphateSurface_Moving_quantile_1_6', 'salinitySurface_Moving_quantile_95_5', 'salinitySurface_Moving_quantile_10_9', 'meridionalCurrentSurface_April_quantile_95', 'salinitySurface_November_quantile_5', 'nitrate100_300_April_min', 'zonalCurrent100_300_Moving_quantile_90_10', 'dissolvedOxygen100_300_February_quantile_1', 'euphoticDepth_years_quantile_99', 'secchiDiskDepth_quantile_10', 'nitrate100_300_August_quantile_10', 'meridionalCurrent100_300_October_quantile_95', 'phosphateSurface_July_max', 'phosphate100_300_December_quantile_90', 'zonalCurrent100_300_Moving_average_8_mean', 'nitrateSurface_Moving_quantile_99_4', 'nitrateSurface_July_median', 'secchiDiskDepth_peak_count', 'salinity100_300_Moving_quantile_99_7', 'salinity100_300_June_min', 'salinity100_300_May_quantile_99', 'temperatureSurface_5savgol_mean', 'temperature100_300_Moving_quantile_1_5', 'zonalCurrent100_300_Moving_quantile_5_6', 'temperature100_300_July_quantile_10', 'dissolvedOxygenSurface_Moving_quantile_5_12', 'salinity100_300_Moving_quantile_1_6', 'salinity100_300_June_max', 'zonalCurrentSurface_Expanding_quantile_99', 'meridionalCurrent100_300_4gauss_spline_mean', 'phosphate100_300_January_quantile_5', 'salinity100_300_April_mean', 'dissolvedOxygenSurface_Expanding_quantile_1', 'dissolvedOxygen100_300_10wiener_mean', 'euphoticDepth_September_mean', 'euphoticDepth_Moving_quantile_1_3', 'dissolvedOxygenSurface_Moving_max_4_mean', 'secchiDiskDepth_Moving_min_8_mean', 'salinity100_300_May_quantile_90', 'nitrate100_300_Moving_skew_7_mean', 'meridionalCurrentSurface_March_quantile_99', 'euphoticDepth_Expanding_quantile_99', 'salinitySurface_April_mean', 'salinitySurface_Moving_average_9_mean', 'salinitySurface_September_quantile_95', 'phosphate100_300_years_quantile_90', 'nitrate100_300_Hann_window_mean', 'phosphateSurface_Moving_min_5_mean', 'phosphateSurface_Moving_quantile_99_7', 'salinity100_300_peak_max', 'phosphateSurface_Moving_median_11_mean', 'zonalCurrentSurface_Moving_quantile_95_2', 'temperatureSurface_October_mean', 'wave_Height_Moving_quantile_10_5', 'zonalCurrentSurface_Moving_quantile_5_9', 'zonalCurrentSurface_April_median', 'meridionalCurrentSurface_February_quantile_1', 'nitrateSurface_May_quantile_99', 'salinity100_300_Moving_quantile_95_8', 'zonalCurrent100_300_2gauss_spline_mean', 'meridionalCurrent100_300_Moving_max_8_mean', 'temperature100_300_10medfilt_mean', 'euphoticDepth_March_median', 'dissolvedOxygen100_300_Moving_quantile_99_12', 'chlorophyll_Moving_std_4_mean', 'chlorophyll_July_max', 'secchiDiskDepth_Moving_median_11_mean', 'temperatureSurface_Moving_std_10_mean', 'nitrateSurface_April_quantile_99', 'nitrate100_300_February_min', 'zonalCurrentSurface_June_quantile_10', 'secchiDiskDepth_years_quantile_90', 'meridionalCurrentSurfaceiqr', 'salinity100_300_Moving_quantile_10_7', 'phosphate100_300_Moving_average_4_mean', 'salinity100_300_Moving_std_3_mean', 'zonalCurrentSurface_Moving_quantile_95_7', 'dissolvedOxygenSurface_July_quantile_95', 'salinitySurface_Moving_quantile_95_2', 'zonalCurrent100_300_Moving_quantile_95_9', 'zonalCurrentSurface_peak_mean', 'dissolvedOxygen100_300_Moving_quantile_5_8', 'phosphate100_300_October_mean', 'wave_Height_Moving_quantile_10_11', 'salinitySurface_March_quantile_5', 'salinitySurface_Moving_quantile_5_4', 'salinity100_300_Moving_average_11_mean', 'zonalCurrentSurface_August_quantile_99', 'salinitySurface_years_quantile_90', 'temperature100_300_Expanding_average_mean', 'nitrateSurface_Moving_quantile_90_11', 'dissolvedOxygen100_300_Moving_std_12_mean', 'meridionalCurrent100_300_Moving_skew_10_mean', 'phosphate100_300_Moving_quantile_5_10', 'dissolvedOxygen100_300_Moving_quantile_10_10', 'dissolvedOxygenSurface_Moving_median_9_mean', 'meridionalCurrentSurface_September_quantile_1', 'nitrate100_300_Moving_quantile_95_9', 'secchiDiskDepth_Moving_min_12_mean', 'zonalCurrentSurface_years_min', 'wave_Height_January_quantile_99', 'salinitySurface_April_min', 'meridionalCurrent100_300_Moving_min_5_mean', 'secchiDiskDepth_April_mean', 'temperatureSurface_May_quantile_10', 'phosphate100_300_Moving_quantile_5_2', 'euphoticDepth_Moving_quantile_95_10', 'salinity100_300_December_median', 'salinitySurface_Expanding_quantile_1', 'euphoticDepth_5savgol_mean', 'wave_Height_Moving_max_5_mean', 'meridionalCurrent100_300_Moving_std_5_mean', 'salinitySurface_symiirorder2_mean', 'salinitySurface_Moving_skew_10_mean', 'meridionalCurrent100_300_quantile_5', 'temperatureSurface_October_quantile_95', 'zonalCurrent100_300_February_quantile_99', 'phosphateSurface_Moving_std_11_mean', 'chlorophyll_Moving_skew_9_mean', 'nitrate100_300_Moving_quantile_99_12', 'zonalCurrentSurface_Moving_skew_9_mean', 'salinitySurface_years_quantile_95', 'phosphateSurface_symiirorder1_mean', 'temperatureSurface_June_min', 'meridionalCurrent100_300_Moving_quantile_10_11', 'temperatureSurface_Moving_skew_9_mean', 'euphoticDepth_Moving_quantile_99_3', 'temperature100_300_quantile_99', 'temperatureSurface_Moving_min_5_mean', 'meridionalCurrent100_300_Moving_std_6_mean', 'zonalCurrentSurface_years_std', 'secchiDiskDepth_Moving_max_7_mean', 'salinitySurface_Hann_window_mean', 'phosphateSurface_Moving_min_11_mean', 'phosphate100_300_Moving_std_7_mean', 'salinity100_300_Moving_quantile_5_12', 'salinity100_300_August_quantile_95', 'phosphate100_300_Moving_max_3_mean', 'chlorophyll_Moving_min_11_mean', 'secchiDiskDepth_Moving_quantile_10_6', 'salinitySurface_Moving_quantile_1_11', 'euphoticDepth_June_quantile_5', 'dissolvedOxygenSurface_February_quantile_5', 'chlorophyll_Moving_quantile_99_6', 'euphoticDepth_Moving_quantile_1_5', 'salinity100_300_Moving_quantile_90_6', 'secchiDiskDepth_Moving_average_10_mean', 'nitrateSurface_Moving_quantile_5_11', 'temperatureSurface_Moving_quantile_99_5', 'salinitySurface_Moving_skew_11_mean', 'wave_Height_Moving_median_4_mean', 'dissolvedOxygen100_300_September_quantile_90', 'zonalCurrent100_300_October_quantile_99', 'secchiDiskDepth_November_min', 'chlorophyll_May_max', 'nitrate100_300_September_quantile_5', 'meridionalCurrent100_300_25savgol_mean', 'dissolvedOxygen100_300_April_max', 'temperature100_300_Moving_quantile_90_5', 'nitrateSurface_Moving_quantile_1_2', 'chlorophyll_quantile_1', 'meridionalCurrentSurface_Moving_quantile_95_7', 'chlorophyll_Moving_quantile_95_6', 'zonalCurrentSurface_January_median', 'chlorophyll_Moving_quantile_5_12', 'temperatureSurface_50wiener_mean', 'salinity100_300_Moving_quantile_10_6', 'phosphateSurface_Moving_quantile_90_3', 'meridionalCurrent100_300_2gauss_spline_mean', 'dissolvedOxygen100_300_April_min', 'zonalCurrent100_300_November_quantile_5', 'salinitySurface_June_quantile_1', 'temperature100_300_mad', 'euphoticDepth_December_median', 'temperatureSurface_February_quantile_5', 'temperatureSurface_April_quantile_10', 'chlorophyll_Hann_window_mean', 'wave_Height_July_quantile_5', 'temperatureSurface_August_quantile_1', 'chlorophyll_50wiener_mean', 'salinitySurface_Moving_quantile_95_10', 'euphoticDepth_mad', 'chlorophyll_Moving_quantile_1_9', 'dissolvedOxygenSurface_Moving_quantile_90_6', 'zonalCurrent100_300_October_quantile_10', 'temperatureSurface_March_quantile_10', 'phosphateSurface_years_quantile_99', 'dissolvedOxygen100_300_May_quantile_1', 'zonalCurrentSurface_years_quantile_99', 'zonalCurrentSurface_Moving_quantile_5_10', 'nitrateSurface_August_quantile_5', 'dissolvedOxygen100_300_years_quantile_99', 'nitrateSurface_Moving_median_3_mean', 'secchiDiskDepth_November_quantile_10', 'salinity100_300_September_quantile_95', 'meridionalCurrentSurface_Moving_quantile_90_10', 'salinitySurface_Moving_quantile_5_2', 'temperature100_300_Moving_quantile_99_2', 'dissolvedOxygen100_300_Moving_quantile_99_8', 'zonalCurrent100_300_October_quantile_5', 'temperatureSurface_December_quantile_10', 'nitrate100_300_Moving_std_6_mean', 'zonalCurrentSurface_July_median', 'zonalCurrentSurface_September_quantile_1', 'chlorophyll_peak_count', 'phosphateSurface_July_quantile_1', 'secchiDiskDepth_October_quantile_99', 'dissolvedOxygenSurface_May_mean', 'nitrateSurface_March_quantile_5', 'dissolvedOxygenSurface_April_quantile_95', 'zonalCurrentSurface_April_quantile_1', 'phosphate100_300_Moving_quantile_5_9', 'wave_Height_Moving_median_8_mean', 'meridionalCurrentSurface_Expanding_quantile_1', 'secchiDiskDepth_mean_change_peak', 'salinitySurface_January_mean', 'zonalCurrentSurface_Moving_quantile_10_7', 'temperature100_300_peak_mean', 'wave_Height_August_quantile_95', 'meridionalCurrent100_300_June_quantile_10', 'meridionalCurrent100_300_Moving_max_7_mean', 'secchiDiskDepth_March_max', 'zonalCurrentSurface_Moving_skew_11_mean', 'phosphate100_300_Moving_min_10_mean', 'meridionalCurrent100_300_Moving_skew_11_mean', 'secchiDiskDepth_median', 'chlorophyll_Moving_quantile_99_3', 'temperature100_300_11savgol_mean', 'phosphateSurfaceiqr', 'secchiDiskDepth_Moving_quantile_5_10', 'meridionalCurrent100_300trim_mean_5%', 'dissolvedOxygenSurface_Expanding_max_mean', 'temperatureSurface_Moving_average_9_mean', 'nitrate100_300_Moving_quantile_5_11', 'salinitySurface_October_median', 'wave_Height_11savgol_mean', 'salinity100_300trim_mean_20%', 'zonalCurrentSurface_2spline_mean', 'meridionalCurrent100_300_Moving_quantile_95_10', 'phosphateSurface_May_quantile_95', 'meridionalCurrentSurface_Moving_quantile_10_11', 'secchiDiskDepth_quantile_5', 'phosphateSurface_2spline_mean', 'wave_Height_March_quantile_10', 'zonalCurrentSurface_Moving_max_9_mean', 'salinitySurface_November_quantile_90', 'temperatureSurface_Moving_median_4_mean', 'phosphateSurface_Moving_min_9_mean', 'meridionalCurrent100_300_July_quantile_95', 'phosphateSurface_Moving_median_7_mean', 'zonalCurrent100_300_May_median', 'temperatureSurface_Moving_skew_7_mean', 'meridionalCurrent100_300_December_quantile_1', 'nitrate100_300_November_mean', 'nitrateSurface_June_max', 'phosphate100_300_November_max', 'dissolvedOxygenSurface_50wiener_mean', 'dissolvedOxygenSurface_October_quantile_1', 'nitrate100_300_Moving_quantile_10_12', 'euphoticDepth_Moving_average_3_mean', 'salinitySurface_July_quantile_1', 'wave_Height_May_std', 'zonalCurrentSurface_years_quantile_10', 'euphoticDepth_January_quantile_10', 'temperature100_300_September_quantile_99', 'salinitySurface_Expanding_quantile_95', 'salinity100_300_May_quantile_95', 'temperatureSurface_April_min', 'temperature100_300_Moving_average_3_mean', 'meridionalCurrentSurface_Moving_quantile_99_2', 'salinitySurface_January_quantile_99', 'phosphate100_300iqr', 'wave_Height_Moving_average_10_mean', 'temperature100_300_April_quantile_90', 'dissolvedOxygen100_300_Moving_quantile_1_9', 'phosphate100_300_Moving_quantile_1_8', 'salinitySurface_4wiener_mean', 'meridionalCurrentSurface_years_quantile_10', 'nitrateSurface_August_quantile_90', 'salinitySurface_Moving_average_6_mean', 'dissolvedOxygen100_300_Moving_std_9_mean', 'chlorophyll_Moving_quantile_10_7', 'chlorophyll_November_quantile_5', 'nitrateSurface_August_max', 'secchiDiskDepth_April_quantile_10', 'euphoticDepth_August_quantile_95', 'chlorophyll_Moving_quantile_99_7', 'meridionalCurrent100_300_Moving_min_10_mean', 'phosphateSurface_Moving_quantile_5_9', 'temperatureSurface_March_quantile_99', 'phosphate100_300_Moving_quantile_90_2', 'phosphate100_300_May_quantile_90', 'wave_Height_quantile_10', 'phosphate100_300_August_max', 'dissolvedOxygenSurface_May_quantile_99', 'secchiDiskDepth_Moving_quantile_1_10', 'zonalCurrentSurfacetrim_mean_5%', 'meridionalCurrentSurface_April_quantile_5', 'wave_Height_September_quantile_5', 'salinity100_300_May_min', 'nitrateSurface_Moving_quantile_95_11', 'dissolvedOxygenSurface_Moving_std_5_mean', 'temperatureSurface_argrelmax_count', 'salinity100_300_years_min', 'meridionalCurrentSurface_August_quantile_5', 'zonalCurrent100_300_Moving_quantile_5_3', 'chlorophyll_Moving_quantile_5_4', 'meridionalCurrent100_300_Moving_quantile_1_9', 'phosphate100_300_June_quantile_99', 'meridionalCurrent100_300_quantile_1', 'euphoticDepth_November_quantile_1', 'meridionalCurrentSurface_August_quantile_10', 'wave_Height_Moving_average_7_mean', 'temperature100_300_Moving_quantile_99_5', 'nitrateSurface_April_std', 'salinitySurface_Expanding_quantile_10', 'euphoticDepth_Moving_quantile_5_7', 'chlorophyll_September_quantile_90', 'nitrateSurface_Moving_quantile_5_3', 'temperature100_300_May_quantile_10', 'phosphate100_300_Moving_quantile_90_5', 'nitrateSurface_December_std', 'meridionalCurrentSurface_20wiener_mean', 'phosphateSurface_5savgol_mean', 'chlorophyll_Moving_skew_10_mean', 'chlorophyll_May_quantile_90', 'secchiDiskDepth_July_quantile_1', 'zonalCurrentSurface_Moving_max_5_mean', 'dissolvedOxygenSurface_argrelminmax_count', 'zonalCurrent100_300_Expanding_quantile_5', 'temperature100_300_Moving_average_9_mean', 'temperatureSurface_Moving_max_5_mean', 'temperatureSurface_February_quantile_10', 'meridionalCurrent100_300_Moving_min_8_mean', 'dissolvedOxygen100_300_February_min', 'temperatureSurface_November_quantile_95', 'phosphate100_300_Moving_median_10_mean', 'temperature100_300_Moving_quantile_95_11', 'secchiDiskDepth_April_quantile_95', 'salinitySurface_years_quantile_1', 'nitrateSurface_quantile_95', 'phosphateSurface_August_quantile_95', 'dissolvedOxygenSurface_March_quantile_1', 'chlorophyll_Moving_quantile_5_8', 'phosphateSurface_October_quantile_5', 'salinitySurface_Moving_quantile_95_12', 'meridionalCurrent100_300_August_quantile_99', 'euphoticDepth_4spline_mean', 'nitrateSurface_Moving_average_4_mean', 'zonalCurrentSurface_max_to_min_diff', 'salinity100_300_50wiener_mean', 'salinity100_300_January_quantile_90', 'nitrate100_300_May_quantile_1', 'chlorophyll_Moving_max_5_mean', 'nitrate100_300_Moving_quantile_99_6', 'meridionalCurrent100_300trim_mean_15%', 'dissolvedOxygenSurface_Moving_quantile_10_5', 'meridionalCurrentSurface_Moving_quantile_1_2', 'euphoticDepthtrim_mean_25%', 'salinity100_300_January_quantile_1', 'salinitySurface_Moving_quantile_90_6', 'zonalCurrentSurface_Moving_quantile_5_3', 'phosphateSurface_August_quantile_10', 'nitrateSurface_11savgol_mean', 'dissolvedOxygenSurface_Moving_quantile_1_10', 'temperature100_300_September_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_95_5', 'zonalCurrent100_300_Moving_quantile_95_10', 'phosphateSurface_Moving_quantile_90_11', 'salinity100_300_11savgol_mean', 'dissolvedOxygenSurface_25medfilt_mean', 'meridionalCurrentSurface_July_quantile_99', 'salinitySurface_March_max', 'euphoticDepth_April_quantile_99', 'euphoticDepth_Moving_quantile_10_5', 'salinity100_300_3medfilt_mean', 'secchiDiskDepth_Moving_skew_6_mean', 'salinity100_300_argrelminmax_count', 'meridionalCurrentSurface_Moving_skew_12_mean', 'dissolvedOxygenSurface_Moving_quantile_95_12', 'dissolvedOxygen100_300_Moving_quantile_90_10', 'wave_Height_Moving_quantile_95_12', 'chlorophyll_Moving_quantile_90_11', 'nitrateSurface_July_quantile_1', 'temperature100_300_December_max', 'phosphateSurface_Moving_quantile_5_6', 'phosphateSurface_Moving_quantile_5_12', 'zonalCurrentSurface_Hilbert_mean', 'dissolvedOxygenSurface_Moving_quantile_10_2', 'salinitySurface_Moving_quantile_1_7', 'phosphateSurface_Expanding_average_mean', 'phosphateSurface_10medfilt_mean', 'dissolvedOxygenSurface_Moving_average_2_mean', 'phosphate100_300_peak_mean', 'zonalCurrentSurface_Moving_quantile_5_12', 'meridionalCurrent100_300_Moving_quantile_99_4', 'temperature100_300_July_mean', 'nitrate100_300_March_max', 'phosphate100_300trim_mean_20%', 'zonalCurrent100_300_quantile_10', 'salinity100_300_Moving_quantile_10_12', 'chlorophyll_October_quantile_10', 'euphoticDepth_February_quantile_90', 'temperatureSurface_May_quantile_1', 'nitrate100_300_Moving_quantile_10_11', 'nitrate100_300_Moving_quantile_1_5', 'secchiDiskDepth_Moving_quantile_5_8', 'wave_Height_October_quantile_5', 'dissolvedOxygen100_300_June_quantile_90', 'zonalCurrentSurface_mean_change_argrelmax', 'nitrate100_300_November_median', 'salinity100_300_Moving_min_12_mean', 'zonalCurrent100_300_Moving_quantile_95_4', 'secchiDiskDepth_Moving_skew_11_mean', 'nitrate100_300_June_median', 'temperatureSurface_October_quantile_5', 'salinity100_300_December_max', 'phosphateSurface_Moving_max_12_mean', 'phosphateSurface_Moving_quantile_99_5', 'phosphate100_300_December_max', 'dissolvedOxygen100_300_Moving_min_5_mean', 'phosphateSurface_June_quantile_5', 'zonalCurrent100_300_Moving_quantile_10_6', 'temperature100_300_3medfilt_mean', 'dissolvedOxygen100_300_Moving_quantile_1_12', 'wave_Height_Moving_quantile_95_9', 'zonalCurrent100_300_January_quantile_99', 'temperature100_300_Moving_average_5_mean', 'chlorophyll_Moving_average_3_mean', 'salinity100_300_Moving_quantile_5_11', 'meridionalCurrentSurface_Moving_quantile_10_2', 'temperature100_300_March_quantile_90', 'dissolvedOxygen100_300_Expanding_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_90_9', 'secchiDiskDepth_Moving_quantile_99_5', 'salinity100_300_June_quantile_90', 'wave_Height_Moving_skew_10_mean', 'wave_Height_October_quantile_99', 'nitrateSurface_September_quantile_5', 'nitrate100_300_Moving_average_8_mean', 'wave_Height_Moving_max_11_mean', 'salinity100_300_Moving_std_6_mean', 'meridionalCurrentSurface_Moving_quantile_99_10', 'zonalCurrentSurface_Moving_quantile_1_2', 'nitrate100_300_Moving_quantile_90_12', 'nitrate100_300_Moving_average_12_mean', 'salinity100_300_Expanding_quantile_99', 'wave_Height_September_quantile_95', 'dissolvedOxygenSurface_Moving_quantile_99_7', 'euphoticDepth_November_quantile_10', 'chlorophyll_Moving_quantile_95_12', 'nitrate100_300_December_mean', 'wave_Height_3spline_mean', 'zonalCurrentSurface_Moving_quantile_5_8', 'zonalCurrentSurface_Moving_quantile_1_8', 'dissolvedOxygenSurface_Moving_min_3_mean', 'nitrate100_300_Expanding_quantile_95', 'salinitySurface_Moving_quantile_99_3', 'salinity100_300_April_min', 'nitrate100_300_25medfilt_mean', 'phosphateSurface_Moving_quantile_1_2', 'dissolvedOxygenSurface_Moving_quantile_95_4', 'zonalCurrent100_300_Moving_max_9_mean', 'secchiDiskDepth_July_mean', 'phosphateSurface_May_quantile_10', 'salinitySurface_Moving_quantile_99_8', 'dissolvedOxygenSurface_Moving_quantile_90_7', 'secchiDiskDepth_September_min', 'salinity100_300_January_max', 'zonalCurrent100_300_Moving_median_6_mean', 'chlorophyll_January_quantile_90', 'secchiDiskDepth_Moving_quantile_95_2', 'phosphate100_300_January_quantile_1', 'salinity100_300_Moving_max_9_mean', 'dissolvedOxygen100_300_August_min', 'meridionalCurrentSurface_10wiener_mean', 'phosphate100_300_Hilbert_mean', 'wave_Height_years_quantile_1', 'dissolvedOxygen100_300trim_mean_15%', 'temperatureSurface_November_max', 'zonalCurrentSurface_Moving_quantile_99_5', 'dissolvedOxygen100_300_Moving_median_3_mean', 'dissolvedOxygenSurface_March_quantile_90', 'secchiDiskDepth_July_min', 'nitrateSurface_October_median', 'temperatureSurface_Moving_max_2_mean', 'secchiDiskDepth_10medfilt_mean', 'zonalCurrent100_300_Moving_quantile_5_5', 'nitrate100_300_Expanding_average_mean', 'euphoticDepth_Hann_window_mean', 'nitrate100_300_November_quantile_10', 'dissolvedOxygen100_300_mad', 'chlorophyll_November_mean', 'euphoticDepth_October_quantile_90', 'secchiDiskDepth_June_median', 'meridionalCurrent100_300_5medfilt_mean', 'dissolvedOxygen100_300_Moving_median_9_mean', 'chlorophyll_Moving_quantile_95_7', 'zonalCurrent100_300_Moving_median_4_mean', 'nitrateSurface_Moving_quantile_99_12', 'meridionalCurrentSurface_March_quantile_95', 'meridionalCurrentSurface_April_quantile_99', 'temperature100_300trim_mean_15%', 'meridionalCurrent100_300_Moving_quantile_10_6', 'salinitySurface_Moving_std_7_mean', 'zonalCurrentSurface_September_quantile_99', 'temperature100_300_June_max', 'dissolvedOxygen100_300_Moving_max_6_mean', 'dissolvedOxygen100_300_May_median', 'meridionalCurrentSurface_December_quantile_10', 'euphoticDepth_quantile_1', 'zonalCurrentSurface_February_quantile_95', 'phosphate100_300_Moving_max_10_mean', 'dissolvedOxygen100_300_Moving_quantile_95_2', 'secchiDiskDepth_years_quantile_95', 'temperature100_300_February_quantile_95', 'zonalCurrent100_300_September_quantile_10', 'meridionalCurrent100_300_Moving_average_5_mean', 'zonalCurrentSurface_Moving_average_4_mean', 'euphoticDepth_Moving_average_5_mean', 'secchiDiskDepth_Moving_quantile_10_12', 'salinitySurface_August_quantile_1', 'temperature100_300_51savgol_mean', 'phosphate100_300_August_median', 'euphoticDepth_Moving_max_11_mean', 'chlorophyll_October_quantile_95', 'nitrateSurface_Expanding_average_mean', 'nitrateSurface_April_quantile_10', 'meridionalCurrent100_300_April_quantile_1', 'temperature100_300_September_quantile_5', 'salinity100_300_December_quantile_5', 'nitrateSurface_Moving_quantile_99_2', 'dissolvedOxygenSurface_August_quantile_10', 'dissolvedOxygen100_300_peak_mean', 'phosphateSurface_September_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_90_2', 'chlorophyll_25savgol_mean', 'secchiDiskDepth_September_median', 'salinity100_300_March_quantile_90', 'phosphateSurface_October_quantile_10', 'nitrate100_300_Moving_quantile_1_6', 'nitrateSurface_10wiener_mean', 'phosphateSurface_years_median', 'zonalCurrent100_300_years_quantile_90', 'temperatureSurface_Expanding_quantile_10', 'salinitySurface_Moving_quantile_90_2', 'nitrate100_300_25savgol_mean', 'phosphate100_300_April_median', 'temperature100_300_December_quantile_99', 'meridionalCurrentSurface_Moving_quantile_1_3', 'salinity100_300_Moving_skew_8_mean', 'salinity100_300_October_quantile_90', 'temperature100_300_years_median', 'temperature100_300_November_max', 'salinitySurface_February_quantile_10', 'dissolvedOxygen100_300_January_quantile_90', 'salinitySurface_October_quantile_95', 'wave_Height_Moving_quantile_95_11', 'phosphate100_300_January_quantile_90', 'salinitySurface_November_min', 'nitrateSurface_Moving_std_11_mean', 'temperatureSurface_August_quantile_10', 'zonalCurrentSurface_Moving_quantile_90_5', 'meridionalCurrent100_300_April_median', 'nitrate100_300_Moving_average_11_mean', 'dissolvedOxygen100_300_August_quantile_10', 'secchiDiskDepth_January_max', 'meridionalCurrent100_300_June_median', 'dissolvedOxygen100_300_Moving_quantile_90_4', 'secchiDiskDepth_December_quantile_95', 'zonalCurrentSurface_Moving_quantile_10_8', 'secchiDiskDepth_Moving_quantile_99_9', 'phosphate100_300_Moving_quantile_5_3', 'dissolvedOxygenSurface_October_quantile_5', 'chlorophyll_Moving_quantile_5_7', 'meridionalCurrent100_300_Hann_window_mean', 'wave_Height_Moving_quantile_1_8', 'dissolvedOxygen100_300_March_quantile_99', 'zonalCurrentSurface_Expanding_min_mean', 'chlorophyll_December_std', 'nitrateSurface_years_quantile_90', 'meridionalCurrentSurface_Moving_quantile_1_8', 'phosphate100_300_January_quantile_99', 'wave_Height_August_quantile_10', 'meridionalCurrentSurface_Moving_min_4_mean', 'dissolvedOxygen100_300_May_mean', 'nitrate100_300_Moving_std_5_mean', 'zonalCurrentSurface_Moving_min_8_mean', 'chlorophyll_years_std', 'dissolvedOxygenSurface_Moving_quantile_1_12', 'salinity100_300_December_quantile_90', 'temperature100_300_symiirorder1_mean', 'nitrate100_300_October_quantile_99', 'zonalCurrent100_300_Moving_quantile_5_11', 'meridionalCurrentSurface_Moving_quantile_1_10', 'salinity100_300_Moving_quantile_1_11', 'euphoticDepth_Moving_max_2_mean', 'euphoticDepth_July_mean', 'meridionalCurrentSurface_December_median', 'salinitySurface_March_quantile_95', 'phosphateSurface_max', 'salinitySurface_mean_change_argrelmin', 'zonalCurrent100_300_Moving_quantile_5_8', 'salinity100_300_November_median', 'phosphateSurface_Moving_median_8_mean', 'nitrate100_300_Moving_max_8_mean', 'secchiDiskDepth_May_quantile_5', 'phosphateSurface_Expanding_quantile_99', 'dissolvedOxygenSurface_Moving_average_11_mean', 'euphoticDepth_January_quantile_5', 'dissolvedOxygenSurface_November_quantile_90', 'meridionalCurrentSurface_mad', 'nitrate100_300_February_mean', 'wave_Height_May_quantile_1', 'wave_Height_February_quantile_10', 'salinity100_300_Moving_quantile_5_6', 'zonalCurrent100_300trim_mean_15%', 'salinitySurface_Moving_quantile_1_12', 'zonalCurrentSurface_June_quantile_95', 'phosphate100_300_October_max', 'salinity100_300_June_mean', 'salinity100_300_October_min', 'dissolvedOxygenSurface_August_min', 'temperatureSurface_Moving_average_4_mean', 'salinitySurface_Moving_quantile_1_3', 'dissolvedOxygenSurface_Moving_quantile_95_7', 'phosphate100_300_August_mean', 'euphoticDepth_quantile_5', 'meridionalCurrent100_300_Moving_quantile_95_6', 'nitrateSurface_January_max', 'zonalCurrent100_300_Moving_median_12_mean', 'secchiDiskDepth_Moving_median_6_mean', 'zonalCurrentSurface_Moving_quantile_1_7', 'nitrateSurface_Moving_min_2_mean', 'nitrateSurface_December_quantile_90', 'dissolvedOxygenSurface_Moving_quantile_90_4', 'zonalCurrentSurface_Moving_average_2_mean', 'salinitySurface_Moving_quantile_1_8', 'temperature100_300_August_quantile_10', 'euphoticDepth_Moving_max_4_mean', 'dissolvedOxygenSurface_November_max', 'salinitySurface_May_quantile_95', 'salinity100_300_Moving_quantile_10_10', 'zonalCurrentSurface_Moving_quantile_95_9', 'nitrateSurface_June_quantile_1', 'phosphate100_300_Moving_quantile_95_2', 'nitrate100_300_October_median', 'euphoticDepth_Moving_quantile_90_3', 'meridionalCurrent100_300_Moving_min_9_mean', 'phosphate100_300_years_max', 'nitrate100_300_Moving_quantile_95_3', 'salinity100_300_October_quantile_5', 'phosphate100_300_September_quantile_1', 'dissolvedOxygen100_300_Moving_average_12_mean', 'zonalCurrentSurface_25savgol_mean', 'phosphateSurface_Moving_std_2_mean', 'nitrate100_300_January_quantile_5', 'phosphate100_300_Moving_quantile_1_2', 'salinitySurface_October_quantile_1', 'wave_Height_November_quantile_95', 'secchiDiskDepth_Moving_quantile_95_9', 'nitrateSurface_November_quantile_5', 'phosphate100_300_Moving_quantile_5_12', 'dissolvedOxygenSurface_Moving_quantile_90_3', 'chlorophyll_November_median', 'temperature100_300_Moving_quantile_5_10', 'nitrateSurface_August_quantile_99', 'chlorophyll_Moving_quantile_10_6', 'dissolvedOxygenSurface_Moving_median_5_mean', 'salinitySurface_July_quantile_5', 'nitrate100_300_Moving_skew_5_mean', 'secchiDiskDepth_April_quantile_5', 'euphoticDepth_August_quantile_99', 'temperature100_300_Hann_window_mean', 'zonalCurrent100_300_Hann_window_mean', 'meridionalCurrentSurface_June_quantile_99', 'meridionalCurrentSurface_5medfilt_mean', 'nitrate100_300_Moving_quantile_95_5', 'dissolvedOxygenSurface_years_max', 'nitrate100_300_Moving_skew_9_mean', 'chlorophyll_June_mean', 'salinitySurface_argrelminmax_count', 'phosphateSurface_Moving_median_6_mean', 'salinitySurface_July_quantile_90', 'phosphate100_300_mad', 'nitrateSurface_Moving_skew_11_mean', 'dissolvedOxygenSurface_quantile_1', 'nitrate100_300_Moving_max_6_mean', 'chlorophyll_Moving_quantile_95_3', 'nitrate100_300_January_quantile_99', 'meridionalCurrent100_300_Moving_quantile_95_3', 'nitrateSurface_quantile_5', 'meridionalCurrentSurface_years_quantile_99', 'zonalCurrentSurface_Moving_quantile_90_3', 'temperature100_300_August_quantile_99', 'dissolvedOxygenSurfacetrim_mean_15%', 'chlorophyll_January_quantile_99', 'meridionalCurrent100_300_Moving_quantile_10_4', 'euphoticDepth_Moving_median_8_mean', 'phosphate100_300_Moving_min_5_mean', 'meridionalCurrent100_300_years_median', 'temperature100_300_April_quantile_5', 'phosphate100_300_Moving_quantile_90_8', 'zonalCurrentSurface_years_mean', 'meridionalCurrent100_300_Expanding_quantile_95', 'temperatureSurface_Moving_quantile_1_7', 'phosphateSurface_November_quantile_1', 'wave_Height_Moving_quantile_5_11', 'dissolvedOxygen100_300_May_quantile_90', 'meridionalCurrent100_300_5savgol_mean', 'wave_Height_peak_max', 'temperatureSurface_Moving_quantile_99_6', 'dissolvedOxygenSurface_August_median', 'dissolvedOxygen100_300_Moving_quantile_99_2', 'meridionalCurrent100_300_Moving_quantile_90_2', 'secchiDiskDepth_years_max', 'temperature100_300_Moving_median_9_mean', 'secchiDiskDepth_Moving_quantile_99_8', 'salinitySurface_August_min', 'salinitySurfacetrim_mean_25%', 'dissolvedOxygen100_300_decimate_mean', 'salinity100_300_November_quantile_5', 'salinitySurface_Moving_average_5_mean', 'temperatureSurface_Moving_quantile_90_8', 'zonalCurrent100_300_July_quantile_99', 'temperatureSurface_Moving_std_7_mean', 'phosphateSurface_Moving_average_2_mean', 'wave_Height_February_quantile_99', 'temperature100_300_Moving_quantile_1_12', 'phosphateSurface_4wiener_mean', 'dissolvedOxygen100_300_March_median', 'chlorophyll_Moving_skew_12_mean', 'chlorophyll_December_mean', 'salinitySurface_December_quantile_5', 'nitrateSurface_Moving_quantile_95_3', 'zonalCurrentSurface_Moving_quantile_95_5', 'secchiDiskDepth_Moving_average_9_mean', 'zonalCurrent100_300_Moving_quantile_5_4', 'wave_Height_Moving_quantile_1_9', 'euphoticDepth_4gauss_spline_mean', 'temperatureSurface_mad', 'dissolvedOxygen100_300_Moving_max_7_mean', 'zonalCurrentSurface_August_quantile_10', 'phosphateSurface_decimate_mean', 'zonalCurrentSurface_Moving_max_11_mean', 'nitrate100_300_Moving_std_12_mean', 'temperature100_300_May_quantile_90', 'temperatureSurface_December_median', 'phosphate100_300_March_quantile_99', 'dissolvedOxygen100_300_November_quantile_5', 'temperature100_300_Moving_quantile_1_7', 'salinitySurface_September_median', 'nitrateSurface_years_median', 'salinity100_300_peak_count', 'zonalCurrent100_300_Moving_min_9_mean', 'zonalCurrentSurface_5savgol_mean', 'temperatureSurface_Expanding_quantile_90', 'zonalCurrent100_300_Moving_quantile_10_2', 'euphoticDepth_Moving_median_5_mean', 'zonalCurrent100_300_June_quantile_1', 'nitrateSurface_Moving_quantile_99_3', 'euphoticDepth_Moving_median_9_mean', 'zonalCurrentSurface_Moving_skew_12_mean', 'meridionalCurrentSurface_Moving_min_6_mean', 'dissolvedOxygen100_300_Moving_quantile_99_7', 'salinity100_300_10wiener_mean', 'dissolvedOxygenSurface_Moving_std_9_mean', 'temperatureSurface_August_median', 'meridionalCurrentSurface_5savgol_mean', 'temperatureSurface_November_quantile_5', 'nitrateSurface_2spline_mean', 'zonalCurrent100_300_Moving_max_11_mean', 'zonalCurrentSurface_March_quantile_99', 'secchiDiskDepth_quantile_90', 'meridionalCurrentSurface_Moving_max_12_mean', 'meridionalCurrentSurface_Moving_std_8_mean', 'nitrateSurface_Moving_quantile_5_9', 'zonalCurrentSurface_Moving_min_6_mean', 'salinity100_300_Moving_quantile_90_5', 'salinity100_300_April_quantile_99', 'temperature100_300_May_min', 'phosphateSurface_July_quantile_90', 'dissolvedOxygen100_300_Moving_quantile_5_10', 'temperatureSurfacetrim_mean_10%', 'wave_Height_Moving_quantile_10_4', 'euphoticDepth_May_quantile_1', 'nitrateSurface_April_quantile_5', 'nitrateSurface_Moving_quantile_99_10', 'zonalCurrent100_300_Moving_quantile_10_10', 'dissolvedOxygenSurface_Moving_median_12_mean', 'meridionalCurrent100_300_Moving_quantile_5_10', 'salinity100_300_symiirorder1_mean', 'dissolvedOxygen100_300_Moving_min_9_mean', 'dissolvedOxygen100_300_Moving_quantile_95_12', 'zonalCurrent100_300_August_quantile_99', 'phosphateSurface_Moving_quantile_95_9', 'nitrateSurface_years_quantile_99', 'meridionalCurrent100_300_Moving_std_7_mean', 'chlorophyll_June_quantile_95', 'phosphateSurface_Moving_quantile_10_5', 'meridionalCurrent100_300_November_quantile_99', 'secchiDiskDepth_June_mean', 'phosphateSurface_January_quantile_10', 'temperature100_300_decimate_mean', 'zonalCurrentSurface_3medfilt_mean', 'secchiDiskDepth_Moving_min_6_mean', 'nitrate100_300_Moving_quantile_90_2', 'nitrateSurface_January_quantile_99', 'temperature100_300_mean_change_argrelmax', 'temperature100_300trim_mean_25%', 'euphoticDepth_July_quantile_1', 'zonalCurrentSurface_Expanding_max_mean', 'salinitySurface_51savgol_mean', 'phosphateSurface_Moving_median_5_mean', 'meridionalCurrent100_300_years_quantile_5', 'zonalCurrent100_300trim_mean_5%', 'wave_Height_years_quantile_95', 'dissolvedOxygen100_300_October_quantile_99', 'salinitySurface_Moving_quantile_95_3', 'meridionalCurrent100_300_years_quantile_10', 'nitrate100_300_Moving_average_3_mean', 'salinity100_300_September_min', 'phosphateSurface_Moving_quantile_95_6', 'nitrateSurface_December_median', 'temperatureSurface_Moving_average_10_mean', 'wave_Height_Moving_max_7_mean', 'meridionalCurrent100_300_Moving_quantile_90_12', 'salinity100_300_Moving_quantile_10_3', 'dissolvedOxygenSurface_Moving_skew_12_mean', 'euphoticDepth_Moving_skew_7_mean', 'salinitySurface_Moving_quantile_90_7', 'salinitySurface_Moving_average_8_mean', 'dissolvedOxygenSurface_Moving_quantile_5_9', 'phosphate100_300_September_quantile_90', 'dissolvedOxygen100_300_Moving_min_6_mean', 'zonalCurrentSurface_Moving_quantile_5_4', 'dissolvedOxygen100_300_Moving_quantile_10_11', 'euphoticDepth_June_median', 'salinitySurface_Moving_quantile_10_8', 'zonalCurrentSurface_May_quantile_10', 'phosphateSurface_Moving_std_4_mean', 'wave_Height_September_quantile_99', 'temperatureSurface_Expanding_quantile_5', 'phosphate100_300_Moving_median_6_mean', 'temperature100_300_January_min', 'phosphateSurface_mad', 'chlorophyll_Moving_skew_11_mean', 'euphoticDepth_Expanding_quantile_95', 'dissolvedOxygenSurface_neighbor_min', 'nitrateSurface_max_to_min_diff', 'nitrate100_300_June_quantile_1', 'euphoticDepth_Moving_quantile_10_11', 'euphoticDepth_October_mean', 'dissolvedOxygen100_300_Expanding_quantile_5', 'zonalCurrent100_300_January_quantile_5', 'meridionalCurrentSurface_Moving_average_4_mean', 'nitrate100_300_Moving_quantile_10_8', 'dissolvedOxygen100_300_years_quantile_90', 'euphoticDepth_symiirorder1_mean', 'zonalCurrentSurface_Moving_quantile_5_11', 'chlorophyll_December_quantile_99', 'nitrateSurface_quantile_90', 'wave_Height_Moving_max_9_mean', 'phosphate100_300_March_quantile_95', 'dissolvedOxygenSurface_symiirorder2_mean', 'chlorophyll_November_quantile_95', 'chlorophyll_June_quantile_10', 'dissolvedOxygenSurfaceiqr1', 'temperatureSurface_November_quantile_1', 'temperatureSurface_Expanding_median_mean', 'nitrateSurface_February_median', 'salinitySurface_Moving_skew_6_mean', 'temperature100_300_Moving_min_2_mean', 'dissolvedOxygenSurface_years_median', 'zonalCurrent100_300_quantile_95', 'salinitySurface_Moving_quantile_99_12', 'meridionalCurrentSurface_Moving_quantile_1_7', 'salinity100_300_April_median', 'meridionalCurrentSurface_quantile_1', 'nitrateSurface_Moving_min_3_mean', 'salinitySurface_Moving_quantile_99_6', 'euphoticDepth_50wiener_mean', 'nitrateSurface_April_quantile_1', 'nitrate100_300_October_quantile_10', 'euphoticDepth_June_quantile_1', 'chlorophyll_May_quantile_99', 'phosphateSurface_September_quantile_1', 'salinity100_300_Moving_min_5_mean', 'salinity100_300_Moving_quantile_99_5', 'nitrate100_300_November_quantile_95', 'nitrate100_300_June_quantile_10', 'dissolvedOxygen100_300_Moving_average_9_mean', 'zonalCurrent100_300_June_quantile_10', 'nitrate100_300_Moving_quantile_99_11', 'euphoticDepth_Moving_average_4_mean', 'chlorophyll_mean_change_argrelmin', 'meridionalCurrentSurface_Expanding_min_mean', 'dissolvedOxygenSurface_July_quantile_1', 'dissolvedOxygen100_300_Moving_median_8_mean', 'salinitySurface_Moving_skew_7_mean', 'dissolvedOxygenSurface_July_quantile_99', 'zonalCurrentSurface_April_quantile_95', 'phosphate100_300_Moving_quantile_95_11', 'zonalCurrent100_300_December_quantile_95', 'euphoticDepth_March_quantile_90', 'wave_Height_Moving_quantile_90_7', 'meridionalCurrentSurface_Moving_quantile_1_5', 'nitrate100_300_Moving_quantile_1_12', 'chlorophyll_4spline_mean', 'phosphate100_300trim_mean_25%', 'dissolvedOxygenSurface_Moving_quantile_90_8', 'zonalCurrentSurface_June_quantile_99', 'dissolvedOxygen100_300_September_mean', 'dissolvedOxygenSurface_mean_change_argrelmin', 'meridionalCurrentSurface_September_quantile_95', 'salinity100_300_February_quantile_1', 'euphoticDepth_May_quantile_5', 'salinity100_300_Expanding_median_mean', 'nitrateSurface_Expanding_median_mean', 'dissolvedOxygen100_300_October_quantile_95', 'dissolvedOxygenSurface_Moving_quantile_90_9', 'secchiDiskDepth_Moving_std_4_mean', 'chlorophyll_May_mean', 'zonalCurrentSurface_Moving_average_8_mean', 'meridionalCurrent100_300_Moving_quantile_1_6', 'phosphateSurface_Moving_quantile_90_10', 'meridionalCurrentSurface_Moving_std_5_mean', 'salinity100_300_August_quantile_99', 'nitrate100_300_October_quantile_1', 'zonalCurrent100_300_Moving_average_7_mean', 'meridionalCurrent100_300_Moving_std_4_mean', 'temperature100_300_quantile_5', 'chlorophyll_Moving_std_12_mean', 'nitrateSurface_Moving_std_2_mean', 'zonalCurrent100_300_20wiener_mean', 'temperature100_300_Moving_max_9_mean', 'meridionalCurrentSurface_Moving_quantile_90_7', 'dissolvedOxygen100_300_Moving_skew_7_mean', 'nitrateSurface_Expanding_quantile_90', 'dissolvedOxygenSurface_10medfilt_mean', 'chlorophyll_Moving_std_3_mean', 'meridionalCurrent100_300_May_median', 'dissolvedOxygen100_300_January_quantile_10', 'secchiDiskDepth_May_quantile_99', 'salinitySurface_Moving_median_4_mean', 'meridionalCurrentSurface_4wiener_mean', 'nitrate100_300_median', 'temperature100_300_January_quantile_99', 'salinity100_300_Moving_quantile_90_7', 'secchiDiskDepth_August_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_5_11', 'phosphate100_300_August_quantile_90', 'chlorophyll_Moving_std_10_mean', 'wave_Height_May_quantile_95', 'meridionalCurrentSurface_Moving_quantile_90_5', 'dissolvedOxygen100_300_December_quantile_1', 'phosphateSurface_Moving_median_10_mean', 'nitrate100_300_May_max', 'temperature100_300trim_mean_20%', 'euphoticDepth_Moving_quantile_99_6', 'dissolvedOxygen100_300_May_quantile_5', 'nitrate100_300_Moving_quantile_10_10', 'temperature100_300_August_median', 'nitrateSurface_March_quantile_99', 'dissolvedOxygen100_300_Moving_max_5_mean', 'euphoticDepth_Moving_quantile_10_2', 'phosphate100_300_Moving_quantile_95_7', 'nitrateSurface_Moving_average_2_mean', 'nitrateSurface_Moving_median_11_mean', 'phosphateSurface_years_max', 'temperatureSurface_January_quantile_90', 'salinitySurface_September_quantile_10', 'temperatureSurface_April_quantile_5', 'temperatureSurface_December_quantile_5', 'nitrate100_300_October_max', 'zonalCurrent100_300_Moving_min_7_mean', 'dissolvedOxygenSurface_Moving_min_11_mean', 'zonalCurrentSurface_Moving_std_9_mean', 'phosphate100_300_3gauss_spline_mean', 'salinitySurface_March_mean', 'salinitySurface_Moving_quantile_99_11', 'secchiDiskDepth_Expanding_min_mean', 'nitrate100_300_Moving_min_8_mean', 'salinitySurface_March_quantile_10', 'nitrate100_300_argrelminmax_count', 'temperatureSurface_August_mean', 'phosphateSurface_March_quantile_10', 'salinity100_300_Moving_skew_7_mean', 'secchiDiskDepth_March_min', 'phosphateSurface_Moving_quantile_95_11', 'chlorophyll_Moving_quantile_95_4', 'secchiDiskDepth_years_median', 'zonalCurrent100_300_4wiener_mean', 'zonalCurrentSurface_April_quantile_99', 'secchiDiskDepth_Moving_median_5_mean', 'phosphateSurface_Moving_min_7_mean', 'euphoticDepth_November_quantile_90', 'nitrateSurface_Moving_std_5_mean', 'temperature100_300_November_quantile_5', 'temperature100_300_May_quantile_95', 'phosphate100_300_May_quantile_95', 'phosphate100_300_December_min', 'phosphateSurface_years_std', 'nitrate100_300_Moving_min_2_mean', 'zonalCurrent100_300_Moving_quantile_95_3', 'salinity100_300_March_max', 'zonalCurrent100_300_years_median', 'wave_Height_Moving_quantile_10_8', 'wave_Height_Moving_max_12_mean', 'nitrate100_300_Moving_quantile_90_3', 'meridionalCurrent100_300_Moving_median_4_mean', 'salinitySurface_Moving_quantile_10_3', 'zonalCurrent100_300_November_median', 'meridionalCurrentSurface_September_median', 'nitrateSurface_Moving_max_4_mean', 'zonalCurrent100_300_Moving_quantile_99_7', 'euphoticDepth_April_mean', 'secchiDiskDepth_Moving_max_4_mean', 'salinity100_300_September_quantile_5', 'phosphate100_300_Moving_quantile_5_5', 'meridionalCurrent100_300_Moving_quantile_95_9', 'euphoticDepth_Moving_std_5_mean', 'nitrate100_300_April_median', 'nitrate100_300_December_median', 'salinity100_300_September_quantile_10', 'secchiDiskDepth_decimate_mean', 'dissolvedOxygenSurface_51savgol_mean', 'temperature100_300_Moving_average_8_mean', 'meridionalCurrent100_300_quantile_99', 'secchiDiskDepth_symiirorder2_mean', 'chlorophyll_February_quantile_90', 'phosphateSurface_May_quantile_5', 'salinity100_300_mean_change_peak_rate', 'temperature100_300_50wiener_mean', 'phosphate100_300_Moving_std_5_mean', 'zonalCurrentSurface_Moving_std_5_mean', 'meridionalCurrentSurface_3spline_mean', 'euphoticDepth_February_quantile_1', 'secchiDiskDepth_Moving_quantile_95_7', 'chlorophyll_January_mean', 'phosphateSurface_Moving_quantile_1_10', 'secchiDiskDepth_4spline_mean', 'secchiDiskDepth_January_quantile_1', 'chlorophyll_February_quantile_10', 'temperatureSurface_Moving_skew_6_mean', 'meridionalCurrentSurface_Moving_quantile_90_3', 'chlorophyll_March_quantile_90', 'meridionalCurrent100_300_Moving_quantile_5_12', 'dissolvedOxygenSurface_symiirorder1_mean', 'zonalCurrent100_300_February_quantile_1', 'phosphate100_300_Moving_skew_9_mean', 'dissolvedOxygenSurface_April_median', 'dissolvedOxygenSurface_September_median', 'phosphate100_300_March_quantile_5', 'zonalCurrent100_300_50wiener_mean', 'chlorophyll_Moving_quantile_90_3', 'secchiDiskDepth_March_quantile_99', 'meridionalCurrentSurface_Moving_min_8_mean', 'dissolvedOxygenSurface_Moving_skew_9_mean', 'zonalCurrentSurface_Moving_max_4_mean', 'zonalCurrent100_300_Moving_quantile_99_3', 'meridionalCurrent100_300_Moving_average_7_mean', 'chlorophyll_February_quantile_1', 'euphoticDepth_Moving_quantile_10_12', 'temperatureSurface_Hilbert_mean', 'chlorophyll_February_quantile_99', 'salinity100_300_March_quantile_1', 'meridionalCurrentSurface_Moving_quantile_10_12', 'dissolvedOxygenSurface_Moving_median_10_mean', 'meridionalCurrentSurface_Moving_quantile_10_10', 'meridionalCurrent100_300_Moving_quantile_99_8', 'dissolvedOxygenSurface_Expanding_min_mean', 'phosphateSurface_Expanding_max_mean', 'salinity100_300_January_median', 'nitrateSurface_Moving_quantile_1_7', 'salinity100_300_Moving_quantile_99_2', 'nitrate100_300_March_median', 'secchiDiskDepthtrim_mean_5%', 'meridionalCurrentSurface_Moving_quantile_95_9', 'chlorophyll_4wiener_mean', 'salinitySurface_Moving_max_5_mean', 'salinitySurface_Moving_quantile_1_5', 'nitrate100_300_Moving_quantile_99_8', 'zonalCurrentSurface_December_quantile_10', 'temperature100_300_January_quantile_90', 'zonalCurrent100_300_Moving_std_5_mean', 'zonalCurrent100_300_Moving_quantile_5_9', 'phosphate100_300_4gauss_spline_mean', 'dissolvedOxygen100_300_Moving_skew_11_mean', 'dissolvedOxygen100_300trim_mean_5%', 'meridionalCurrent100_300_Moving_quantile_90_9', 'chlorophyll_years_quantile_90', 'temperature100_300_Moving_quantile_90_8', 'secchiDiskDepth_Moving_min_11_mean', 'nitrate100_300_February_quantile_5', 'nitrateSurface_years_quantile_5', 'temperatureSurface_October_quantile_99', 'phosphate100_300_December_quantile_99', 'euphoticDepth_June_quantile_95', 'nitrate100_300_Moving_quantile_5_2', 'nitrateSurface_Moving_quantile_99_9', 'secchiDiskDepth_Moving_quantile_90_4', 'temperatureSurface_July_min', 'zonalCurrentSurface_March_quantile_10', 'temperatureSurface_Moving_quantile_99_11', 'phosphate100_300_October_quantile_90', 'phosphateSurface_Expanding_quantile_1', 'nitrate100_300_Expanding_quantile_90', 'salinity100_300_Moving_quantile_1_5', 'nitrate100_300_February_quantile_95', 'temperatureSurface_August_quantile_99', 'wave_Height_Moving_quantile_1_12', 'dissolvedOxygen100_300_Moving_quantile_95_9', 'phosphate100_300_Moving_max_5_mean', 'chlorophyll_10medfilt_mean', 'dissolvedOxygen100_300_Moving_std_6_mean', 'salinitySurface_Moving_max_4_mean', 'nitrate100_300_September_max', 'temperature100_300_Expanding_quantile_1', 'secchiDiskDepth_mean_change_argrelmax', 'zonalCurrentSurface_Moving_quantile_90_9', 'phosphateSurface_Moving_quantile_95_10', 'salinitySurface_April_max', 'nitrateSurface_January_quantile_1', 'chlorophyll_Moving_quantile_1_8', 'wave_Height_Moving_quantile_5_5', 'nitrate100_300_May_quantile_95', 'dissolvedOxygenSurface_years_mean', 'secchiDiskDepth_September_quantile_99', 'dissolvedOxygen100_300_Moving_max_4_mean', 'chlorophyll_Expanding_quantile_5', 'salinitySurface_Moving_quantile_10_6', 'wave_Height_Moving_quantile_10_7', 'phosphateSurface_Moving_min_12_mean', 'salinity100_300_Moving_max_7_mean', 'meridionalCurrentSurface_Moving_median_4_mean', 'salinitySurface_Moving_quantile_99_7', 'zonalCurrent100_300_Moving_skew_6_mean', 'euphoticDepth_Moving_max_9_mean', 'salinitySurface_Moving_quantile_95_4', 'euphoticDepth_Moving_average_6_mean', 'nitrateSurface_Moving_median_6_mean', 'dissolvedOxygen100_300_December_quantile_90', 'wave_Height_Moving_average_5_mean', 'temperatureSurface_February_quantile_1', 'zonalCurrentSurface_February_quantile_5', 'salinitySurfacetrim_mean_15%', 'phosphate100_300_50wiener_mean', 'temperatureSurface_Moving_median_9_mean', 'meridionalCurrentSurface_Moving_quantile_5_8', 'temperature100_300_Moving_std_9_mean', 'wave_Height_Moving_quantile_5_8', 'chlorophyll_20wiener_mean', 'zonalCurrent100_300_Moving_quantile_90_7', 'nitrateSurface_Moving_quantile_99_8', 'nitrateSurface_Moving_quantile_99_11', 'phosphate100_300_Moving_skew_11_mean', 'phosphateSurface_Moving_std_12_mean', 'zonalCurrent100_300_quantile_1', 'zonalCurrent100_300trim_mean_10%', 'phosphate100_300_Moving_quantile_95_10', 'temperatureSurface_Moving_max_3_mean', 'temperatureSurface_Moving_quantile_10_5', 'salinity100_300_December_quantile_99', 'dissolvedOxygen100_300_November_median', 'salinitySurface_Moving_std_5_mean', 'meridionalCurrentSurface_argrelmax_count', 'nitrate100_300_Moving_quantile_99_10', 'dissolvedOxygenSurface_Moving_min_10_mean', 'nitrateSurface_quantile_1', 'secchiDiskDepth_Moving_quantile_95_3', 'euphoticDepth_Moving_min_11_mean', 'salinity100_300_November_quantile_10', 'phosphate100_300_March_quantile_10', 'dissolvedOxygen100_300_October_quantile_90', 'zonalCurrentSurface_Moving_median_6_mean', 'dissolvedOxygen100_300_Moving_quantile_10_5', 'dissolvedOxygenSurface_Moving_quantile_95_9', 'dissolvedOxygenSurface_Moving_quantile_10_9', 'salinitySurface_Expanding_std_mean', 'meridionalCurrent100_300_May_quantile_5', 'secchiDiskDepth_Expanding_average_mean', 'dissolvedOxygen100_300_October_median', 'nitrateSurface_Moving_median_4_mean', 'secchiDiskDepth_February_min', 'zonalCurrentSurface_argrelminmax_count', 'nitrate100_300_50wiener_mean', 'dissolvedOxygen100_300_max_to_min', 'dissolvedOxygen100_300_July_quantile_5', 'phosphate100_300_Moving_quantile_95_6', 'temperature100_300_January_quantile_10', 'phosphate100_300_Moving_quantile_1_4', 'dissolvedOxygen100_300iqr', 'temperature100_300_November_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_1_7', 'secchiDiskDepth_Moving_min_7_mean', 'phosphate100_300_February_quantile_95', 'phosphateSurface_October_quantile_95', 'phosphate100_300_April_quantile_90', 'nitrateSurface_November_median', 'salinitySurface_November_quantile_95', 'nitrateSurface_Moving_min_5_mean', 'salinity100_300_Moving_quantile_10_11', 'temperature100_300_December_quantile_95', 'zonalCurrent100_300_Expanding_quantile_90', 'meridionalCurrentSurface_Moving_skew_6_mean', 'meridionalCurrentSurface_August_quantile_99', 'dissolvedOxygenSurface_Moving_std_12_mean', 'phosphateSurface_Moving_skew_11_mean', 'chlorophyll_Expanding_quantile_99', 'euphoticDepth_Moving_quantile_1_4', 'nitrate100_300_February_median', 'euphoticDepth_Expanding_max_mean', 'salinity100_300_Moving_median_11_mean', 'wave_Height_Moving_quantile_99_8', 'meridionalCurrent100_300_Moving_average_2_mean', 'secchiDiskDepth_Moving_quantile_99_2', 'nitrate100_300_Moving_std_11_mean', 'phosphate100_300_Moving_quantile_90_3', 'meridionalCurrent100_300_mad', 'chlorophyll_Moving_skew_7_mean', 'secchiDiskDepth_November_quantile_5', 'phosphateSurface_Moving_max_5_mean', 'phosphateSurface_quantile_5', 'zonalCurrentSurface_Moving_quantile_90_12', 'dissolvedOxygenSurface_Moving_quantile_90_10', 'zonalCurrent100_300_Moving_quantile_95_6', 'phosphateSurface_December_quantile_90', 'salinity100_300_Moving_average_3_mean', 'zonalCurrent100_300_March_quantile_95', 'meridionalCurrent100_300_Moving_quantile_95_11', 'temperature100_300_Moving_max_12_mean', 'phosphate100_300_February_quantile_1', 'meridionalCurrentSurface_March_quantile_5', 'chlorophyll_Moving_quantile_5_2', 'phosphateSurface_Moving_quantile_1_11', 'nitrateSurface_Moving_max_8_mean', 'dissolvedOxygenSurface_June_quantile_99', 'phosphate100_300_Moving_std_2_mean', 'nitrate100_300_June_quantile_90', 'dissolvedOxygenSurface_June_median', 'chlorophyll_August_quantile_10', 'temperatureSurface_years_skew', 'dissolvedOxygen100_300_July_quantile_90', 'temperature100_300_December_quantile_90', 'zonalCurrent100_300_December_quantile_10', 'meridionalCurrent100_300_years_max', 'dissolvedOxygen100_300_Moving_min_7_mean', 'salinitySurface_years_quantile_10', 'zonalCurrentSurface_Expanding_quantile_95', 'zonalCurrentSurface_January_quantile_99', 'zonalCurrent100_300_Expanding_average_mean', 'nitrate100_300_September_median', 'wave_Height_Moving_quantile_90_3', 'dissolvedOxygenSurface_January_quantile_1', 'chlorophyll_Expanding_quantile_90', 'nitrateSurface_3spline_mean', 'salinity100_300_November_quantile_99', 'zonalCurrent100_300_October_quantile_95', 'dissolvedOxygenSurface_October_quantile_10', 'nitrateSurface_June_median', 'zonalCurrent100_300_February_quantile_5', 'temperature100_300_October_quantile_10', 'euphoticDepth_March_max', 'euphoticDepth_mean_change_argrelmin', 'wave_Height_Moving_quantile_5_12', 'meridionalCurrent100_300_November_median', 'secchiDiskDepth_November_quantile_90', 'nitrateSurface_January_median', 'dissolvedOxygen100_300_Moving_median_7_mean', 'euphoticDepth_November_median', 'salinitySurface_January_quantile_1', 'nitrateSurface_Moving_skew_10_mean', 'dissolvedOxygenSurface_Moving_quantile_1_6', 'dissolvedOxygen100_300_April_quantile_99', 'phosphateSurfacetrim_mean_15%', 'nitrateSurface_April_max', 'meridionalCurrent100_300_Hilbert_mean', 'salinity100_300_Moving_average_6_mean', 'phosphate100_300_argrelmax_count', 'phosphate100_300_Moving_min_12_mean', 'phosphateSurface_July_median', 'phosphate100_300_Moving_quantile_99_8', 'dissolvedOxygen100_300_August_quantile_90', 'nitrateSurface_March_quantile_10', 'meridionalCurrent100_300_20wiener_mean', 'salinitySurface_May_quantile_90', 'temperature100_300_Expanding_quantile_99', 'chlorophyll_Moving_quantile_95_9', 'nitrateSurface_Moving_quantile_10_11', 'salinity100_300_Moving_quantile_5_8', 'zonalCurrentSurface_Moving_quantile_5_2', 'zonalCurrentSurface_quantile_99', 'temperatureSurface_quantile_5', 'meridionalCurrentSurface_Moving_quantile_95_11', 'meridionalCurrentSurface_Moving_min_10_mean', 'phosphateSurface_Moving_std_8_mean', 'secchiDiskDepth_August_quantile_5', 'zonalCurrent100_300_5savgol_mean', 'nitrate100_300_quantile_10', 'dissolvedOxygenSurface_Expanding_quantile_95', 'salinity100_300_June_quantile_5', 'wave_Height_December_quantile_10', 'temperatureSurface_Expanding_average_mean', 'secchiDiskDepth_October_quantile_10', 'phosphate100_300_Moving_median_11_mean', 'euphoticDepth_December_quantile_95', 'meridionalCurrent100_300_Moving_quantile_90_3', 'meridionalCurrentSurface_Moving_quantile_5_2', 'zonalCurrent100_300_Moving_average_5_mean', 'dissolvedOxygen100_300_December_quantile_95', 'chlorophyll_Moving_min_7_mean', 'zonalCurrent100_300_Expanding_quantile_1', 'nitrate100_300_Moving_quantile_10_2', 'meridionalCurrentSurface_Moving_quantile_95_10', 'salinity100_300_years_quantile_90', 'nitrate100_300_January_max', 'wave_Height_June_quantile_5', 'euphoticDepth_October_median', 'phosphateSurface_3gauss_spline_mean', 'dissolvedOxygenSurface_November_quantile_99', 'chlorophyll_Moving_quantile_1_10', 'nitrate100_300_Moving_average_2_mean', 'chlorophyll_December_quantile_1', 'temperature100_300_years_quantile_5', 'phosphateSurface_argrelminmax_count', 'dissolvedOxygen100_300_years_std', 'dissolvedOxygenSurface_Expanding_std_mean', 'chlorophyll_March_std', 'phosphateSurface_September_quantile_90', 'zonalCurrentSurface_50wiener_mean', 'wave_Height_October_quantile_1', 'phosphateSurface_Moving_average_3_mean', 'chlorophyll_Moving_quantile_90_10', 'temperature100_300_quantile_90', 'temperature100_300_September_quantile_90', 'meridionalCurrentSurface_years_min', 'salinity100_300_Moving_quantile_1_9', 'secchiDiskDepth_October_min', 'salinitySurface_Moving_median_10_mean', 'chlorophyll_February_quantile_95', 'salinitySurface_Expanding_quantile_99', 'secchiDiskDepth_neighbor_max', 'zonalCurrentSurface_December_median', 'chlorophyll_Moving_quantile_1_3', 'zonalCurrent100_300_years_std', 'phosphate100_300_September_max', 'temperatureSurface_Moving_quantile_90_4', 'phosphateSurface_October_median', 'phosphateSurface_quantile_99', 'meridionalCurrent100_300_Moving_quantile_5_3', 'euphoticDepth_May_quantile_10', 'meridionalCurrentSurface_Moving_quantile_5_7', 'nitrateSurface_Moving_quantile_95_2', 'dissolvedOxygenSurface_5medfilt_mean', 'phosphateSurface_Moving_average_9_mean', 'temperatureSurface_Moving_min_8_mean', 'salinitySurface_June_mean', 'phosphateSurface_May_quantile_99', 'zonalCurrent100_300_April_quantile_95', 'zonalCurrent100_300_Moving_quantile_99_6', 'dissolvedOxygenSurface_Moving_quantile_10_3', 'salinitySurface_August_max', 'meridionalCurrentSurfaceiqr1', 'phosphate100_300_March_min', 'nitrateSurface_Moving_quantile_1_3', 'dissolvedOxygenSurface_Moving_average_6_mean', 'euphoticDepth_Expanding_min_mean', 'dissolvedOxygenSurface_Moving_quantile_10_10', 'nitrateSurface_June_quantile_10', 'zonalCurrentSurface_Moving_min_11_mean', 'zonalCurrentSurface_Moving_quantile_90_2', 'chlorophyll_Moving_min_8_mean', 'secchiDiskDepth_Moving_quantile_5_3', 'dissolvedOxygenSurface_July_quantile_90', 'temperatureSurface_Moving_median_11_mean', 'phosphateSurface_Moving_quantile_10_12', 'temperature100_300_quantile_1', 'nitrateSurface_quantile_99', 'salinitySurface_June_quantile_5', 'euphoticDepth_50medfilt_mean', 'phosphate100_300_Moving_skew_8_mean', 'temperatureSurface_Moving_quantile_10_6', 'zonalCurrentSurface_September_median', 'wave_Height_Moving_quantile_5_3', 'temperatureSurface_Moving_quantile_1_9', 'zonalCurrentSurface_Moving_median_11_mean', 'dissolvedOxygen100_300_May_quantile_99', 'dissolvedOxygenSurface_June_quantile_10', 'zonalCurrentSurface_May_quantile_99', 'meridionalCurrentSurface_Moving_quantile_99_8', 'chlorophyll_June_quantile_90', 'wave_Height_November_quantile_99', 'temperature100_300_May_quantile_5', 'phosphateSurface_25medfilt_mean', 'phosphateSurface_January_quantile_99', 'nitrateSurface_May_quantile_5', 'meridionalCurrentSurface_Expanding_quantile_5', 'meridionalCurrentSurface_December_quantile_99', 'phosphateSurfacetrim_mean_25%', 'euphoticDepth_April_quantile_90', 'meridionalCurrent100_300_Moving_quantile_99_9', 'salinitySurface_Moving_average_4_mean', 'nitrateSurface_Moving_max_11_mean', 'zonalCurrent100_300_June_quantile_99', 'dissolvedOxygenSurface_March_quantile_10', 'meridionalCurrentSurface_3medfilt_mean', 'salinitySurface_Moving_max_7_mean', 'dissolvedOxygen100_300_January_mean', 'dissolvedOxygenSurface_Moving_quantile_5_3', 'meridionalCurrent100_300_March_quantile_5', 'secchiDiskDepth_Moving_average_6_mean', 'temperature100_300_Moving_median_5_mean', 'zonalCurrent100_300_Moving_quantile_1_12', 'meridionalCurrentSurface_Moving_quantile_10_6', 'salinity100_300_May_quantile_5', 'zonalCurrent100_300_Moving_quantile_10_3', 'zonalCurrent100_300_October_quantile_1', 'chlorophyll_3spline_mean', 'phosphate100_300_argrelminmax_count', 'phosphate100_300_June_min', 'secchiDiskDepthiqr', 'secchiDiskDepth_November_mean', 'zonalCurrentSurface_May_quantile_95', 'dissolvedOxygenSurface_Moving_quantile_5_2', 'salinity100_300_August_max', 'secchiDiskDepth_August_quantile_99', 'temperatureSurface_Moving_quantile_10_7', 'temperatureSurface_Moving_max_11_mean', 'secchiDiskDepth_October_max', 'nitrate100_300_years_quantile_95', 'salinity100_300_years_max', 'salinitySurface_November_quantile_99', 'euphoticDepth_September_quantile_95', 'salinitySurface_Moving_average_7_mean', 'zonalCurrent100_300_July_median', 'nitrate100_300_March_quantile_90', 'nitrateSurface_October_quantile_5', 'temperature100_300_Moving_min_6_mean', 'wave_Height_Moving_quantile_99_6', 'nitrateSurface_February_quantile_1', 'salinity100_300_Moving_std_9_mean', 'euphoticDepth_Expanding_average_mean', 'dissolvedOxygen100_300_Expanding_quantile_99', 'dissolvedOxygenSurface_Moving_quantile_5_5', 'meridionalCurrent100_300_51savgol_mean', 'secchiDiskDepth_Moving_quantile_90_7', 'zonalCurrent100_300_February_median', 'zonalCurrent100_300_5medfilt_mean', 'temperature100_300_July_median', 'temperature100_300_25medfilt_mean', 'euphoticDepth_October_quantile_99', 'phosphate100_300_quantile_99', 'zonalCurrent100_300_Moving_quantile_1_3', 'nitrate100_300_February_quantile_1', 'zonalCurrent100_300_Moving_quantile_95_2', 'zonalCurrent100_300_Moving_min_10_mean', 'phosphateSurface_July_quantile_99', 'salinitySurface_May_quantile_5', 'phosphate100_300_February_mean', 'secchiDiskDepth_Moving_quantile_95_10', 'temperatureSurface_Moving_median_10_mean', 'salinity100_300_December_min', 'euphoticDepth_May_quantile_99', 'secchiDiskDepth_Moving_skew_10_mean', 'dissolvedOxygen100_300_June_quantile_95', 'nitrateSurface_Moving_quantile_95_5', 'salinity100_300_July_quantile_95', 'chlorophyll_years_max', 'euphoticDepth_Moving_max_12_mean', 'temperature100_300_Moving_median_3_mean', 'meridionalCurrent100_300_November_quantile_1', 'phosphateSurface_Moving_quantile_5_3', 'nitrateSurface_Moving_quantile_5_2', 'euphoticDepth_Moving_quantile_95_11', 'nitrate100_300_Moving_max_3_mean', 'secchiDiskDepth_Moving_quantile_10_11', 'phosphate100_300_Moving_average_8_mean', 'zonalCurrentSurface_Moving_quantile_90_7', 'secchiDiskDepth_December_median', 'meridionalCurrentSurface_Moving_quantile_10_3', 'salinity100_300_July_quantile_5', 'phosphate100_300_Moving_quantile_99_6', 'phosphateSurface_August_quantile_5', 'nitrate100_300_Moving_std_7_mean', 'dissolvedOxygenSurfacetrim_mean_10%', 'chlorophylltrim_mean_20%', 'zonalCurrentSurface_Moving_quantile_99_12', 'secchiDiskDepth_October_quantile_90', 'wave_Height_mad', 'euphoticDepth_Moving_quantile_90_5', 'phosphate100_300_October_quantile_99', 'dissolvedOxygenSurface_July_median', 'secchiDiskDepth_Moving_quantile_99_10', 'meridionalCurrent100_300_March_quantile_95', 'salinity100_300_April_quantile_10', 'chlorophyll_August_max', 'meridionalCurrentSurface_January_quantile_10', 'secchiDiskDepth_quantile_99', 'temperatureSurface_Moving_quantile_1_8', 'salinitySurface_Moving_min_5_mean', 'temperature100_300_Expanding_quantile_10', 'meridionalCurrent100_300_Moving_quantile_5_11', 'phosphateSurface_February_quantile_10', 'temperatureSurface_September_quantile_1', 'meridionalCurrentSurface_Moving_quantile_90_11', 'phosphate100_300_Moving_max_6_mean', 'zonalCurrentSurfacetrim_mean_20%', 'zonalCurrentSurfacetrim_mean_25%', 'meridionalCurrentSurface_December_quantile_95', 'meridionalCurrent100_300_March_quantile_1', 'temperatureSurface_years_quantile_95', 'salinitySurface_July_quantile_99', 'chlorophyll_Moving_quantile_99_11', 'euphoticDepth_Moving_skew_10_mean', 'nitrate100_300_July_quantile_90', 'secchiDiskDepth_Moving_quantile_10_10', 'nitrateSurface_Moving_quantile_90_8', 'salinitySurface_January_quantile_5', 'euphoticDepth_Expanding_median_mean', 'nitrateSurface_Moving_median_10_mean', 'phosphate100_300_September_quantile_95', 'secchiDiskDepth_April_median', 'phosphateSurface_August_quantile_1', 'salinity100_300_Moving_quantile_95_2', 'phosphate100_300_Moving_average_11_mean', 'chlorophyll_5savgol_mean', 'dissolvedOxygenSurface_quantile_5', 'nitrate100_300_May_quantile_90', 'temperature100_300_Moving_quantile_1_9', 'dissolvedOxygenSurface_December_median', 'temperatureSurface_argrelminmax_count', 'salinity100_300_March_min', 'zonalCurrentSurface_Moving_quantile_10_5', 'chlorophyll_October_quantile_1', 'zonalCurrentSurface_July_quantile_10', 'nitrateSurface_October_quantile_90', 'secchiDiskDepth_25savgol_mean', 'phosphateSurface_Moving_min_4_mean', 'temperature100_300_Moving_median_7_mean', 'nitrate100_300_Moving_quantile_95_10', 'nitrateSurfacetrim_mean_5%', 'wave_Height_Moving_quantile_1_11', 'phosphateSurface_Moving_quantile_99_10', 'chlorophyll_November_quantile_10', 'secchiDiskDepth_August_quantile_90', 'wave_Height_Moving_max_2_mean', 'dissolvedOxygen100_300_Moving_median_4_mean', 'wave_Height_July_quantile_1', 'nitrateSurface_Moving_quantile_10_6', 'meridionalCurrentSurface_November_quantile_5', 'zonalCurrent100_300_Moving_std_8_mean', 'nitrate100_300_years_quantile_99', 'nitrate100_300_May_min', 'meridionalCurrent100_300_February_quantile_10', 'zonalCurrentSurface_March_median', 'temperatureSurface_11savgol_mean', 'temperatureSurface_July_quantile_95', 'temperature100_300_Moving_quantile_10_10', 'temperature100_300_August_mean', 'meridionalCurrent100_300_June_quantile_1', 'temperature100_300_Moving_max_4_mean', 'euphoticDepth_Expanding_quantile_10', 'temperature100_300_Moving_std_4_mean', 'salinity100_300_10medfilt_mean', 'nitrateSurface_April_quantile_90', 'nitrate100_300_December_quantile_5', 'meridionalCurrentSurface_August_quantile_95', 'salinity100_300_Moving_quantile_95_7', 'euphoticDepth_Moving_quantile_5_12', 'euphoticDepth_Expanding_quantile_5', 'nitrate100_300_Moving_median_10_mean', 'phosphate100_300_Moving_quantile_95_5', 'salinitySurface_August_mean', 'zonalCurrent100_300_Moving_quantile_99_4', 'nitrate100_300_January_quantile_95', 'chlorophyll_11savgol_mean', 'chlorophyll_July_quantile_1', 'dissolvedOxygen100_300_Moving_quantile_10_3', 'phosphate100_300_Moving_quantile_10_2', 'zonalCurrentSurface_Moving_quantile_5_5', 'chlorophyll_March_median', 'temperatureSurface_Moving_min_10_mean', 'zonalCurrentSurface_June_quantile_5', 'phosphate100_300_Moving_quantile_90_4', 'meridionalCurrentSurface_Moving_average_9_mean', 'zonalCurrent100_300_Moving_quantile_10_9', 'meridionalCurrentSurface_Moving_max_10_mean', 'wave_Height_Moving_quantile_90_5', 'dissolvedOxygen100_300_Moving_std_4_mean', 'chlorophyll_Moving_std_11_mean', 'temperatureSurface_Moving_max_6_mean', 'euphoticDepth_Moving_quantile_99_8', 'wave_Height_Moving_min_7_mean', 'phosphateSurface_Moving_quantile_90_2', 'phosphateSurface_March_quantile_1', 'salinity100_300_Moving_quantile_1_3', 'dissolvedOxygenSurface_Moving_quantile_10_11', 'salinitySurface_Moving_std_4_mean', 'nitrate100_300_Moving_std_2_mean', 'meridionalCurrent100_300_July_quantile_5', 'temperature100_300_Moving_quantile_99_9', 'zonalCurrent100_300_April_median', 'euphoticDepth_December_quantile_1', 'euphoticDepth_January_median', 'dissolvedOxygen100_300_51savgol_mean', 'dissolvedOxygen100_300_April_mean', 'nitrate100_300_Moving_quantile_95_2', 'nitrate100_300_January_quantile_1', 'salinitySurface_May_quantile_10', 'zonalCurrent100_300_Expanding_quantile_95', 'wave_Height_20wiener_mean', 'salinity100_300_4wiener_mean', 'phosphateSurface_July_quantile_5', 'meridionalCurrentSurface_Moving_quantile_99_9', 'zonalCurrent100_300_September_quantile_99', 'meridionalCurrentSurface_October_quantile_10', 'secchiDiskDepth_July_quantile_5', 'chlorophyll_years_quantile_1', 'meridionalCurrentSurface_Moving_std_3_mean', 'salinity100_300_Moving_quantile_10_2', 'euphoticDepth_years_max', 'temperatureSurface_October_max', 'chlorophyll_February_std', 'euphoticDepth_Moving_median_6_mean', 'zonalCurrent100_300_Moving_std_12_mean', 'meridionalCurrentSurface_Moving_max_5_mean', 'temperature100_300_Moving_quantile_5_9', 'zonalCurrentSurface_Moving_quantile_1_4', 'nitrateSurface_Moving_median_7_mean', 'temperatureSurface_Moving_std_3_mean', 'temperature100_300_August_quantile_95', 'nitrate100_300_Moving_max_9_mean', 'dissolvedOxygen100_300_Moving_quantile_10_4', 'phosphate100_300_51savgol_mean', 'chlorophyll_Moving_average_8_mean', 'salinity100_300_April_quantile_5', 'dissolvedOxygenSurface_March_mean', 'salinity100_300_Moving_min_4_mean', 'nitrateSurfaceiqr', 'phosphate100_300_Moving_quantile_90_9', 'phosphateSurface_20wiener_mean', 'dissolvedOxygenSurface_Moving_quantile_99_6', 'temperatureSurface_December_max', 'phosphate100_300_decimate_mean', 'zonalCurrentSurface_4gauss_spline_mean', 'zonalCurrent100_300_Moving_std_11_mean', 'temperatureSurface_Moving_quantile_10_8', 'nitrateSurface_March_std', 'zonalCurrentSurface_2gauss_spline_mean', 'salinitySurface_October_quantile_99', 'temperature100_300_August_quantile_90', 'nitrate100_300_July_mean', 'secchiDiskDepth_50medfilt_mean', 'dissolvedOxygen100_300_January_quantile_95', 'meridionalCurrentSurface_Moving_quantile_95_8', 'temperature100_300_October_quantile_95', 'secchiDiskDepth_June_max', 'phosphate100_300_November_quantile_5', 'phosphateSurface_quantile_95', 'phosphateSurface_October_quantile_1', 'phosphateSurface_50wiener_mean', 'dissolvedOxygen100_300_symiirorder2_mean', 'phosphate100_300_25medfilt_mean', 'chlorophyll_51savgol_mean', 'secchiDiskDepth_Moving_quantile_1_3', 'nitrateSurface_Moving_quantile_90_4', 'zonalCurrent100_300_June_quantile_95', 'euphoticDepth_Moving_average_10_mean', 'secchiDiskDepth_Moving_quantile_5_12', 'phosphate100_300_quantile_10', 'salinity100_300_Moving_quantile_10_8', 'temperature100_300_October_mean', 'temperatureSurface_Moving_quantile_99_4', 'zonalCurrent100_300_Moving_max_12_mean', 'salinity100_300_April_max', 'temperatureSurface_September_median', 'chlorophyll_July_quantile_95', 'euphoticDepth_Moving_skew_11_mean', 'nitrate100_300_Moving_max_2_mean', 'chlorophyll_November_quantile_90', 'temperatureSurface_Moving_quantile_1_2', 'salinity100_300_Moving_std_7_mean', 'wave_Height_Moving_skew_7_mean', 'temperature100_300_Moving_quantile_5_8', 'dissolvedOxygenSurface_4wiener_mean', 'dissolvedOxygenSurface_Moving_quantile_95_3', 'zonalCurrentSurface_March_quantile_5', 'temperatureSurface_Moving_quantile_90_10', 'wave_Height_Moving_quantile_90_12', 'temperature100_300_max_to_min_diff', 'meridionalCurrentSurface_Moving_quantile_99_4', 'secchiDiskDepth_Moving_quantile_90_6', 'meridionalCurrentSurface_Moving_min_11_mean', 'phosphateSurface_years_quantile_95', 'temperatureSurface_Moving_quantile_10_4', 'zonalCurrent100_300_Moving_max_7_mean', 'secchiDiskDepth_April_quantile_99', 'phosphate100_300_November_median', 'temperature100_300_October_max', 'phosphate100_300_Expanding_min_mean', 'salinitySurfacetrim_mean_10%', 'temperatureSurface_Moving_median_8_mean', 'secchiDiskDepth_February_median', 'meridionalCurrentSurface_March_quantile_1', 'secchiDiskDepth_February_quantile_5', 'wave_Height_Moving_max_6_mean', 'phosphate100_300_Moving_quantile_10_10', 'phosphate100_300_Moving_quantile_5_4', 'chlorophyll_Moving_quantile_10_2', 'wave_Height_Moving_median_9_mean', 'salinitySurface_Moving_quantile_95_11', 'wave_Height_Moving_quantile_95_3', 'zonalCurrentSurface_Moving_min_7_mean', 'salinity100_300_Moving_skew_9_mean', 'phosphate100_300_December_quantile_95', 'dissolvedOxygen100_300_Moving_std_8_mean', 'euphoticDepth_10wiener_mean', 'temperatureSurface_years_quantile_99', 'meridionalCurrent100_300_Moving_max_10_mean', 'temperature100_300_June_quantile_99', 'phosphate100_300_Moving_std_9_mean', 'temperature100_300_Hilbert_mean', 'meridionalCurrent100_300_Moving_std_9_mean', 'chlorophyll_Moving_max_2_mean', 'salinity100_300_Moving_min_9_mean', 'zonalCurrent100_300_April_quantile_5', 'temperature100_300_20wiener_mean', 'salinitySurface_February_quantile_95', 'temperatureSurface_Moving_quantile_5_9', 'nitrateSurface_years_quantile_10', 'salinity100_300_February_median', 'secchiDiskDepth_May_quantile_1', 'phosphate100_300_Moving_max_4_mean', 'temperatureSurface_decimate_mean', 'secchiDiskDepth_Moving_max_2_mean', 'nitrateSurface_Moving_quantile_5_4', 'meridionalCurrent100_300_September_quantile_95', 'euphoticDepth_January_quantile_99', 'phosphate100_300_November_quantile_99', 'nitrateSurface_Moving_min_12_mean', 'zonalCurrentSurface_Moving_quantile_10_2', 'nitrate100_300_Moving_min_6_mean', 'temperatureSurface_Expanding_quantile_95', 'dissolvedOxygenSurface_Moving_quantile_99_3', 'zonalCurrentSurface_January_quantile_90', 'dissolvedOxygenSurface_Moving_quantile_99_10', 'euphoticDepth_May_median', 'nitrate100_300_Moving_quantile_90_10', 'salinitySurface_decimate_mean', 'salinitySurface_mean_change_argrelmax', 'dissolvedOxygen100_300_June_mean', 'phosphate100_300_Moving_min_11_mean', 'euphoticDepth_June_min', 'phosphate100_300_Moving_min_3_mean', 'nitrateSurface_Moving_quantile_95_4', 'temperatureSurface_Moving_max_9_mean', 'chlorophyll_Moving_quantile_90_12', 'meridionalCurrent100_300_August_quantile_1', 'secchiDiskDepth_quantile_1', 'phosphate100_300_November_quantile_90', 'salinitySurface_Expanding_quantile_5', 'dissolvedOxygen100_300_Moving_quantile_90_6', 'secchiDiskDepth_Moving_std_5_mean', 'nitrateSurface_November_quantile_10', 'phosphate100_300_Moving_quantile_90_10', 'meridionalCurrentSurfacetrim_mean_20%', 'nitrate100_300_2spline_mean', 'nitrate100_300_Moving_quantile_1_8', 'temperatureSurface_September_max', 'temperature100_300_Expanding_quantile_95', 'euphoticDepth_Moving_min_3_mean', 'dissolvedOxygenSurface_March_median', 'temperature100_300_April_quantile_95', 'nitrate100_300_June_max', 'dissolvedOxygen100_300_Moving_average_7_mean', 'zonalCurrentSurface_51savgol_mean', 'phosphate100_300_Moving_quantile_10_12', 'meridionalCurrentSurface_Moving_quantile_10_8', 'dissolvedOxygenSurface_August_quantile_90', 'zonalCurrentSurfacetrim_mean_15%', 'chlorophyll_Moving_quantile_90_9', 'temperatureSurface_January_max', 'zonalCurrentSurface_Moving_max_10_mean', 'zonalCurrent100_300_Moving_std_2_mean', 'salinitySurface_50medfilt_mean', 'salinitySurface_Hilbert_mean', 'temperature100_300_September_quantile_95', 'temperature100_300_Moving_quantile_99_10', 'dissolvedOxygen100_300_September_min', 'salinitySurface_Moving_average_2_mean', 'zonalCurrentSurface_December_quantile_99', 'salinity100_300_Expanding_quantile_90', 'meridionalCurrentSurface_2gauss_spline_mean', 'salinitySurface_Moving_min_2_mean', 'salinity100_300_Moving_quantile_90_9', 'phosphateSurface_April_quantile_90', 'chlorophylliqr1', 'temperature100_300_Moving_quantile_5_5', 'salinitySurface_June_max', 'nitrateSurface_November_quantile_95', 'nitrateSurface_25medfilt_mean', 'zonalCurrent100_300_25savgol_mean', 'zonalCurrent100_300_December_median', 'wave_Height_May_quantile_99', 'secchiDiskDepth_March_quantile_90', 'nitrateSurface_Moving_quantile_1_9', 'meridionalCurrentSurfacetrim_mean_10%', 'phosphate100_300_January_max', 'nitrate100_300_November_quantile_1', 'salinitySurface_years_mean', 'wave_Height_Moving_quantile_1_6', 'wave_Height_5savgol_mean', 'temperatureSurface_Moving_quantile_1_5', 'zonalCurrentSurface_11savgol_mean', 'phosphateSurface_peak_mean', 'zonalCurrentSurface_symiirorder1_mean', 'chlorophyll_quantile_5', 'dissolvedOxygen100_300_July_quantile_99', 'nitrateSurface_Moving_median_8_mean', 'euphoticDepth_August_quantile_10', 'dissolvedOxygen100_300_5savgol_mean', 'temperature100_300_June_min', 'meridionalCurrent100_300_Moving_std_10_mean', 'secchiDiskDepth_Moving_quantile_90_3', 'temperatureSurfaceiqr1', 'euphoticDepth_3medfilt_mean', 'zonalCurrentSurface_years_quantile_95', 'salinitySurface_February_quantile_99', 'salinitySurface_May_median', 'meridionalCurrentSurface_November_median', 'secchiDiskDepth_April_quantile_90', 'zonalCurrentSurface_Moving_quantile_95_4', 'meridionalCurrent100_300_February_median', 'nitrate100_300_Expanding_quantile_5', 'dissolvedOxygenSurface_December_quantile_10', 'secchiDiskDepth_Moving_quantile_99_6', 'nitrate100_300_20wiener_mean', 'zonalCurrent100_300_January_quantile_10', 'chlorophyll_March_quantile_5', 'euphoticDepth_Moving_median_7_mean', 'chlorophyll_Moving_quantile_95_11', 'temperature100_300_August_max', 'secchiDiskDepth_3medfilt_mean', 'phosphateSurface_quantile_90', 'temperature100_300_September_max', 'zonalCurrentSurface_Moving_quantile_90_10', 'dissolvedOxygen100_300_50medfilt_mean', 'temperatureSurface_peak_max', 'zonalCurrentSurfaceiqr1', 'phosphate100_300_Moving_average_12_mean', 'nitrateSurface_Expanding_quantile_5', 'meridionalCurrentSurface_December_quantile_1', 'euphoticDepth_Moving_quantile_10_6', 'secchiDiskDepth_October_quantile_5', 'phosphateSurface_Moving_quantile_5_2', 'meridionalCurrent100_300_Moving_min_4_mean', 'temperatureSurface_Moving_quantile_5_6', 'dissolvedOxygen100_300_Expanding_quantile_90', 'meridionalCurrent100_300_February_quantile_95', 'chlorophyll_March_max', 'zonalCurrent100_300_January_quantile_95', 'temperatureSurface_Moving_std_4_mean', 'phosphateSurface_Moving_quantile_10_11', 'chlorophyll_May_std', 'salinitySurface_February_quantile_1', 'temperatureSurface_December_quantile_95', 'zonalCurrentSurface_mad', 'temperature100_300_years_quantile_99', 'chlorophyll_October_max', 'temperatureSurface_January_quantile_1', 'salinitySurface_September_max', 'temperatureSurface_September_quantile_5', 'chlorophyll_Moving_quantile_5_9', 'secchiDiskDepth_Moving_average_5_mean', 'meridionalCurrent100_300_December_quantile_99', 'nitrate100_300_Moving_quantile_5_8', 'nitrate100_300_Moving_std_9_mean', 'phosphate100_300_Moving_average_6_mean', 'salinitySurface_25savgol_mean', 'dissolvedOxygen100_300_Moving_average_8_mean', 'salinity100_300_Moving_quantile_95_12', 'meridionalCurrent100_300_Moving_quantile_90_7', 'meridionalCurrent100_300_argrelminmax_count', 'euphoticDepth_Moving_std_6_mean', 'salinity100_300_Moving_median_12_mean', 'euphoticDepth_February_quantile_99', 'salinitySurface_Moving_average_12_mean', 'zonalCurrent100_300_Moving_median_10_mean', 'zonalCurrentSurface_Moving_skew_7_mean', 'phosphateSurface_50medfilt_mean', 'secchiDiskDepth_Moving_average_4_mean', 'temperature100_300_Moving_quantile_95_3', 'nitrate100_300_April_quantile_99', 'meridionalCurrent100_300_Moving_quantile_99_5', 'temperatureSurface_10wiener_mean', 'nitrateSurface_max', 'nitrate100_300_Moving_quantile_99_9', 'meridionalCurrent100_300_December_quantile_10', 'dissolvedOxygen100_300_Moving_max_10_mean', 'meridionalCurrent100_300_symiirorder2_mean', 'dissolvedOxygen100_300_Moving_std_7_mean', 'phosphateSurface_December_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_1_2', 'phosphate100_300_Moving_median_12_mean', 'dissolvedOxygen100_300_November_quantile_99', 'secchiDiskDepth_Moving_quantile_10_9', 'phosphate100_300_Moving_quantile_10_11', 'phosphate100_300_Moving_quantile_1_9', 'nitrate100_300_December_quantile_10', 'meridionalCurrent100_300_July_quantile_10', 'chlorophyll_July_quantile_90', 'salinitySurface_Moving_average_3_mean', 'meridionalCurrent100_300_Moving_quantile_5_6', 'salinity100_300_November_quantile_90', 'dissolvedOxygenSurface_November_median', 'secchiDiskDepth_Moving_quantile_90_2', 'nitrate100_300_July_quantile_99', 'salinity100_300_September_median', 'zonalCurrentSurface_November_quantile_90', 'meridionalCurrentSurface_Moving_std_7_mean', 'dissolvedOxygenSurface_Hann_window_mean', 'meridionalCurrent100_300_Moving_quantile_95_4', 'temperature100_300_Moving_quantile_99_12', 'zonalCurrent100_300_Moving_quantile_5_2', 'salinitySurface_June_quantile_95', 'nitrate100_300_Moving_quantile_90_4', 'temperatureSurface_Moving_quantile_99_7', 'temperatureSurface_Moving_quantile_5_11', 'wave_Height_Moving_quantile_90_10', 'phosphateSurfaceiqr1', 'phosphate100_300_Moving_average_2_mean', 'salinitySurface_December_quantile_1', 'zonalCurrent100_300_years_quantile_5', 'meridionalCurrent100_300_Moving_average_12_mean', 'temperature100_300_February_quantile_1', 'phosphate100_300_November_min', 'secchiDiskDepth_February_max', 'temperatureSurface_Expanding_quantile_99', 'phosphateSurface_Moving_min_3_mean', 'nitrateSurface_decimate_mean', 'meridionalCurrent100_300_Moving_quantile_90_11', 'euphoticDepth_February_mean', 'salinitySurface_11savgol_mean', 'phosphateSurface_Moving_max_9_mean', 'salinitySurface_November_max', 'secchiDiskDepth_Moving_quantile_99_7', 'phosphate100_300_Moving_median_4_mean', 'dissolvedOxygen100_300_years_median', 'nitrateSurface_Moving_quantile_95_9', 'nitrateSurface_Moving_skew_9_mean', 'meridionalCurrentSurface_Moving_quantile_90_9', 'temperature100_300_Moving_min_3_mean', 'temperature100_300_Moving_average_11_mean', 'temperature100_300_February_max', 'dissolvedOxygen100_300_Moving_quantile_10_12', 'chlorophyll_Moving_min_10_mean', 'wave_Height_Moving_std_12_mean', 'euphoticDepth_symiirorder2_mean', 'nitrate100_300_4wiener_mean', 'phosphate100_300_Expanding_max_mean', 'zonalCurrent100_300_Moving_quantile_99_9', 'nitrateSurface_October_quantile_1', 'zonalCurrent100_300_Moving_quantile_1_11', 'salinity100_300_Moving_std_11_mean', 'phosphateSurface_June_median', 'temperatureSurface_Expanding_std_mean', 'wave_Height_4wiener_mean', 'euphoticDepth_February_quantile_5', 'dissolvedOxygenSurface_Moving_quantile_10_7', 'chlorophyll_Moving_quantile_10_8', 'dissolvedOxygen100_300_Moving_quantile_99_4', 'temperatureSurface_May_quantile_5', 'phosphate100_300_May_quantile_1', 'nitrate100_300_3medfilt_mean', 'dissolvedOxygen100_300_February_quantile_99', 'phosphate100_300_January_quantile_10', 'nitrateSurface_September_median', 'euphoticDepth_Moving_quantile_1_7', 'dissolvedOxygenSurface_December_quantile_95', 'euphoticDepth_Moving_quantile_99_11', 'meridionalCurrentSurface_September_quantile_5', 'temperatureSurface_Moving_quantile_1_11', 'secchiDiskDepth_August_mean', 'phosphateSurface_peak_max', 'meridionalCurrent100_300_Expanding_max_mean', 'chlorophyll_December_quantile_10', 'zonalCurrentSurface_May_quantile_5', 'chlorophyll_January_max', 'phosphateSurface_Moving_quantile_1_5', 'zonalCurrentSurface_Moving_quantile_90_4', 'temperatureSurface_Moving_quantile_95_10', 'salinitySurface_Expanding_quantile_90', 'temperature100_300_January_quantile_1', 'temperature100_300_February_quantile_5', 'temperature100_300_March_quantile_5', 'meridionalCurrent100_300_July_quantile_99', 'euphoticDepth_Moving_quantile_5_4', 'salinity100_300_January_quantile_5', 'temperature100_300_years_quantile_10', 'secchiDiskDepth_February_quantile_10', 'phosphate100_300_April_quantile_1', 'meridionalCurrentSurface_years_std', 'meridionalCurrent100_300_Moving_quantile_1_11', 'zonalCurrent100_300_Moving_max_5_mean', 'salinity100_300_Moving_quantile_1_4', 'meridionalCurrent100_300_decimate_mean', 'temperature100_300_March_quantile_95', 'phosphateSurface_June_quantile_95', 'dissolvedOxygenSurface_April_quantile_1', 'euphoticDepth_October_quantile_5', 'nitrateSurface_September_quantile_10', 'dissolvedOxygenSurface_Moving_quantile_99_12', 'zonalCurrent100_300_June_median', 'zonalCurrent100_300_September_quantile_95', 'wave_Height_Moving_min_5_mean', 'meridionalCurrentSurface_Moving_quantile_10_4', 'chlorophyll_January_quantile_10', 'chlorophyll_December_quantile_90', 'secchiDiskDepth_Moving_std_11_mean', 'zonalCurrent100_300_Moving_quantile_99_5', 'nitrate100_300_Moving_quantile_1_3', 'temperature100_300_Moving_skew_6_mean', 'salinity100_300_Expanding_max_mean', 'salinitySurface_Moving_quantile_10_4', 'zonalCurrent100_300_Moving_quantile_95_7', 'temperature100_300_April_median', 'meridionalCurrent100_300_May_quantile_10', 'chlorophyll_Moving_quantile_10_12', 'wave_Height_4spline_mean', 'salinity100_300_November_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_90_5', 'dissolvedOxygen100_300_Moving_quantile_10_2', 'phosphate100_300_Moving_median_7_mean', 'phosphate100_300_October_quantile_95', 'dissolvedOxygenSurface_years_quantile_5', 'zonalCurrent100_300_Moving_quantile_5_7', 'meridionalCurrentSurfacetrim_mean_25%', 'temperatureSurface_Moving_quantile_95_2', 'nitrate100_300_symiirorder1_mean', 'phosphateSurface_Moving_quantile_10_7', 'wave_Height_Moving_quantile_90_6', 'phosphateSurface_Moving_quantile_90_8', 'temperatureSurfacetrim_mean_25%', 'phosphateSurface_Moving_average_11_mean', 'nitrate100_300_Moving_quantile_5_12', 'nitrate100_300_October_quantile_5', 'wave_Height_Moving_std_2_mean', 'nitrate100_300_Expanding_quantile_10', 'temperature100_300_November_quantile_99', 'nitrate100_300trim_mean_20%', 'temperature100_300_quantile_10', 'salinitySurface_Moving_quantile_1_4', 'secchiDiskDepth_Moving_min_3_mean', 'wave_Height_Moving_quantile_95_7', 'chlorophylltrim_mean_15%', 'phosphateSurface_Moving_max_10_mean', 'phosphate100_300_Moving_quantile_5_8', 'euphoticDepth_November_quantile_5', 'euphoticDepth_Moving_quantile_90_7', 'phosphate100_300_Moving_quantile_95_3', 'zonalCurrentSurface_Moving_quantile_1_9', 'temperature100_300_Moving_quantile_5_7', 'salinity100_300trim_mean_15%', 'dissolvedOxygenSurface_5savgol_mean', 'zonalCurrent100_300_3gauss_spline_mean', 'dissolvedOxygenSurface_Moving_min_4_mean', 'dissolvedOxygen100_300_September_quantile_5', 'phosphate100_300_Moving_std_12_mean', 'temperature100_300_October_median', 'euphoticDepth_Moving_min_4_mean', 'nitrateSurface_July_quantile_90', 'chlorophyll_Expanding_quantile_95', 'euphoticDepth_years_quantile_90', 'dissolvedOxygen100_300_June_quantile_1', 'chlorophylliqr', 'salinity100_300_July_min', 'secchiDiskDepth_Moving_quantile_99_3', 'dissolvedOxygenSurface_November_quantile_5', 'phosphateSurface_August_quantile_99', 'euphoticDepth_Moving_min_6_mean', 'temperature100_300_Moving_quantile_10_4', 'chlorophyll_June_quantile_5', 'secchiDiskDepth_September_quantile_90', 'nitrateSurface_peak_mean', 'temperatureSurface_Moving_quantile_99_10', 'temperature100_300_March_max', 'meridionalCurrent100_300_years_quantile_99', 'dissolvedOxygen100_300_July_median', 'temperatureSurface_Moving_quantile_5_12', 'phosphateSurface_February_quantile_99', 'chlorophyll_Moving_quantile_95_5', 'euphoticDepth_September_quantile_90', 'wave_Height_peak_mean', 'meridionalCurrentSurface_February_quantile_95', 'nitrateSurface_Moving_median_12_mean', 'nitrate100_300_Moving_median_8_mean', 'euphoticDepth_Moving_quantile_90_6', 'temperatureSurface_4wiener_mean', 'nitrateSurface_February_quantile_90', 'meridionalCurrent100_300_October_quantile_1', 'zonalCurrent100_300_Moving_std_3_mean', 'zonalCurrentSurface_Moving_min_12_mean', 'wave_Height_Moving_quantile_99_4', 'meridionalCurrent100_300_June_quantile_99', 'zonalCurrent100_300_January_median', 'secchiDiskDepth_Moving_min_5_mean', 'zonalCurrent100_300_September_quantile_5', 'zonalCurrentSurface_December_quantile_95', 'temperature100_300_Moving_quantile_90_9', 'meridionalCurrent100_300_Moving_quantile_90_4', 'chlorophyll_Moving_quantile_90_8', 'temperature100_300_Moving_max_6_mean', 'nitrate100_300_March_mean', 'meridionalCurrentSurface_Moving_quantile_10_7', 'chlorophyll_August_quantile_90', 'chlorophyll_April_quantile_5', 'salinitySurface_January_median', 'temperatureSurface_April_quantile_90', 'secchiDiskDepth_Moving_quantile_90_12', 'salinity100_300_Moving_max_2_mean', 'zonalCurrent100_300_Moving_quantile_5_10', 'temperature100_300_February_min', 'meridionalCurrentSurface_October_median', 'meridionalCurrent100_300_Moving_skew_6_mean', 'temperature100_300iqr1', 'dissolvedOxygenSurface_Moving_average_8_mean', 'chlorophyll_5medfilt_mean', 'euphoticDepth_August_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_99_4', 'phosphateSurface_Moving_median_3_mean', 'phosphate100_300_Moving_quantile_90_12', 'chlorophyll_Expanding_quantile_1', 'secchiDiskDepth_Moving_quantile_1_12', 'wave_Height_December_quantile_1', 'temperature100_300_50medfilt_mean', 'salinity100_300_quantile_10', 'dissolvedOxygenSurface_Moving_max_7_mean', 'secchiDiskDepth_September_quantile_5', 'wave_Height_years_quantile_90', 'nitrateSurface_Moving_quantile_5_12', 'salinity100_300_Moving_std_10_mean', 'meridionalCurrent100_300_Moving_quantile_1_5', 'temperature100_300_January_quantile_5', 'temperature100_300_October_min', 'chlorophyll_September_quantile_95', 'salinity100_300_50medfilt_mean', 'dissolvedOxygen100_300_August_quantile_5', 'chlorophyll_mean_change_argrelmax', 'meridionalCurrentSurface_September_quantile_10', 'nitrateSurface_Moving_std_3_mean', 'secchiDiskDepth_April_min', 'dissolvedOxygen100_300_Moving_max_9_mean', 'nitrateSurface_August_median', 'zonalCurrent100_300_Moving_quantile_90_3', 'meridionalCurrent100_300_Expanding_min_mean', 'zonalCurrent100_300_symiirorder1_mean', 'temperature100_300_Moving_median_11_mean', 'temperatureSurface_January_quantile_5', 'zonalCurrent100_300_Moving_quantile_90_6', 'nitrate100_300_Moving_skew_11_mean', 'zonalCurrent100_300_April_quantile_99', 'temperature100_300_Moving_min_10_mean', 'zonalCurrent100_300trim_mean_25%', 'meridionalCurrent100_300_Moving_median_8_mean', 'phosphateSurface_Moving_median_9_mean', 'dissolvedOxygenSurface_Moving_std_8_mean', 'wave_Height_November_quantile_10', 'zonalCurrent100_300_Moving_median_3_mean', 'salinity100_300_max_to_min', 'nitrateSurface_argrelminmax_count', 'zonalCurrentSurface_Moving_quantile_99_8', 'dissolvedOxygenSurface_Moving_std_11_mean', 'temperatureSurface_Moving_max_7_mean', 'salinity100_300_Moving_quantile_90_11', 'wave_Height_Moving_quantile_99_11', 'chlorophyll_April_quantile_1', 'dissolvedOxygenSurface_25savgol_mean', 'salinitySurface_Moving_std_10_mean', 'temperatureSurface_Moving_quantile_99_9', 'temperatureSurface_July_quantile_10', 'wave_Height_years_max', 'meridionalCurrentSurface_December_quantile_5', 'nitrate100_300_August_mean', 'dissolvedOxygenSurface_Moving_quantile_95_10', 'phosphateSurface_Moving_average_10_mean', 'zonalCurrentSurface_Moving_std_6_mean', 'temperatureSurface_Moving_average_8_mean', 'salinity100_300_Moving_median_8_mean', 'nitrate100_300_Moving_quantile_95_12', 'temperatureSurface_December_quantile_99', 'phosphate100_300_Moving_quantile_10_3', 'meridionalCurrentSurface_Moving_quantile_5_12', 'salinity100_300_June_quantile_95', 'salinity100_300_Moving_quantile_99_6', 'salinitySurface_Moving_quantile_5_9', 'meridionalCurrentSurface_years_quantile_90', 'phosphateSurface_Moving_max_4_mean', 'temperatureSurface_December_min', 'phosphate100_300_Moving_quantile_99_5', 'temperatureSurface_50medfilt_mean', 'secchiDiskDepth_May_quantile_95', 'meridionalCurrent100_300_Moving_max_12_mean', 'dissolvedOxygenSurface_February_quantile_99', 'secchiDiskDepth_February_quantile_1', 'temperatureSurface_March_min', 'zonalCurrentSurface_January_quantile_1', 'meridionalCurrent100_300_Moving_std_8_mean', 'salinity100_300_July_median', 'secchiDiskDepth_August_median', 'nitrateSurface_25savgol_mean', 'temperatureSurface_Moving_median_12_mean', 'wave_Height_3medfilt_mean', 'meridionalCurrentSurface_Moving_quantile_5_11', 'euphoticDepth_Moving_max_3_mean', 'secchiDiskDepth_March_quantile_5', 'secchiDiskDepth_Moving_quantile_5_2', 'meridionalCurrent100_300_October_quantile_99', 'meridionalCurrentSurface_October_quantile_5', 'nitrateSurface_Moving_median_9_mean', 'zonalCurrentSurface_November_quantile_10', 'meridionalCurrentSurface_May_median', 'meridionalCurrent100_300_Moving_quantile_1_4', 'phosphate100_300_Moving_quantile_10_7', 'temperatureSurface_Moving_std_12_mean', 'meridionalCurrent100_300_years_std', 'wave_Height_March_std', 'nitrateSurface_October_max', 'zonalCurrentSurface_Moving_quantile_1_10', 'dissolvedOxygenSurface_September_quantile_5', 'temperatureSurface_Moving_quantile_95_4', 'euphoticDepth_Moving_min_2_mean', 'wave_Height_Moving_median_5_mean', 'euphoticDepth_Moving_quantile_10_8', 'chlorophyll_Moving_quantile_1_11', 'phosphateSurface_mean_change_argrelmin', 'salinity100_300_August_quantile_5', 'meridionalCurrent100_300_10wiener_mean', 'nitrate100_300_Moving_median_4_mean', 'zonalCurrentSurface_May_median', 'secchiDiskDepth_5medfilt_mean', 'secchiDiskDepth_years_mean', 'meridionalCurrent100_300_Moving_quantile_10_12', 'temperature100_300_December_min', 'secchiDiskDepth_March_quantile_1', 'meridionalCurrent100_300_Moving_average_9_mean', 'secchiDiskDepth_5savgol_mean', 'meridionalCurrentSurface_Moving_median_11_mean', 'temperature100_300_Moving_quantile_1_3', 'temperature100_300_argrelminmax_count', 'temperatureSurface_Moving_quantile_99_3', 'zonalCurrent100_300_Moving_average_12_mean', 'salinity100_300trim_mean_5%', 'phosphateSurface_April_quantile_95', 'zonalCurrentSurface_April_quantile_5', 'chlorophyll_Moving_quantile_90_6', 'zonalCurrent100_300_years_quantile_99', 'dissolvedOxygen100_300_20wiener_mean', 'meridionalCurrent100_300_Moving_skew_8_mean', 'temperatureSurface_September_quantile_95', 'phosphate100_300_Moving_quantile_99_4', 'salinity100_300_September_mean', 'phosphate100_300_Moving_quantile_5_6', 'secchiDiskDepth_Moving_skew_7_mean', 'salinity100_300_May_quantile_10', 'phosphate100_300_Moving_max_8_mean', 'nitrateSurface_Moving_average_6_mean', 'temperature100_300_Moving_std_12_mean', 'temperatureSurface_February_mean', 'nitrate100_300_Moving_skew_10_mean', 'phosphate100_300_Moving_skew_12_mean', 'chlorophyll_June_max', 'meridionalCurrent100_300_Moving_skew_7_mean', 'salinitySurface_10wiener_mean', 'nitrateSurface_November_quantile_90', 'chlorophyll_Moving_min_2_mean', 'salinitySurface_Moving_min_4_mean', 'salinitySurface_Moving_quantile_90_3', 'dissolvedOxygen100_300_mean_change_argrelmin', 'meridionalCurrent100_300_Expanding_average_mean', 'euphoticDepth_October_max', 'chlorophyll_Moving_quantile_5_11', 'wave_Height_years_min', 'salinitySurface_Moving_max_10_mean', 'meridionalCurrent100_300_Moving_quantile_99_12', 'euphoticDepth_December_quantile_10', 'temperatureSurface_Moving_quantile_1_3', 'dissolvedOxygen100_300_quantile_99', 'wave_Height_Moving_std_3_mean', 'dissolvedOxygenSurface_Moving_max_12_mean', 'dissolvedOxygenSurface_May_quantile_1', 'meridionalCurrent100_300_May_quantile_1', 'dissolvedOxygen100_300_March_quantile_95', 'phosphateSurface_September_quantile_5', 'phosphateSurface_Moving_quantile_5_7', 'zonalCurrentSurface_Moving_skew_8_mean', 'temperature100_300_August_quantile_5', 'euphoticDepth_Moving_min_8_mean', 'salinitySurface_Moving_min_10_mean', 'phosphate100_300_Moving_quantile_1_11', 'chlorophyll_Moving_quantile_10_11', 'phosphateSurface_November_max', 'temperature100_300_symiirorder2_mean', 'meridionalCurrent100_300_April_quantile_99', 'euphoticDepth_Moving_quantile_95_2', 'nitrate100_300_Moving_quantile_99_5', 'phosphate100_300_March_quantile_1', 'dissolvedOxygen100_300_August_quantile_99', 'temperature100_300_April_quantile_1', 'dissolvedOxygenSurface_September_quantile_10', 'euphoticDepth_September_quantile_99', 'chlorophyll_peak_mean', 'chlorophyll_Moving_median_5_mean', 'phosphate100_300_May_min', 'chlorophyll_February_mean', 'dissolvedOxygenSurface_August_quantile_95', 'phosphate100_300_Expanding_quantile_90', 'dissolvedOxygen100_300_February_quantile_95', 'zonalCurrentSurface_Moving_std_11_mean', 'phosphate100_300_July_quantile_1', 'dissolvedOxygen100_300_Expanding_std_mean', 'nitrateSurface_Hilbert_mean', 'meridionalCurrentSurface_Moving_average_5_mean', 'dissolvedOxygen100_300_Moving_quantile_5_5', 'salinity100_300_Moving_quantile_1_10', 'dissolvedOxygen100_300_March_quantile_10', 'zonalCurrent100_300_Moving_median_7_mean', 'secchiDiskDepth_Moving_average_2_mean', 'chlorophyll_Moving_quantile_10_10', 'meridionalCurrentSurface_Moving_skew_9_mean', 'meridionalCurrent100_300_January_quantile_1', 'meridionalCurrentSurface_Expanding_median_mean', 'chlorophyll_Moving_quantile_99_4', 'temperatureSurface_Moving_quantile_95_8', 'chlorophyll_Moving_average_11_mean', 'dissolvedOxygen100_300_Moving_quantile_99_9', 'wave_Height_25medfilt_mean', 'zonalCurrent100_300_Moving_quantile_1_4', 'euphoticDepth_April_median', 'salinity100_300_January_quantile_99', 'nitrateSurface_March_quantile_95', 'chlorophyll_years_quantile_95', 'salinitySurface_Moving_quantile_10_10', 'secchiDiskDepth_Moving_max_5_mean', 'wave_Height_April_quantile_1', 'phosphate100_300_May_quantile_10', 'dissolvedOxygenSurface_March_quantile_95', 'dissolvedOxygen100_300_Moving_quantile_95_8', 'salinity100_300_May_median', 'salinitySurface_Moving_quantile_90_5', 'secchiDiskDepth_argrelminmax_count', 'wave_Height_Moving_quantile_95_6', 'dissolvedOxygenSurface_Moving_skew_11_mean', 'meridionalCurrentSurface_Moving_quantile_5_10', 'salinitySurface_argrelmax_count', 'salinitySurface_February_median', 'nitrate100_300_February_quantile_10', 'nitrate100_300_Moving_quantile_10_9', 'zonalCurrentSurface_Moving_quantile_1_5', 'chlorophyll_Moving_quantile_90_7', 'nitrate100_300_August_min', 'secchiDiskDepth_Expanding_quantile_5', 'salinitySurface_Moving_quantile_1_2', 'salinitySurfaceiqr1', 'nitrateSurface_Moving_quantile_95_6', 'wave_Height_2spline_mean', 'dissolvedOxygenSurface_May_quantile_90', 'secchiDiskDepth_May_median', 'dissolvedOxygen100_300_Moving_min_2_mean', 'nitrateSurface_July_quantile_10', 'meridionalCurrentSurface_50medfilt_mean', 'euphoticDepth_Expanding_std_mean', 'chlorophyll_Moving_max_7_mean', 'nitrate100_300_Moving_average_9_mean', 'nitrate100_300_Moving_quantile_90_7', 'dissolvedOxygen100_300_Moving_average_6_mean', 'secchiDiskDepth_25medfilt_mean', 'meridionalCurrentSurface_February_quantile_5', 'secchiDiskDepth_June_quantile_99', 'dissolvedOxygenSurface_Moving_average_7_mean', 'temperatureSurface_July_median', 'euphoticDepthiqr1', 'nitrateSurface_January_quantile_5', 'nitrate100_300_Moving_quantile_1_10', 'chlorophyll_January_quantile_95', 'meridionalCurrentSurface_median', 'nitrate100_300_August_quantile_95', 'secchiDiskDepth_2gauss_spline_mean', 'wave_Height_Moving_quantile_99_3', 'salinity100_300_August_min', 'wave_Height_Moving_quantile_99_9', 'nitrate100_300_Moving_quantile_99_7', 'phosphateSurface_Moving_min_8_mean', 'chlorophyll_Moving_median_10_mean', 'dissolvedOxygen100_300_Moving_quantile_99_10', 'temperature100_300_Moving_max_2_mean', 'temperature100_300_March_median', 'meridionalCurrentSurface_June_median', 'meridionalCurrentSurface_Moving_quantile_1_12', 'chlorophyll_quantile_90', 'dissolvedOxygen100_300_October_quantile_1', 'phosphateSurface_December_quantile_1', 'phosphateSurface_Moving_quantile_5_8', 'temperature100_300_July_quantile_99', 'zonalCurrent100_300_Moving_quantile_95_11', 'temperature100_300_January_max', 'dissolvedOxygen100_300_Moving_median_6_mean', 'wave_Height_Moving_max_8_mean', 'chlorophyll_quantile_95', 'euphoticDepth_Moving_quantile_5_11', 'temperatureSurface_December_mean', 'euphoticDepth_Moving_quantile_90_2', 'secchiDiskDepth_Moving_quantile_95_11', 'salinitySurface_symiirorder1_mean', 'dissolvedOxygen100_300_September_quantile_99', 'temperatureSurface_Moving_std_9_mean', 'euphoticDepth_neighbor_min', 'chlorophylltrim_mean_5%', 'temperatureSurface_Moving_min_4_mean', 'euphoticDepth_Moving_quantile_99_12', 'nitrateSurface_Moving_average_3_mean', 'phosphate100_300_Moving_quantile_99_3', 'dissolvedOxygenSurface_Moving_quantile_90_12', 'meridionalCurrent100_300_Moving_quantile_1_10', 'salinitySurface_quantile_5', 'dissolvedOxygenSurface_January_quantile_10', 'phosphate100_300_Moving_median_5_mean', 'euphoticDepth_Moving_std_10_mean', 'dissolvedOxygen100_300_50wiener_mean', 'salinity100_300_November_min', 'nitrate100_300_argrelmax_count', 'chlorophyll_Moving_std_9_mean', 'temperatureSurface_neighbor_max', 'wave_Height_3gauss_spline_mean', 'salinity100_300_Moving_quantile_95_9', 'dissolvedOxygenSurface_Moving_quantile_1_9', 'phosphateSurface_quantile_1', 'wave_Height_August_quantile_99', 'salinity100_300_5savgol_mean', 'temperature100_300_Moving_quantile_90_2', 'dissolvedOxygen100_300_November_quantile_95', 'zonalCurrent100_300_Moving_quantile_1_8', 'nitrateSurface_Moving_quantile_99_6', 'zonalCurrent100_300_Moving_quantile_90_5', 'dissolvedOxygenSurface_Moving_max_9_mean', 'temperatureSurface_Moving_quantile_95_7', 'salinitySurface_Moving_quantile_99_4', 'zonalCurrent100_300_Moving_quantile_5_12', 'nitrateSurface_February_max', 'temperatureSurface_May_quantile_99', 'zonalCurrentSurface_quantile_5', 'temperatureSurface_Moving_average_2_mean', 'dissolvedOxygen100_300_3medfilt_mean', 'chlorophyll_Moving_median_12_mean', 'phosphateSurface_Moving_std_6_mean', 'wave_Height_Moving_std_9_mean', 'salinity100_300_Moving_quantile_1_2', 'phosphate100_300_Moving_median_9_mean', 'meridionalCurrentSurface_years_median', 'meridionalCurrentSurface_Moving_quantile_5_3', 'meridionalCurrent100_300_3medfilt_mean', 'wave_Height_July_quantile_95', 'secchiDiskDepth_50wiener_mean', 'temperatureSurface_years_quantile_90', 'temperature100_300_January_quantile_95', 'nitrate100_300_Moving_median_12_mean', 'phosphateSurface_Moving_quantile_10_10', 'euphoticDepth_November_quantile_99', 'nitrateSurface_Moving_quantile_1_4', 'nitrate100_300_quantile_90', 'dissolvedOxygen100_300_Moving_quantile_99_3', 'wave_Height_August_quantile_1', 'zonalCurrent100_300_4spline_mean', 'meridionalCurrent100_300_mean_change_peak', 'meridionalCurrent100_300_June_quantile_5', 'nitrateSurface_May_std', 'meridionalCurrentSurface_Moving_quantile_5_5', 'meridionalCurrentSurface_Moving_quantile_95_3', 'secchiDiskDepth_Moving_quantile_95_4', 'meridionalCurrentSurface_Moving_quantile_5_6', 'meridionalCurrent100_300_Moving_quantile_95_7', 'secchiDiskDepth_Moving_median_9_mean', 'euphoticDepth_March_mean', 'dissolvedOxygen100_300_max_to_min_diff', 'salinitySurface_Moving_skew_8_mean', 'zonalCurrentSurface_argrelmax_count', 'nitrate100_300_Moving_quantile_5_5', 'dissolvedOxygenSurface_August_quantile_99', 'nitrateSurface_Expanding_quantile_99', 'salinity100_300trim_mean_25%', 'salinitySurface_Expanding_average_mean', 'wave_Height_Moving_min_6_mean', 'phosphateSurface_Moving_quantile_90_5', 'chlorophyll_max', 'secchiDiskDepth_December_min', 'euphoticDepth_June_mean', 'phosphate100_300_Moving_quantile_1_6', 'dissolvedOxygenSurface_April_quantile_10', 'nitrateSurface_quantile_10', 'temperature100_300_Moving_quantile_95_8', 'dissolvedOxygenSurface_March_max', 'nitrate100_300_10wiener_mean', 'dissolvedOxygenSurface_December_quantile_99', 'zonalCurrent100_300_April_quantile_10', 'dissolvedOxygen100_300_Moving_std_11_mean', 'secchiDiskDepth_Moving_quantile_1_6', 'dissolvedOxygenSurface_Moving_quantile_1_3', 'temperature100_300_Moving_skew_12_mean', 'chlorophyll_mad', 'phosphateSurface_Moving_max_7_mean', 'nitrateSurface_kurt', 'meridionalCurrent100_300_Moving_max_5_mean', 'euphoticDepth_Moving_quantile_95_4', 'phosphateSurface_51savgol_mean', 'salinitySurface_Moving_quantile_99_2', 'salinitySurface_April_quantile_90', 'chlorophyll_Moving_quantile_99_8', 'secchiDiskDepth_years_quantile_10', 'meridionalCurrent100_300_Moving_max_3_mean', 'wave_Height_Moving_quantile_1_5', 'salinitySurface_years_quantile_99', 'salinity100_300_August_quantile_90', 'salinity100_300_March_quantile_99', 'nitrateSurface_June_std', 'phosphate100_300_October_quantile_10', 'phosphate100_300_December_quantile_5', 'chlorophyll_January_quantile_5', 'euphoticDepth_August_mean', 'temperatureSurface_March_quantile_5', 'nitrate100_300_years_quantile_90', 'euphoticDepth_decimate_mean', 'salinitySurface_Moving_quantile_10_5', 'meridionalCurrentSurface_max_to_min_diff', 'chlorophyll_Moving_quantile_99_12', 'meridionalCurrentSurface_May_quantile_10', 'salinity100_300_September_quantile_1', 'phosphateSurface_Moving_quantile_1_8', 'dissolvedOxygen100_300_September_median', 'temperature100_300_Moving_skew_9_mean', 'salinity100_300_Moving_quantile_95_6', 'phosphateSurface_Moving_max_6_mean', 'temperature100_300_Moving_median_4_mean', 'chlorophyll_Moving_quantile_95_8', 'temperatureSurface_years_quantile_5', 'phosphate100_300_Moving_max_12_mean', 'secchiDiskDepthtrim_mean_20%', 'temperature100_300_years_mean', 'nitrate100_300_2gauss_spline_mean', 'dissolvedOxygenSurface_January_quantile_99', 'temperature100_300_Moving_quantile_90_11', 'temperature100_300_Moving_min_9_mean', 'temperature100_300_Moving_max_8_mean', 'wave_Height_max_to_min_diff', 'salinitySurface_Moving_quantile_99_5', 'temperature100_300_Moving_median_8_mean', 'phosphateSurface_March_quantile_5', 'zonalCurrentSurface_Moving_max_7_mean', 'phosphateSurface_February_quantile_90', 'nitrateSurface_January_quantile_10', 'phosphate100_300_3medfilt_mean', 'meridionalCurrentSurface_peak_max', 'euphoticDepth_January_quantile_1', 'temperatureSurface_20wiener_mean', 'nitrateSurfacetrim_mean_10%', 'euphoticDepth_Moving_quantile_90_4', 'salinity100_300_Moving_min_8_mean', 'salinitySurface_January_quantile_90', 'temperatureSurface_quantile_99', 'nitrateSurface_Moving_quantile_10_12', 'nitrate100_300_Moving_min_11_mean', 'phosphateSurface_Moving_average_5_mean', 'meridionalCurrent100_300_11savgol_mean', 'phosphate100_300_July_median', 'meridionalCurrent100_300iqr', 'phosphate100_300_August_min', 'salinity100_300_mean_change_argrelmax', 'secchiDiskDepth_July_quantile_95', 'temperature100_300_Moving_quantile_90_12', 'nitrateSurface_September_quantile_90', 'chlorophyll_Moving_quantile_1_2', 'zonalCurrentSurface_Moving_average_3_mean', 'secchiDiskDepth_Expanding_quantile_10', 'temperatureSurface_quantile_95', 'euphoticDepth_20wiener_mean', 'meridionalCurrent100_300_Moving_quantile_1_3', 'meridionalCurrentSurface_Moving_std_11_mean', 'chlorophyll_quantile_99', 'dissolvedOxygen100_300_Moving_quantile_1_4', 'meridionalCurrent100_300_3gauss_spline_mean', 'meridionalCurrent100_300_50medfilt_mean', 'nitrateSurface_May_quantile_1', 'meridionalCurrent100_300_April_quantile_10', 'meridionalCurrentSurface_Moving_max_7_mean', 'zonalCurrentSurface_Moving_quantile_1_11', 'temperatureSurface_Moving_quantile_90_9', 'wave_Height_Moving_quantile_99_7', 'phosphate100_300_January_mean', 'salinitySurface_Moving_quantile_5_6', 'zonalCurrent100_300_Moving_max_10_mean', 'nitrate100_300_January_min', 'secchiDiskDepthtrim_mean_25%', 'zonalCurrent100_300_Expanding_std_mean', 'wave_Height_September_quantile_10', 'salinitySurface_July_quantile_95', 'meridionalCurrentSurface_February_quantile_99', 'temperatureSurface_peak_mean', 'zonalCurrentSurface_Moving_max_8_mean', 'dissolvedOxygen100_300_Moving_skew_12_mean', 'dissolvedOxygenSurface_Moving_quantile_90_2', 'meridionalCurrent100_300_Moving_quantile_5_4', 'salinitySurface_December_quantile_10', 'temperatureSurface_neighbor_mean', 'salinitySurface_March_min', 'meridionalCurrentSurface_Expanding_quantile_95', 'secchiDiskDepth_Moving_quantile_5_6', 'phosphate100_300_November_quantile_95', 'euphoticDepth_10medfilt_mean', 'salinity100_300_peak_mean', 'phosphateSurface_Moving_quantile_90_6', 'meridionalCurrentSurface_11savgol_mean', 'wave_Height_quantile_95', 'euphoticDepth_May_mean', 'meridionalCurrentSurface_Expanding_max_mean', 'salinitySurface_Moving_average_11_mean', 'zonalCurrentSurface_Moving_quantile_90_6', 'secchiDiskDepth_Moving_median_7_mean', 'phosphate100_300_February_quantile_90', 'chlorophyll_September_mean', 'salinity100_300_Moving_std_4_mean', 'wave_Height_quantile_5', 'chlorophyll_Moving_max_12_mean', 'zonalCurrentSurface_Moving_average_11_mean', 'phosphate100_300_July_min', 'phosphate100_300_quantile_95', 'meridionalCurrent100_300_Moving_quantile_10_10', 'nitrate100_300_Moving_min_10_mean', 'salinity100_300_October_quantile_10', 'secchiDiskDepth_July_quantile_99', 'nitrate100_300_years_mean', 'secchiDiskDepth_November_quantile_95', 'salinity100_300_March_quantile_10', 'dissolvedOxygenSurface_years_quantile_90', 'salinity100_300_October_quantile_99', 'nitrate100_300_Moving_quantile_90_9', 'wave_Height_Moving_min_4_mean', 'phosphateSurface_April_median', 'secchiDiskDepth_Moving_std_10_mean', 'meridionalCurrent100_300_Moving_quantile_90_6', 'salinity100_300_5medfilt_mean', 'chlorophyll_November_max', 'dissolvedOxygen100_300_August_max', 'phosphateSurface_Expanding_std_mean', 'zonalCurrentSurface_Moving_median_5_mean', 'nitrateSurface_September_quantile_99', 'temperatureSurface_June_quantile_95', 'nitrateSurface_4wiener_mean', 'salinitySurface_Moving_median_7_mean', 'zonalCurrentSurface_Moving_median_9_mean', 'salinity100_300_February_quantile_10', 'phosphate100_300_Moving_average_3_mean', 'nitrateSurface_December_quantile_5', 'wave_Height_Moving_quantile_90_9', 'phosphate100_300_July_quantile_95', 'meridionalCurrent100_300_April_quantile_95', 'wave_Height_April_quantile_95', 'phosphateSurface_April_quantile_10', 'temperatureSurface_Moving_quantile_10_9', 'dissolvedOxygen100_300_years_min', 'salinitySurface_Moving_min_7_mean', 'euphoticDepthtrim_mean_5%', 'dissolvedOxygenSurface_July_quantile_5', 'secchiDiskDepth_February_mean', 'dissolvedOxygenSurfacetrim_mean_20%', 'secchiDiskDepth_October_mean', 'phosphateSurface_November_quantile_95', 'phosphateSurface_Moving_quantile_95_4', 'phosphateSurface_Hann_window_mean', 'meridionalCurrent100_300_3spline_mean', 'zonalCurrent100_300_March_median', 'temperature100_300_Moving_min_8_mean', 'zonalCurrent100_300_Moving_quantile_90_2', 'secchiDiskDepth_Moving_median_8_mean', 'nitrate100_300_Moving_quantile_95_8', 'zonalCurrent100_300_Moving_skew_7_mean', 'dissolvedOxygenSurface_Moving_quantile_1_8', 'nitrateSurface_Moving_quantile_5_5', 'wave_Height_October_quantile_10', 'nitrate100_300_Moving_min_4_mean', 'salinitySurface_December_quantile_90', 'meridionalCurrentSurface_Moving_quantile_10_5', 'zonalCurrentSurface_50medfilt_mean', 'salinitySurface_25medfilt_mean', 'salinitySurface_May_quantile_1', 'temperature100_300_Moving_quantile_5_3', 'euphoticDepth_April_quantile_1', 'temperatureSurface_March_mean', 'dissolvedOxygen100_300_Moving_max_8_mean', 'chlorophyll_February_median', 'meridionalCurrentSurface_Moving_quantile_90_2', 'meridionalCurrentSurface_January_quantile_95', 'secchiDiskDepth_Moving_quantile_90_9', 'meridionalCurrentSurface_decimate_mean', 'salinitySurface_March_quantile_1', 'euphoticDepth_Moving_quantile_5_2', 'euphoticDepth_July_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_99_6', 'euphoticDepth_January_quantile_90', 'meridionalCurrent100_300_Moving_quantile_90_10', 'secchiDiskDepth_argrelmax_count', 'chlorophyll_Expanding_median_mean', 'wave_Height_Moving_min_9_mean', 'secchiDiskDepth_Moving_max_3_mean', 'nitrate100_300_Moving_quantile_5_4', 'salinity100_300_Moving_quantile_5_7', 'meridionalCurrentSurface_argrelminmax_count', 'secchiDiskDepth_August_quantile_95', 'nitrateSurface_Moving_average_7_mean', 'dissolvedOxygenSurface_April_min', 'phosphate100_300_April_quantile_99', 'meridionalCurrent100_300_Expanding_quantile_5', 'secchiDiskDepth_Moving_skew_8_mean', 'temperatureSurface_February_quantile_90', 'wave_Height_Moving_quantile_1_7', 'wave_Height_Expanding_quantile_10', 'euphoticDepth_December_quantile_5', 'temperatureSurface_June_quantile_1', 'temperature100_300_Moving_max_3_mean', 'salinity100_300_Moving_quantile_90_8', 'secchiDiskDepth_years_quantile_99', 'dissolvedOxygenSurface_Moving_average_9_mean', 'temperature100_300_Moving_max_7_mean', 'chlorophyll_Moving_max_10_mean', 'secchiDiskDepth_September_quantile_10', 'temperatureSurface_years_max', 'nitrateSurface_February_quantile_10', 'secchiDiskDepth_Moving_max_8_mean', 'phosphate100_300_Moving_quantile_95_9', 'salinity100_300_July_quantile_10', 'temperature100_300_Moving_median_10_mean', 'nitrateSurface_Moving_quantile_1_12', 'euphoticDepth_Moving_average_9_mean', 'wave_Height_Moving_quantile_1_3', 'zonalCurrent100_300_February_quantile_10', 'temperatureSurface_Moving_quantile_1_10', 'dissolvedOxygenSurface_August_quantile_1', 'wave_Height_years_quantile_10', 'phosphate100_300_June_quantile_95', 'meridionalCurrent100_300_Moving_max_4_mean', 'dissolvedOxygen100_300_June_quantile_5', 'dissolvedOxygenSurface_June_quantile_1', 'meridionalCurrent100_300_years_quantile_1', 'zonalCurrentSurface_quantile_1', 'wave_Height_mean_change_argrelmin', 'dissolvedOxygenSurface_Moving_min_8_mean', 'meridionalCurrent100_300_Expanding_quantile_99', 'phosphateSurface_Moving_quantile_5_5', 'nitrateSurface_June_quantile_5', 'secchiDiskDepth_September_mean', 'temperatureSurface_March_max', 'zonalCurrentSurface_July_quantile_95', 'zonalCurrent100_300_November_quantile_10', 'wave_Height_January_quantile_10', 'nitrateSurface_Moving_std_6_mean', 'temperatureSurface_Moving_std_5_mean', 'euphoticDepth_2gauss_spline_mean', 'temperatureSurface_years_min', 'nitrate100_300_July_quantile_5', 'phosphate100_300_February_quantile_5', 'wave_Heighttrim_mean_5%', 'secchiDiskDepth_Expanding_max_mean', 'meridionalCurrentSurface_3gauss_spline_mean', 'zonalCurrentSurface_20wiener_mean', 'phosphate100_300_peak_max', 'salinity100_300_Moving_std_8_mean', 'euphoticDepth_2spline_mean', 'salinitySurface_August_quantile_10', 'nitrate100_300_Moving_median_11_mean', 'wave_Height_Moving_median_7_mean', 'temperature100_300_quantile_95', 'dissolvedOxygen100_300_December_median', 'euphoticDepth_Moving_quantile_99_7', 'euphoticDepth_Moving_quantile_1_8', 'meridionalCurrent100_300_mean_change_peak_rate', 'salinity100_300_Moving_average_2_mean', 'salinity100_300_March_quantile_95', 'nitrateSurface_July_quantile_95', 'euphoticDepth_Moving_max_10_mean', 'salinity100_300_25medfilt_mean', 'chlorophyll_decimate_mean', 'chlorophyll_years_min', 'dissolvedOxygen100_300_Moving_quantile_90_7', 'wave_Heightiqr1', 'euphoticDepth_August_median', 'phosphate100_300_September_quantile_99', 'wave_Height_Moving_quantile_99_12', 'euphoticDepth_February_quantile_95', 'dissolvedOxygenSurface_Moving_quantile_5_7', 'meridionalCurrentSurface_Moving_quantile_5_9', 'zonalCurrent100_300_Moving_quantile_99_11', 'secchiDiskDepth_years_min', 'meridionalCurrent100_300_years_quantile_90', 'zonalCurrent100_300_Moving_skew_11_mean', 'zonalCurrentSurface_Moving_median_12_mean', 'salinity100_300_Expanding_quantile_95', 'phosphate100_300_January_min', 'salinity100_300_Moving_median_6_mean', 'phosphate100_300_Moving_quantile_1_7', 'dissolvedOxygen100_300_Moving_quantile_5_3', 'secchiDiskDepth_Expanding_quantile_99', 'euphoticDepth_Moving_quantile_5_5', 'chlorophyll_Moving_average_5_mean', 'salinity100_300_Moving_average_12_mean', 'salinitySurface_Moving_std_9_mean', 'meridionalCurrent100_300_Expanding_median_mean', 'dissolvedOxygenSurface_June_quantile_95', 'dissolvedOxygen100_300_Moving_quantile_90_8', 'dissolvedOxygen100_300_June_quantile_10', 'zonalCurrent100_300_Moving_average_6_mean', 'phosphate100_300_May_quantile_5', 'temperatureSurface_April_quantile_99', 'phosphateSurface_March_max', 'wave_Height_Moving_quantile_10_2', 'nitrate100_300_Moving_std_4_mean', 'nitrateSurface_Moving_average_5_mean', 'nitrate100_300_November_min', 'temperature100_300_Moving_quantile_5_6', 'phosphate100_300_Moving_quantile_90_6', 'wave_Height_May_quantile_5', 'chlorophyll_years_quantile_10', 'dissolvedOxygen100_300_Moving_quantile_1_7', 'nitrate100_300_October_mean', 'meridionalCurrent100_300_Moving_std_12_mean', 'meridionalCurrentSurface_January_quantile_1', 'nitrate100_300_3spline_mean', 'nitrateSurface_December_quantile_99', 'euphoticDepth_peak_max', 'chlorophyll_September_quantile_99', 'temperatureSurface_Moving_quantile_1_6', 'secchiDiskDepth_September_quantile_1', 'wave_Height_Moving_quantile_95_2', 'salinitySurface_Moving_min_12_mean', 'euphoticDepth_March_quantile_1', 'zonalCurrent100_300_Moving_max_4_mean', 'chlorophyll_October_quantile_90', 'chlorophyll_Hilbert_mean', 'nitrateSurface_Moving_quantile_1_11', 'dissolvedOxygenSurface_March_quantile_99', 'zonalCurrent100_300_Moving_min_2_mean', 'nitrateSurface_Moving_quantile_5_7', 'meridionalCurrentSurface_Moving_quantile_1_4', 'meridionalCurrent100_300_Moving_quantile_99_7', 'wave_Height_Moving_std_11_mean', 'phosphate100_300_50medfilt_mean', 'chlorophyll_August_median', 'wave_Height_March_quantile_99', 'temperature100_300_Moving_quantile_10_5', 'chlorophyll_Moving_skew_8_mean', 'meridionalCurrentSurface_November_quantile_99', 'temperatureSurface_June_quantile_99', 'zonalCurrentSurface_Moving_median_4_mean', 'chlorophyll_Moving_quantile_10_3', 'phosphateSurface_symiirorder2_mean', 'euphoticDepth_Moving_std_4_mean', 'salinity100_300_October_mean', 'salinity100_300_Hilbert_mean', 'meridionalCurrentSurface_January_quantile_5', 'dissolvedOxygenSurface_Moving_average_3_mean', 'meridionalCurrent100_300_November_quantile_5', 'salinitySurface_Moving_quantile_90_12', 'salinitySurface_Moving_quantile_90_11', 'phosphateSurface_January_median', 'meridionalCurrent100_300_September_quantile_5', 'phosphateSurfacetrim_mean_20%', 'dissolvedOxygenSurface_50medfilt_mean', 'wave_Height_argrelmax_count', 'wave_Height_March_quantile_95', 'dissolvedOxygenSurface_Moving_average_10_mean', 'chlorophyll_May_quantile_95', 'salinity100_300_Moving_min_6_mean', 'zonalCurrent100_300_July_quantile_10', 'zonalCurrentSurface_Moving_std_4_mean', 'salinity100_300_May_quantile_1', 'temperature100_300_February_quantile_99', 'temperatureSurface_Moving_quantile_99_2', 'nitrateSurface_Moving_quantile_99_5', 'secchiDiskDepth_January_median', 'temperature100_300_years_quantile_90', 'salinity100_300_June_quantile_1', 'salinitySurface_Moving_quantile_1_6', 'wave_Height_June_quantile_10', 'salinitySurface_November_quantile_10', 'wave_Height_10medfilt_mean', 'temperatureSurfaceiqr', 'zonalCurrentSurface_October_quantile_1', 'nitrate100_300_5savgol_mean', 'secchiDiskDepth_Moving_average_12_mean', 'dissolvedOxygen100_300_Moving_quantile_5_2', 'chlorophyll_Moving_max_6_mean', 'temperatureSurface_51savgol_mean', 'phosphate100_300_September_min', 'dissolvedOxygenSurface_peak_width_max_to_min', 'dissolvedOxygen100_300_Moving_min_12_mean', 'secchiDiskDepth_Moving_quantile_95_8', 'dissolvedOxygenSurface_Hilbert_mean', 'temperature100_300_Moving_median_12_mean', 'dissolvedOxygenSurface_Moving_quantile_99_8', 'dissolvedOxygenSurface_Moving_quantile_95_2', 'dissolvedOxygenSurface_years_quantile_95', 'phosphate100_300_Moving_skew_10_mean', 'salinity100_300_Moving_median_5_mean', 'phosphate100_300trim_mean_5%', 'dissolvedOxygen100_300_Moving_quantile_90_5', 'phosphateSurface_Moving_max_3_mean', 'phosphate100_300_Expanding_quantile_99', 'euphoticDepth_Moving_quantile_10_4', 'phosphate100_300_Moving_quantile_95_4', 'euphoticDepth_Moving_median_10_mean', 'temperature100_300_October_quantile_90', 'dissolvedOxygen100_300_Moving_quantile_10_7', 'nitrate100_300_mad', 'euphoticDepth_Moving_median_12_mean', 'secchiDiskDepth_May_quantile_90', 'meridionalCurrent100_300_Moving_quantile_1_7', 'zonalCurrent100_300_Moving_median_5_mean', 'temperature100_300_Moving_average_4_mean', 'chlorophyll_Moving_std_6_mean', 'temperature100_300_Moving_average_12_mean', 'meridionalCurrent100_300_Expanding_quantile_10', 'chlorophyll_Moving_max_3_mean', 'euphoticDepth_April_quantile_95', 'nitrateSurface_May_quantile_90', 'zonalCurrentSurface_Moving_skew_6_mean', 'dissolvedOxygenSurface_Moving_quantile_1_5', 'wave_Height_Moving_std_8_mean', 'chlorophyll_October_median', 'nitrateSurface_July_max', 'meridionalCurrentSurface_Moving_quantile_95_12', 'euphoticDepth_September_quantile_1', 'dissolvedOxygenSurface_February_quantile_10', 'nitrateSurface_argrelmax_count', 'chlorophyll_July_mean', 'phosphateSurface_October_quantile_90', 'temperature100_300_Moving_max_5_mean', 'meridionalCurrentSurface_years_quantile_95', 'temperatureSurface_Moving_quantile_90_2', 'phosphateSurface_Moving_quantile_5_10', 'chlorophyll_February_max', 'dissolvedOxygen100_300_November_quantile_90', 'temperatureSurface_Moving_min_6_mean', 'phosphate100_300_June_quantile_1', 'nitrateSurface_Moving_quantile_90_12', 'phosphate100_300_November_quantile_10', 'salinitySurface_Moving_quantile_5_12', 'salinitySurface_Moving_quantile_1_10', 'phosphate100_300_Moving_quantile_90_11', 'salinity100_300_January_min', 'zonalCurrent100_300_11savgol_mean', 'wave_Height_Moving_skew_12_mean', 'chlorophyll_Moving_min_6_mean', 'temperatureSurface_October_median', 'salinitySurface_January_min', 'phosphateSurface_Moving_std_10_mean', 'salinity100_300_years_quantile_5', 'zonalCurrent100_300_Moving_min_11_mean', 'phosphate100_300_December_median', 'chlorophyll_November_quantile_99', 'temperature100_300_April_min', 'zonalCurrent100_300_years_quantile_1', 'dissolvedOxygen100_300_Moving_std_5_mean', 'nitrateSurface_Moving_median_5_mean', 'wave_Heighttrim_mean_20%', 'wave_Height_50wiener_mean', 'temperature100_300_June_quantile_1', 'euphoticDepth_December_quantile_90', 'nitrate100_300_Moving_median_6_mean', 'zonalCurrent100_300_May_quantile_99', 'nitrateSurface_Moving_quantile_1_8', 'meridionalCurrentSurface_Moving_max_8_mean', 'wave_Height_Moving_quantile_10_12', 'dissolvedOxygenSurface_Moving_average_12_mean', 'meridionalCurrent100_300_Moving_quantile_5_7', 'meridionalCurrentSurface_Moving_skew_7_mean', 'temperature100_300_Moving_quantile_95_5', 'phosphate100_300_quantile_5', 'meridionalCurrent100_300_Moving_quantile_99_10', 'salinity100_300_July_quantile_1', 'zonalCurrentSurface_Moving_quantile_90_8', 'meridionalCurrent100_300_Moving_quantile_1_12', 'phosphate100_300_August_quantile_5', 'temperatureSurface_neighbor_min', 'secchiDiskDepth_February_quantile_99', 'salinitySurface_Moving_median_3_mean', 'nitrate100_300_Moving_std_10_mean', 'zonalCurrentSurface_Moving_quantile_1_3', 'salinitySurface_March_quantile_90', 'nitrateSurface_Expanding_max_mean', 'temperature100_300_Moving_quantile_1_2', 'phosphate100_300_May_max', 'nitrateSurface_Moving_min_11_mean', 'chlorophyll_argrelminmax_count', 'nitrateSurface_Moving_min_9_mean', 'zonalCurrentSurface_November_quantile_1', 'zonalCurrent100_300_years_quantile_95', 'salinity100_300_Moving_median_9_mean', 'dissolvedOxygenSurface_November_quantile_1', 'secchiDiskDepth_Moving_std_9_mean', 'phosphateSurface_October_quantile_99', 'phosphate100_300_Moving_average_7_mean', 'zonalCurrent100_300_mean_change_peak_rate', 'salinity100_300_Moving_average_4_mean', 'secchiDiskDepth_Moving_quantile_90_11', 'zonalCurrentSurface_Hann_window_mean', 'temperature100_300_February_mean', 'temperature100_300_Moving_quantile_95_7', 'chlorophyll_3medfilt_mean', 'salinity100_300_April_quantile_95', 'nitrateSurfacetrim_mean_25%', 'meridionalCurrentSurface_May_quantile_99', 'wave_Height_Moving_skew_6_mean', 'phosphate100_300_August_quantile_99', 'wave_Height_Moving_quantile_95_5', 'nitrateSurface_years_quantile_1', 'euphoticDepth_January_max', 'nitrateSurface_Moving_average_8_mean', 'chlorophyll_Moving_quantile_90_2', 'euphoticDepth_April_quantile_10', 'zonalCurrent100_300_August_quantile_5', 'dissolvedOxygenSurface_Moving_average_4_mean', 'salinitySurface_August_quantile_90', 'secchiDiskDepth_3gauss_spline_mean', 'zonalCurrent100_300_Moving_quantile_10_5', 'meridionalCurrent100_300_Moving_average_11_mean', 'temperatureSurface_March_quantile_1', 'meridionalCurrent100_300_Moving_quantile_99_6', 'phosphate100_300_years_quantile_99', 'wave_Height_Moving_max_4_mean', 'dissolvedOxygenSurface_decimate_mean', 'wave_Height_November_std', 'nitrate100_300_Moving_quantile_90_5', 'salinity100_300_May_max', 'phosphate100_300_years_median', 'dissolvedOxygenSurface_Moving_quantile_10_8', 'temperatureSurface_April_quantile_95', 'zonalCurrentSurface_September_quantile_95', 'secchiDiskDepth_October_median', 'temperature100_300_June_mean', 'temperatureSurface_symiirorder2_mean', 'temperature100_300_Moving_skew_11_mean', 'temperature100_300_October_quantile_99', 'euphoticDepth_Moving_quantile_90_9', 'phosphate100_300_Moving_quantile_99_12', 'nitrateSurface_June_quantile_90', 'temperatureSurface_Moving_quantile_95_5', 'salinitySurface_December_quantile_95', 'phosphate100_300_Moving_min_4_mean', 'phosphateSurface_max_to_min_diff', 'euphoticDepth_Moving_quantile_5_9', 'salinity100_300_25savgol_mean', 'temperature100_300_November_quantile_90', 'temperature100_300_Moving_quantile_1_11', 'phosphateSurface_years_quantile_1', 'zonalCurrentSurface_Expanding_std_mean', 'phosphateSurface_years_quantile_90', 'phosphateSurface_11savgol_mean', 'salinity100_300_April_quantile_90', 'temperature100_300_Moving_min_5_mean', 'dissolvedOxygen100_300_Moving_average_5_mean', 'dissolvedOxygenSurface_Expanding_quantile_99', 'wave_Height_Moving_quantile_5_4', 'dissolvedOxygenSurface_Moving_median_7_mean', 'salinitySurface_February_max', 'dissolvedOxygenSurface_January_median', 'salinity100_300_mad', 'meridionalCurrentSurface_Moving_average_12_mean', 'euphoticDepth_March_quantile_99', 'dissolvedOxygen100_300_Moving_skew_10_mean', 'phosphate100_300_Moving_quantile_99_2', 'temperatureSurface_Moving_median_7_mean', 'salinity100_300_years_quantile_99', 'temperatureSurface_January_quantile_10', 'meridionalCurrentSurface_September_quantile_99', 'phosphate100_300_March_max', 'salinity100_300_Moving_quantile_90_2', 'secchiDiskDepthiqr1', 'meridionalCurrentSurface_August_quantile_1', 'temperature100_300_July_quantile_1', 'euphoticDepth_25savgol_mean', 'temperature100_300_Moving_std_8_mean', 'zonalCurrentSurface_July_quantile_1', 'temperatureSurface_Moving_quantile_99_12', 'meridionalCurrentSurface_Expanding_std_mean', 'zonalCurrent100_300_April_quantile_1', 'phosphate100_300_July_quantile_5', 'salinitySurface_Moving_max_9_mean', 'zonalCurrentSurface_Moving_std_10_mean', 'phosphate100_300_June_quantile_90', 'zonalCurrentSurface_mean_change_argrelmin', 'euphoticDepth_Moving_quantile_5_8', 'dissolvedOxygen100_300iqr1', 'chlorophyll_September_median', 'temperature100_300_Moving_quantile_99_4', 'phosphateSurface_Moving_max_2_mean', 'secchiDiskDepth_September_max', 'meridionalCurrent100_300_Moving_quantile_95_5', 'phosphate100_300_June_median', 'euphoticDepthtrim_mean_10%', 'dissolvedOxygenSurface_Moving_skew_8_mean', 'nitrate100_300_July_max', 'euphoticDepth_July_min', 'dissolvedOxygen100_300_Expanding_quantile_1', 'wave_Height_Moving_median_10_mean', 'salinity100_300_Moving_median_3_mean', 'meridionalCurrentSurface_March_median', 'wave_Height_Moving_quantile_5_2', 'nitrate100_300_April_mean', 'nitrateSurface_Moving_max_9_mean', 'phosphate100_300_Moving_std_11_mean', 'phosphate100_300_Moving_quantile_99_10', 'chlorophyll_Moving_median_9_mean', 'dissolvedOxygen100_300_argrelmax_count', 'temperature100_300_argrelmax_count', 'meridionalCurrentSurface_Moving_average_8_mean', 'secchiDiskDepth_November_quantile_99', 'temperatureSurface_Moving_quantile_1_4', 'salinitySurface_20wiener_mean', 'chlorophyll_Moving_average_7_mean', 'dissolvedOxygenSurfacetrim_mean_25%', 'wave_Height_January_quantile_5', 'temperature100_300_Moving_quantile_99_11', 'salinitySurface_June_quantile_99', 'phosphateSurface_Moving_average_8_mean', 'phosphateSurface_Moving_quantile_10_8', 'secchiDiskDepth_Moving_average_7_mean', 'meridionalCurrentSurface_June_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_90_11', 'nitrateSurface_Moving_quantile_95_12', 'phosphate100_300_September_quantile_10', 'euphoticDepth_September_max', 'phosphate100_300_May_mean', 'zonalCurrent100_300_Moving_min_5_mean', 'temperature100_300_Moving_std_3_mean', 'meridionalCurrentSurface_November_quantile_1', 'temperature100_300_July_max', 'zonalCurrent100_300_Moving_median_9_mean', 'phosphate100_300_April_mean', 'nitrateSurface_Moving_quantile_5_10', 'nitrate100_300_Moving_quantile_10_7', 'zonalCurrent100_300_August_quantile_1', 'zonalCurrent100_300_March_quantile_10', 'meridionalCurrent100_300_Moving_min_11_mean', 'nitrateSurface_July_quantile_5', 'nitrate100_300_Moving_quantile_5_7', 'meridionalCurrentSurface_quantile_95', 'zonalCurrent100_300_July_quantile_1', 'meridionalCurrent100_300_Expanding_quantile_90', 'phosphate100_300_5savgol_mean', 'secchiDiskDepth_August_max', 'secchiDiskDepth_July_max', 'phosphate100_300_Hann_window_mean', 'meridionalCurrentSurface_Moving_average_6_mean', 'salinity100_300_quantile_1', 'nitrate100_300_September_mean', 'salinitySurface_October_quantile_5', 'nitrate100_300_April_quantile_95', 'chlorophyll_June_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_99_2', 'euphoticDepth_11savgol_mean', 'dissolvedOxygen100_300_years_quantile_10', 'chlorophyll_25medfilt_mean', 'secchiDiskDepth_Expanding_quantile_95', 'zonalCurrent100_300_Moving_average_9_mean', 'dissolvedOxygenSurface_May_median', 'nitrateSurface_January_quantile_90', 'phosphateSurface_Moving_quantile_99_4', 'secchiDiskDepth_July_median', 'phosphate100_300_Moving_quantile_1_3', 'secchiDiskDepth_Moving_max_9_mean', 'chlorophyll_Moving_quantile_10_9', 'dissolvedOxygen100_300_December_quantile_99', 'meridionalCurrentSurface_Moving_median_12_mean', 'temperatureSurface_Moving_skew_10_mean', 'meridionalCurrent100_300_April_quantile_5', 'meridionalCurrent100_300_December_quantile_95', 'meridionalCurrent100_300_Moving_min_7_mean', 'phosphateSurface_September_quantile_95', 'temperatureSurface_Moving_min_9_mean', 'euphoticDepth_Moving_average_2_mean', 'zonalCurrentSurface_July_quantile_99', 'secchiDiskDepth_June_quantile_90', 'dissolvedOxygen100_300_May_quantile_95', 'temperatureSurface_Moving_quantile_95_3', 'phosphateSurface_3spline_mean', 'meridionalCurrentSurface_Moving_average_2_mean', 'temperatureSurface_median', 'salinity100_300_Moving_quantile_95_11', 'meridionalCurrentSurface_June_quantile_5', 'nitrateSurface_Expanding_std_mean', 'salinitySurface_Moving_quantile_90_9', 'salinity100_300_Moving_quantile_99_3', 'wave_Height_Moving_quantile_99_10', 'euphoticDepth_Moving_median_4_mean', 'meridionalCurrentSurface_Moving_std_12_mean', 'salinitySurface_February_min', 'temperatureSurface_December_quantile_1', 'wave_Height_May_quantile_10', 'salinity100_300_argrelmax_count', 'dissolvedOxygen100_300_5medfilt_mean', 'nitrate100_300_Moving_average_10_mean', 'salinity100_300_Moving_skew_10_mean', 'nitrate100_300_peak_mean', 'euphoticDepth_December_quantile_99', 'dissolvedOxygenSurface_Moving_max_6_mean', 'zonalCurrentSurface_July_quantile_5', 'nitrateSurface_March_median', 'phosphateSurface_Moving_skew_10_mean', 'nitrate100_300_Moving_quantile_95_11', 'zonalCurrentSurface_Moving_std_3_mean', 'euphoticDepth_years_median', 'dissolvedOxygenSurface_Moving_quantile_99_11', 'wave_Height_Moving_quantile_10_3', 'euphoticDepth_Moving_max_7_mean', 'zonalCurrentSurface_Moving_average_12_mean', 'meridionalCurrentSurface_Moving_quantile_99_6', 'dissolvedOxygenSurface_Expanding_quantile_90', 'salinitySurface_Moving_median_8_mean', 'dissolvedOxygenSurface_Expanding_median_mean', 'nitrateSurface_October_quantile_95', 'salinitySurface_Moving_std_11_mean', 'secchiDiskDepth_Moving_std_6_mean', 'salinity100_300_August_quantile_10', 'chlorophyll_Expanding_std_mean', 'temperature100_300_July_quantile_95', 'temperatureSurface_Moving_quantile_10_12', 'secchiDiskDepth_4wiener_mean', 'wave_Height_symiirorder1_mean', 'dissolvedOxygenSurface_February_quantile_95', 'chlorophyll_April_median', 'nitrateSurface_20wiener_mean', 'meridionalCurrentSurface_Moving_median_6_mean', 'salinitySurface_Moving_quantile_5_3', 'nitrateSurface_4spline_mean', 'phosphateSurface_10wiener_mean', 'phosphate100_300_July_mean', 'phosphate100_300_August_quantile_10', 'wave_Height_Moving_quantile_10_9', 'meridionalCurrentSurface_Moving_max_11_mean', 'zonalCurrentSurface_October_quantile_5', 'salinity100_300_December_quantile_95', 'nitrate100_300_April_quantile_10', 'phosphate100_300_Moving_average_10_mean', 'meridionalCurrentSurface_Moving_std_9_mean', 'euphoticDepth_Moving_quantile_99_5', 'nitrateSurface_3gauss_spline_mean', 'phosphateSurface_Moving_std_3_mean', 'chlorophyll_Moving_min_5_mean', 'zonalCurrentSurface_February_quantile_1', 'secchiDiskDepth_December_quantile_99', 'phosphate100_300_June_quantile_10', 'salinity100_300_February_min', 'zonalCurrentSurface_October_quantile_10', 'zonalCurrent100_300_Moving_quantile_99_2', 'dissolvedOxygen100_300_median', 'salinity100_300_Moving_average_10_mean', 'salinitySurface_years_quantile_5', 'phosphate100_300_years_quantile_10', 'temperatureSurface_Moving_quantile_10_2', 'meridionalCurrentSurface_Moving_std_2_mean', 'dissolvedOxygen100_300_September_quantile_10', 'wave_Height_Moving_quantile_5_9', 'zonalCurrent100_300_years_min', 'salinity100_300_Moving_min_10_mean', 'temperature100_300_years_min', 'dissolvedOxygen100_300_October_quantile_5', 'dissolvedOxygen100_300_August_median', 'meridionalCurrent100_300_Moving_average_4_mean', 'zonalCurrentSurface_Moving_quantile_95_10', 'meridionalCurrent100_300_January_quantile_95', 'temperatureSurface_Moving_median_5_mean', 'salinity100_300_decimate_mean', 'phosphate100_300_20wiener_mean', 'zonalCurrent100_300_Moving_quantile_1_6', 'phosphate100_300_November_mean', 'meridionalCurrentSurface_Moving_average_7_mean', 'dissolvedOxygenSurface_November_quantile_95', 'dissolvedOxygenSurface_September_quantile_95', 'salinity100_300_Moving_quantile_1_12', 'temperature100_300_Moving_quantile_5_12', 'nitrateSurface_Moving_min_4_mean', 'dissolvedOxygen100_300_Moving_median_10_mean', 'euphoticDepth_Moving_quantile_99_10', 'zonalCurrentSurface_March_quantile_95', 'euphoticDepth_Moving_quantile_1_10', 'salinitySurface_Moving_max_3_mean', 'phosphateSurface_Moving_average_12_mean', 'chlorophyll_Moving_quantile_99_10', 'euphoticDepth_Moving_max_8_mean', 'dissolvedOxygenSurface_December_quantile_1', 'meridionalCurrent100_300_January_median', 'salinity100_300_Moving_average_9_mean', 'dissolvedOxygenSurface_Moving_quantile_95_8', 'phosphate100_300_Moving_min_9_mean', 'salinitySurface_mad', 'salinitySurface_quantile_1', 'dissolvedOxygen100_300_Moving_average_2_mean', 'chlorophyll_Moving_median_6_mean', 'salinitySurface_Moving_quantile_10_2', 'salinitySurface_May_max', 'dissolvedOxygen100_300_July_quantile_95', 'dissolvedOxygenSurface_December_quantile_5', 'dissolvedOxygen100_300_symiirorder1_mean', 'meridionalCurrent100_300_Moving_quantile_99_2', 'meridionalCurrentSurface_April_quantile_1', 'salinitySurface_January_quantile_95', 'nitrateSurface_Moving_quantile_90_6', 'phosphate100_300_February_min', 'salinity100_300_February_quantile_5', 'temperature100_300_Moving_average_7_mean', 'salinitySurface_quantile_99', 'dissolvedOxygenSurface_Moving_max_3_mean', 'dissolvedOxygen100_300_January_quantile_99', 'dissolvedOxygenSurface_Expanding_quantile_5', 'temperatureSurface_Moving_quantile_5_2', 'wave_Height_Moving_median_11_mean', 'phosphateSurface_January_quantile_95', 'zonalCurrentSurface_Moving_quantile_1_6', 'wave_Height_Moving_quantile_99_2', 'meridionalCurrent100_300trim_mean_20%', 'wave_Height_25savgol_mean', 'dissolvedOxygenSurface_September_quantile_99', 'zonalCurrent100_300_Moving_median_8_mean', 'meridionalCurrentSurface_April_median', 'secchiDiskDepth_Moving_quantile_99_12', 'chlorophyll_Moving_min_9_mean', 'secchiDiskDepth_Expanding_median_mean', 'chlorophyll_Moving_std_2_mean', 'nitrate100_300_September_quantile_10', 'nitrate100_300_Moving_quantile_95_6', 'phosphate100_300_Expanding_quantile_10', 'zonalCurrentSurface_Expanding_average_mean', 'zonalCurrentSurface_Moving_median_3_mean', 'zonalCurrent100_300_10wiener_mean', 'salinitySurface_April_quantile_1', 'dissolvedOxygen100_300_years_mean', 'dissolvedOxygen100_300_Moving_quantile_10_9', 'zonalCurrentSurface_Expanding_quantile_90', 'dissolvedOxygen100_300_March_quantile_1', 'meridionalCurrent100_300_May_quantile_95', 'secchiDiskDepth_Moving_median_10_mean', 'phosphateSurface_Moving_quantile_10_4', 'nitrate100_300_Moving_max_4_mean', 'temperature100_300_October_quantile_5', 'nitrateSurface_October_std', 'wave_Height_Moving_average_11_mean', 'phosphate100_300_Moving_quantile_95_12', 'meridionalCurrent100_300_quantile_95', 'dissolvedOxygen100_300_4wiener_mean', 'euphoticDepth_Moving_quantile_10_3', 'zonalCurrent100_300_Moving_min_12_mean', 'meridionalCurrent100_300_Moving_median_11_mean', 'dissolvedOxygenSurface_Moving_max_11_mean', 'nitrateSurface_5medfilt_mean', 'salinity100_300iqr1', 'meridionalCurrentSurface_Moving_average_3_mean', 'meridionalCurrentSurface_Moving_max_9_mean', 'zonalCurrentSurface_3gauss_spline_mean', 'euphoticDepth_Moving_std_3_mean', 'salinitySurfacetrim_mean_20%', 'chlorophyll_symiirorder1_mean', 'meridionalCurrentSurface_10medfilt_mean', 'zonalCurrent100_300_Moving_std_6_mean', 'nitrate100_300_quantile_1', 'euphoticDepth_July_quantile_99', 'chlorophyll_Moving_min_12_mean', 'chlorophyll_Expanding_max_mean', 'euphoticDepth_Moving_quantile_1_12', 'secchiDiskDepth_August_min', 'phosphateSurface_June_quantile_90', 'temperatureSurface_10medfilt_mean', 'phosphateSurface_April_quantile_5', 'dissolvedOxygen100_300_Moving_min_4_mean', 'nitrateSurface_10medfilt_mean', 'salinitySurface_October_min', 'salinity100_300trim_mean_10%', 'wave_Height_November_quantile_5', 'salinity100_300_Moving_std_12_mean', 'dissolvedOxygen100_300_Moving_quantile_95_7', 'temperatureSurface_November_quantile_90', 'meridionalCurrent100_300_Moving_median_5_mean', 'nitrate100_300_Moving_skew_8_mean', 'phosphate100_300_January_median', 'dissolvedOxygen100_300_Moving_quantile_95_10', 'zonalCurrentSurface_quantile_95', 'secchiDiskDepth_March_quantile_95', 'zonalCurrentSurface_Moving_quantile_10_11', 'phosphateSurface_Moving_quantile_90_7', 'meridionalCurrentSurface_Moving_std_10_mean', 'temperature100_300_Moving_std_7_mean', 'temperatureSurface_February_quantile_99', 'phosphate100_300_September_quantile_5', 'secchiDiskDepth_December_quantile_1', 'temperature100_300_May_mean', 'euphoticDepth_November_mean', 'meridionalCurrentSurface_October_quantile_99', 'nitrate100_300_quantile_95', 'meridionalCurrentSurface_quantile_5', 'wave_Height_April_quantile_99', 'meridionalCurrentSurface_Moving_quantile_90_12', 'secchiDiskDepth_May_min', 'temperatureSurface_February_max', 'salinitySurface_Moving_quantile_90_4', 'nitrate100_300_Expanding_quantile_99', 'salinity100_300_August_mean', 'nitrate100_300_December_quantile_90', 'secchiDiskDepth_Moving_quantile_90_10', 'meridionalCurrentSurface_Moving_min_5_mean', 'chlorophyll_Moving_std_7_mean', 'zonalCurrentSurface_August_quantile_1', 'salinity100_300_Moving_max_8_mean', 'secchiDiskDepth_Moving_min_2_mean', 'dissolvedOxygenSurface_Expanding_average_mean', 'temperature100_300_Moving_max_10_mean', 'secchiDiskDepth_June_quantile_1', 'chlorophyll_October_quantile_99', 'phosphate100_300_Moving_quantile_99_9', 'meridionalCurrentSurface_Expanding_quantile_90', 'salinitySurface_March_quantile_99', 'dissolvedOxygenSurface_Moving_min_6_mean', 'nitrate100_300trim_mean_15%', 'nitrate100_300_December_min', 'temperature100_300_10wiener_mean', 'temperatureSurface_January_min', 'zonalCurrent100_300_Moving_min_3_mean', 'zonalCurrent100_300_Moving_average_2_mean', 'wave_Height_Moving_min_8_mean', 'euphoticDepth_Moving_quantile_5_10', 'salinitySurface_Moving_average_10_mean', 'meridionalCurrent100_300_March_quantile_99', 'salinitySurface_June_min', 'dissolvedOxygenSurface_September_quantile_1', 'secchiDiskDepth_March_quantile_10', 'temperatureSurface_Moving_average_11_mean', 'phosphate100_300_Moving_max_11_mean', 'phosphateSurface_Moving_min_10_mean', 'salinity100_300_mean_change_rate', 'nitrateSurface_Moving_quantile_1_10', 'euphoticDepth_Moving_average_12_mean', 'meridionalCurrent100_300_May_quantile_99', 'nitrateSurface_mean_change_argrelmin', 'salinity100_300_January_quantile_10', 'secchiDiskDepth_Moving_average_8_mean', 'dissolvedOxygen100_300_Moving_quantile_1_11', 'salinitySurface_Moving_min_8_mean', 'zonalCurrent100_300trim_mean_20%', 'temperatureSurface_years_std', 'euphoticDepth_May_quantile_90', 'temperatureSurface_Moving_quantile_99_8', 'phosphate100_300_years_quantile_95', 'chlorophyll_May_quantile_10', 'zonalCurrent100_300_December_quantile_1', 'euphoticDepth_Moving_max_6_mean', 'chlorophyll_August_quantile_95', 'meridionalCurrentSurface_Moving_quantile_99_7', 'phosphate100_300_Expanding_median_mean', 'wave_Height_February_quantile_5', 'meridionalCurrent100_300_Expanding_quantile_1', 'temperatureSurface_August_quantile_90', 'temperature100_300_April_quantile_10', 'temperatureSurface_Moving_max_12_mean', 'meridionalCurrent100_300_Moving_quantile_95_8', 'dissolvedOxygenSurface_peak_max', 'chlorophyll_Moving_median_7_mean', 'salinity100_300_Moving_quantile_90_4', 'nitrate100_300_Moving_quantile_1_7', 'nitrateSurface_Moving_average_10_mean', 'dissolvedOxygenSurface_years_quantile_99', 'nitrate100_300_July_quantile_1', 'secchiDiskDepth_10wiener_mean', 'nitrateSurface_Moving_quantile_95_10', 'nitrateSurface_symiirorder1_mean', 'dissolvedOxygenSurface_Moving_median_8_mean', 'zonalCurrentSurface_December_quantile_1', 'euphoticDepth_November_quantile_95', 'meridionalCurrent100_300_Moving_quantile_10_9', 'temperature100_300_years_quantile_95', 'chlorophyll_December_quantile_5', 'nitrateSurface_Moving_max_10_mean', 'zonalCurrentSurface_Moving_median_2_mean', 'euphoticDepth_5medfilt_mean', 'dissolvedOxygen100_300_Moving_quantile_1_6', 'euphoticDepth_Moving_quantile_95_3', 'dissolvedOxygen100_300_25medfilt_mean', 'zonalCurrentSurface_August_median', 'zonalCurrentSurface_Moving_std_7_mean', 'meridionalCurrent100_300_February_quantile_5', 'euphoticDepth_July_median', 'euphoticDepth_Moving_quantile_95_7', 'meridionalCurrent100_300_December_quantile_5', 'salinity100_300_March_quantile_5', 'dissolvedOxygenSurface_Moving_median_3_mean', 'phosphate100_300_5medfilt_mean', 'temperatureSurface_Moving_quantile_95_6', 'dissolvedOxygenSurface_October_max', 'temperatureSurface_April_quantile_1', 'nitrateSurface_July_quantile_99', 'salinitySurface_Moving_quantile_10_12', 'meridionalCurrent100_300_Moving_quantile_90_8', 'wave_Height_2gauss_spline_mean', 'salinity100_300_December_quantile_10', 'salinitySurface_Moving_median_9_mean', 'meridionalCurrent100_300_February_quantile_99', 'zonalCurrent100_300_August_median', 'phosphate100_300_symiirorder1_mean', 'nitrateSurface_Moving_quantile_90_2', 'nitrateSurface_Moving_average_11_mean', 'zonalCurrentSurface_Moving_quantile_95_11', 'meridionalCurrent100_300_peak_mean', 'zonalCurrent100_300_Moving_max_8_mean', 'zonalCurrentSurface_Moving_quantile_90_11', 'meridionalCurrentSurface_Moving_quantile_5_4', 'dissolvedOxygenSurface_Moving_median_6_mean', 'zonalCurrentSurface_Moving_average_7_mean', 'dissolvedOxygen100_300_April_quantile_5', 'nitrate100_300_Expanding_quantile_1', 'nitrateSurface_December_quantile_95', 'meridionalCurrentSurface_Expanding_quantile_10', 'meridionalCurrentSurface_Moving_skew_11_mean', 'dissolvedOxygen100_300_Moving_min_11_mean', 'salinitySurface_August_quantile_95', 'salinity100_300_quantile_99', 'secchiDiskDepth_Moving_quantile_1_7', 'phosphateSurface_quantile_10', 'salinity100_300_Moving_quantile_95_10', 'nitrateSurface_Moving_skew_12_mean', 'nitrate100_300_June_quantile_5', 'phosphateSurface_December_quantile_99', 'nitrateSurface_Moving_quantile_5_6', 'phosphate100_300_January_quantile_95', 'meridionalCurrent100_300_Moving_quantile_10_3', 'secchiDiskDepth_Moving_quantile_1_4', 'zonalCurrent100_300_September_quantile_1', 'temperatureSurface_Moving_quantile_10_3', 'dissolvedOxygen100_300_Moving_max_3_mean', 'nitrate100_300_Moving_quantile_90_6', 'dissolvedOxygen100_300_Moving_quantile_95_3', 'zonalCurrentSurface_10wiener_mean', 'nitrateSurface_50wiener_mean', 'temperature100_300_Moving_quantile_5_4', 'dissolvedOxygen100_300_Moving_quantile_95_4', 'euphoticDepth_median', 'zonalCurrent100_300_Moving_quantile_1_10', 'zonalCurrent100_300_Moving_quantile_1_7', 'dissolvedOxygenSurface_Moving_quantile_5_4', 'temperatureSurface_quantile_1', 'dissolvedOxygenSurface_Moving_quantile_1_2', 'dissolvedOxygenSurface_3medfilt_mean', 'phosphate100_300_Expanding_quantile_1', 'euphoticDepth_quantile_10', 'wave_Height_Hann_window_mean', 'secchiDiskDepth_Moving_min_4_mean', 'nitrateSurface_Moving_quantile_90_3', 'secchiDiskDepth_July_quantile_90', 'euphoticDepth_max_to_min', 'nitrate100_300_Moving_quantile_1_11', 'zonalCurrentSurface_Moving_average_5_mean', 'salinitySurface_May_min', 'zonalCurrentSurface_February_quantile_99', 'nitrate100_300_March_quantile_95', 'chlorophyll_April_quantile_90', 'nitrateSurface_Moving_max_12_mean', 'zonalCurrentSurface_February_quantile_10', 'temperatureSurface_July_quantile_1', 'dissolvedOxygen100_300_July_quantile_1', 'meridionalCurrentSurface_Expanding_quantile_99', 'dissolvedOxygenSurface_Moving_median_11_mean', 'dissolvedOxygenSurface_Moving_max_8_mean', 'nitrateSurface_Moving_quantile_10_7', 'phosphate100_300_April_min', 'salinitySurface_Moving_max_11_mean', 'chlorophyll_May_quantile_5', 'nitrate100_300_Moving_max_11_mean', 'euphoticDepth_April_max', 'meridionalCurrentSurface_Moving_std_6_mean', 'dissolvedOxygen100_300_May_quantile_10', 'euphoticDepth_October_quantile_95', 'phosphate100_300_September_mean', 'dissolvedOxygen100_300_Expanding_min_mean', 'phosphateSurface_August_median', 'euphoticDepth_Moving_std_7_mean', 'chlorophyll_Moving_quantile_99_9', 'chlorophyll_years_quantile_5', 'phosphate100_300_February_max', 'wave_Heighttrim_mean_10%', 'chlorophyll_Moving_std_5_mean', 'salinity100_300_June_quantile_99', 'phosphateSurface_December_quantile_95', 'salinity100_300_Moving_quantile_10_4', 'nitrate100_300_June_mean', 'meridionalCurrent100_300_Expanding_std_mean', 'phosphateSurface_mean_change_argrelmax', 'temperature100_300_years_max', 'phosphateSurface_June_quantile_10', 'temperature100_300_April_quantile_99', 'meridionalCurrent100_300_Moving_quantile_10_2', 'phosphateSurface_Moving_skew_12_mean', 'dissolvedOxygen100_300_Moving_skew_6_mean', 'salinitySurface_Expanding_min_mean', 'secchiDiskDepth_Moving_quantile_5_5', 'zonalCurrent100_300_25medfilt_mean', 'meridionalCurrent100_300_Moving_min_6_mean', 'meridionalCurrent100_300_August_quantile_5', 'phosphateSurface_October_max', 'wave_Height_Expanding_quantile_90', 'dissolvedOxygen100_300_March_mean', 'wave_Height_mean_change_argrelmax', 'zonalCurrent100_300_Moving_quantile_10_4', 'phosphate100_300_quantile_90', 'dissolvedOxygen100_300_March_max', 'salinity100_300_Moving_median_10_mean', 'wave_Height_Moving_quantile_90_8', 'temperature100_300_Moving_quantile_5_11', 'wave_Height_Moving_quantile_10_6', 'phosphateSurface_Moving_min_2_mean', 'secchiDiskDepth_mean_change_peak_rate', 'chlorophyll_April_mean', 'dissolvedOxygen100_300_quantile_1', 'temperature100_300_September_quantile_1', 'dissolvedOxygenSurface_January_quantile_5', 'zonalCurrentSurface_Moving_quantile_10_4', 'nitrate100_300_May_quantile_5', 'zonalCurrentSurface_Moving_max_6_mean', 'salinitySurface_Moving_quantile_5_7', 'phosphateSurface_February_quantile_95', 'dissolvedOxygen100_300_Moving_quantile_1_3', 'nitrate100_300_11savgol_mean', 'temperature100_300_Moving_min_12_mean', 'zonalCurrentSurface_Moving_average_10_mean', 'wave_Height_June_quantile_99', 'salinity100_300_November_quantile_95', 'meridionalCurrent100_300_October_quantile_5', 'dissolvedOxygen100_300_February_quantile_5', 'chlorophyll_Moving_quantile_10_5', 'zonalCurrent100_300_3medfilt_mean', 'dissolvedOxygen100_300_Moving_quantile_5_12', 'zonalCurrentSurface_years_quantile_90', 'phosphate100_300_Moving_max_2_mean', 'chlorophyll_Moving_max_4_mean', 'phosphateSurface_years_quantile_10', 'salinitySurface_July_mean', 'temperatureSurface_November_min', 'secchiDiskDepth_years_quantile_5', 'nitrate100_300_March_quantile_99', 'meridionalCurrentSurface_Moving_average_10_mean', 'chlorophyll_Moving_std_8_mean', 'phosphate100_300_years_min', 'dissolvedOxygen100_300_April_quantile_1', 'meridionalCurrent100_300_Moving_quantile_1_2', 'nitrate100_300_May_quantile_10', 'secchiDiskDepth_June_quantile_5', 'salinity100_300_Expanding_quantile_1', 'wave_Height_Moving_min_10_mean', 'euphoticDepth_Moving_quantile_10_9', 'salinity100_300iqr', 'meridionalCurrent100_300_Moving_quantile_5_2', 'phosphate100_300_4wiener_mean', 'nitrate100_300_51savgol_mean', 'chlorophyll_Moving_min_3_mean', 'meridionalCurrentSurface_4spline_mean', 'euphoticDepth_Moving_min_7_mean', 'phosphateSurface_Moving_quantile_95_3', 'nitrateSurface_May_median', 'zonalCurrentSurface_Moving_quantile_10_9', 'dissolvedOxygen100_300_Moving_median_12_mean', 'secchiDiskDepth_years_quantile_1', 'meridionalCurrentSurface_Moving_average_11_mean', 'chlorophyll_February_quantile_5', 'temperature100_300_June_median', 'dissolvedOxygenSurface_September_mean', 'salinitySurface_3medfilt_mean', 'zonalCurrent100_300_Moving_average_11_mean', 'zonalCurrent100_300_Moving_quantile_90_11', 'euphoticDepth_September_quantile_10', 'phosphateSurface_December_median', 'phosphate100_300_June_mean', 'chlorophyll_2spline_mean', 'secchiDiskDepth_Moving_quantile_90_8', 'zonalCurrentSurface_Moving_min_9_mean', 'zonalCurrent100_300_Expanding_median_mean', 'phosphateSurface_Moving_quantile_95_8', 'euphoticDepth_May_max', 'meridionalCurrentSurface_July_quantile_1', 'nitrate100_300_December_quantile_1', 'meridionalCurrent100_300_years_min', 'salinity100_300_Moving_max_10_mean', 'meridionalCurrentSurface_Moving_quantile_90_6', 'chlorophyll_Moving_median_3_mean', 'chlorophyll_Moving_skew_6_mean', 'meridionalCurrentSurface_October_quantile_95', 'nitrateSurface_mad', 'temperature100_300_Moving_quantile_95_9', 'zonalCurrent100_300_Moving_quantile_95_12', 'temperature100_300_Moving_quantile_99_6', 'meridionalCurrentSurface_Hann_window_mean', 'secchiDiskDepth_20wiener_mean', 'nitrateSurface_Moving_quantile_95_8', 'temperatureSurface_25savgol_mean', 'zonalCurrent100_300_May_quantile_5', 'nitrate100_300_quantile_5', 'dissolvedOxygen100_300_April_quantile_95', 'nitrate100_300_Moving_quantile_5_9', 'secchiDiskDepth_Hann_window_mean', 'meridionalCurrent100_300_Moving_quantile_99_11', 'salinity100_300_Moving_quantile_99_9', 'temperature100_300_max_to_min', 'nitrate100_300iqr', 'temperatureSurface_July_mean', 'salinity100_300_years_quantile_95', 'zonalCurrent100_300_February_quantile_95', 'meridionalCurrentSurface_Moving_quantile_95_5', 'euphoticDepth_Moving_average_7_mean', 'nitrateSurface_Moving_max_3_mean', 'meridionalCurrentSurface_Moving_median_3_mean', 'wave_Height_Moving_average_9_mean', 'meridionalCurrentSurface_Moving_median_7_mean', 'meridionalCurrentSurface_Moving_median_9_mean', 'temperatureSurface_Moving_quantile_90_11', 'secchiDiskDepth_January_mean', 'meridionalCurrent100_300_Moving_average_6_mean', 'temperatureSurface_years_quantile_1', 'chlorophyll_10wiener_mean', 'zonalCurrent100_300_June_quantile_5', 'meridionalCurrent100_300trim_mean_25%', 'euphoticDepth_September_median', 'temperature100_300_March_quantile_99', 'dissolvedOxygen100_300_Moving_average_4_mean', 'chlorophyll_August_quantile_99', 'phosphateSurface_Moving_quantile_1_7', 'meridionalCurrentSurface_Moving_min_3_mean', 'salinity100_300_Moving_max_12_mean', 'nitrateSurface_5savgol_mean', 'phosphateSurface_Moving_quantile_99_12', 'nitrate100_300_Moving_average_5_mean', 'zonalCurrentSurface_Moving_quantile_95_8', 'salinity100_300_Moving_quantile_99_4', 'phosphate100_300_mean_change_argrelmax', 'meridionalCurrent100_300_Moving_std_11_mean', 'temperature100_300_November_quantile_10', 'temperatureSurface_October_min', 'wave_Height_years_quantile_5', 'phosphate100_300_10wiener_mean', 'temperature100_300_June_quantile_95', 'meridionalCurrent100_300_4wiener_mean', 'temperatureSurface_January_mean', 'salinitySurface_October_quantile_90', 'chlorophyll_April_quantile_95', 'euphoticDepth_October_quantile_10', 'euphoticDepth_Moving_median_3_mean', 'salinitySurface_5medfilt_mean', 'wave_Height_decimate_mean', 'temperatureSurface_November_quantile_99', 'phosphateSurfacetrim_mean_5%', 'wave_Height_Moving_min_2_mean', 'zonalCurrentSurface_decimate_mean', 'salinity100_300_November_mean', 'euphoticDepth_Moving_quantile_5_3', 'dissolvedOxygenSurface_May_quantile_95', 'nitrateSurface_March_max', 'chlorophyll_mean_change_peak', 'temperatureSurface_Moving_quantile_5_10', 'phosphateSurface_Moving_quantile_95_2', 'zonalCurrentSurface_Moving_std_12_mean', 'temperatureSurface_Moving_min_3_mean', 'chlorophyll_July_quantile_5', 'secchiDiskDepth_January_min', 'dissolvedOxygen100_300trim_mean_25%', 'phosphate100_300_years_quantile_5', 'chlorophyll_September_quantile_10', 'meridionalCurrent100_300_Moving_skew_9_mean', 'wave_Height_Moving_median_12_mean', 'nitrateSurface_50medfilt_mean', 'dissolvedOxygenSurface_September_quantile_90', 'zonalCurrent100_300_decimate_mean', 'salinity100_300_February_max', 'nitrateSurface_December_quantile_10', 'secchiDiskDepthtrim_mean_15%', 'salinitySurface_April_median', 'phosphateSurface_February_max', 'euphoticDepth_June_max', 'salinity100_300_51savgol_mean', 'nitrate100_300_September_quantile_99', 'nitrateSurface_51savgol_mean', 'zonalCurrentSurface_Moving_quantile_99_6', 'temperatureSurface_Moving_quantile_5_5', 'wave_Height_December_quantile_95', 'euphoticDepth_July_quantile_95', 'euphoticDepth_Moving_min_5_mean', 'phosphate100_300_2spline_mean', 'salinity100_300_quantile_5', 'secchiDiskDepth_December_mean', 'temperatureSurface_years_quantile_10', 'nitrateSurface_Hann_window_mean', 'nitrate100_300_April_max', 'chlorophyll_May_median', 'dissolvedOxygen100_300_August_quantile_1', 'temperatureSurface_January_median']
train.drop(labels=kendall_hick_corr_feat, axis=1, inplace=True)
test.drop(labels=kendall_hick_corr_feat, axis=1, inplace=True)

### We find features that are high correlated to target

In [16]:
def high_targ_correl(method = "pearson"):
    if method=="pearson":
        corr_mat = train.corr().abs()
    elif method=="kendall":
        corr_mat = train.corr("kendall").abs()
    elif method=="spearman":
        corr_mat = train.corr("spearman").abs()
    else:
        print("Invalid Method")
    cor_target = corr_mat['Overall Probability'].sort_values(kind="quicksort", ascending=False)
    print(cor_target)
    return cor_target

In [17]:
# target_correl = high_targ_correl(method = "spearman")
# cols_to_excl = train.loc[:,target_correl<0.3].columns

### Find Best features using SelectKBest

In [18]:
def select_KBest(x_train,y_train,x_test,num_features,method="Mutual_InReg"):
    if method == "Mutual_InReg":
        mi_select = SelectKBest(mutual_info_regression, k=num_features)
    elif  method == "f_Regre":
        mi_select = SelectKBest(f_regression, k=num_features)
    else:
        print("Invalide_method")
    x_train_new = mi_select.fit_transform(x_train, y_train)
    x_test_new = mi_select.transform(x_test)
    return pd.DataFrame(x_train_new),pd.DataFrame(x_test_new)                            

### Dimensionality Reduction Using PCA

In [19]:
def PCA_dim_reduct(x_train,x_test,num_features):
    PCA_features = PCA(n_components=num_features)
    x_train = PCA_features.fit_transform(x_train)
    x_train = pd.DataFrame(data = x_train)
    x_test = PCA_features.transform(x_test)
    x_test = pd.DataFrame(data = x_test)
    return x_train,x_test

### Basic Feature Engineering

In [20]:
def bathymetry_correction():
    train['bathymetry'] = np.where((train['bathymetry'] < -2000), -2000,train['bathymetry'] )
    test['bathymetry'] = np.where((test['bathymetry'] < -2000), -2000,test['bathymetry'] )
    return 
def bathym_below_400():
    train["Bathym_below_400"] = np.where((train['bathymetry'] < -400), 1,0)
    return

In [21]:
#bathymetry_correction()

## Train Models

In [22]:
ntrain = train.shape[0]
ntest = test.shape[0]
SEED = 2019 # for reproducibility
def cross_valid_method(method,NFOLDS,SEED):
    # Define Cross Validation
    if method=="SKF":
        folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)
    else:
        folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=SEED)

# Define evaluation function (Root Mean Square Error)
def cv_rmse(model, X, y):
    rmse = np.sqrt(-cross_val_score(model, X, y, scoring="neg_mean_squared_error", cv=folds))
    return (rmse)

In [23]:
def preprocess(train,test,outlier_method="quantiles",col_excl=[],KBEST=False,KBest_Feat = 450,KBest_Method ="f_Regre", PCA=False, PCA_Feat = 250):
    cols_to_exclude = ['obs_id', 'Overall Probability']+list(col_excl)
    y_train = train['Overall Probability'].ravel() #ravel coverts a series to a numpy array
    df_train_columns = [c for c in train.columns if c not in cols_to_exclude]
    x_train = outlier_replacement(train[df_train_columns],method = outlier_method) # converts a dataframe to a numpy array
    x_test = outlier_replacement(test[df_train_columns],method = outlier_method)
    x_train,x_test = stand_norm(x_train,x_test,"StandardScale")
    if KBEST ==True:
        x_train,x_test = select_KBest(x_train,y_train,x_test,KBest_Feat,method=KBest_Method)
    if PCA == True:
        x_train,x_test = PCA_dim_reduct(x_train,x_test,PCA_Feat)
    return x_train.values,x_test.values, y_train,df_train_columns

In [24]:
x_train,x_test,y_train,df_train_columns = preprocess(train,test,"mad",KBEST=False,KBest_Feat = 500,KBest_Method ="Mutual_InReg", PCA=False, PCA_Feat = 250)

In [25]:
folds = cross_valid_method("SKF",10,SEED)

In [26]:
def model_cross_valid(x_train,y_train,folds,model):
    score = cv_rmse(model, x_train, y_train)
    print("CV mean score: {:.4f} ({:.4f})".format(score.mean(), score.std()))
    return score

### LightGBM

In [27]:
# lightgbm = LGBMRegressor(objective='regression', 
#                                        num_leaves=20,
#                                        learning_rate=0.01 ,
#                                        n_estimators=2000,
#                                        max_bin=400,
#                                        max_depth = 5,
#                                        bagging_fraction=0.75,
#                                        bagging_freq=5, 
#                                        bagging_seed=7,
#                                        feature_fraction=0.2,
#                                        feature_fraction_seed=7,
#                                        lambda_l1= 0.00001,
#                                        verbose=-1,
#                                        )
# model_cross_valid(x_train,y_train,folds,lightgbm)

#### LightGBM Predict

In [28]:
#Early Stoping
# x_train, X_val, y_train, Y_val = train_test_split(x_train_m, y_train, test_size=0.15, random_state=SEED)
# lgb_model = lightgbm.fit(x_train, y_train,eval_set=[(X_val, Y_val)],early_stopping_rounds=150,verbose=True)
# lgb_model = lightgbm.fit(x_train, y_train)
# prediction = lgb_model.predict(x_test)

# lgb_model.booster_.feature_importance()

# # importance of each attribute
# fea_imp = pd.DataFrame({'cols':train[x_train].columns, 'fea_imp':lightgbm.feature_importances_})
# fea_imp.loc[fea_imp.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)[0:20]

### Linear Regression

In [29]:
# LinearRegr = LinearRegression()
# model_cross_valid(x_train,y_train,folds,LinearRegr)

#### Linear Reg Predict

In [30]:
# lin_reg_model = LinearRegr.fit(x_train, y_train)
# prediction = LinearRegr.predict(x_test)

### XGBRegressor

In [31]:
# XGBReg = XGB.XGBRegressor(objective ='reg:squarederror',
#                                 n_estimators = 1500,
#                                 colsample_bytree = 0.25,
#                                 learning_rate = 0.01,
#                                 max_depth = 5,
#                                 alpha = 0.15,)
XGBReg = XGB.XGBRegressor(objective ='reg:squarederror',
                                reg_lambda =  0.004281597410975359,
                                alpha = 0.002199542276200622,
                                n_estimators = 4000,
                                subsample =  0.7,
                                colsample_bytree = 0.3,
                                learning_rate = 0.018,
                                max_depth = 9,
                                min_child_weight= 14,
                                )
model_cross_valid(x_train,y_train,folds,XGBReg)


CV mean score: 0.1760 (0.0331)


array([0.2372018 , 0.17837164, 0.15577205, 0.14052943, 0.16833879])

In [32]:
def objective(trial,data=x_train,target=y_train):
    
    train_x, test_x, train_y, test_y = train_test_split(x_train,y_train, test_size=0.15,random_state=SEED)
    param = {
        'lambda': trial.suggest_loguniform('lambda', 1e-3, 10.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-3, 10.0),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3,0.4,0.5,0.6,0.7,0.8,0.9, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.4,0.5,0.6,0.7,0.8,1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.008,0.009,0.01,0.012,0.014,0.016,0.018, 0.02]),
        'n_estimators': 4000,
        'max_depth': trial.suggest_categorical('max_depth', [5,7,9,11,13,15,17,20]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 300),
    }
    model = XGB.XGBRegressor(objective ='reg:squarederror',**param)  
    
    model.fit(train_x,train_y,eval_set=[(test_x,test_y)],early_stopping_rounds=100,verbose=False)
    
    preds = model.predict(test_x)
    
    rmse = mean_squared_error(test_y, preds,squared=False)
    
    return rmse

In [33]:
#Best trial: {'lambda': 0.004281597410975359, 'alpha': 0.002199542276200622, 'colsample_bytree': 0.3, 'subsample': 0.7, 'learning_rate': 0.018, 'max_depth': 9, 'min_child_weight': 14}

In [34]:
# study = optuna.create_study(direction='minimize')
# study.optimize(objective, n_trials=50)
# print('Number of finished trials:', len(study.trials))
# print('Best trial:', study.best_trial.params)

In [35]:
xgb_model = XGBReg.fit(x_train,y_train)
prediction = xgb_model.predict(x_test)
fea_imp = pd.DataFrame({'cols':train[df_train_columns].columns, 'fea_imp':XGBReg.feature_importances_})
fea_imp.loc[fea_imp.fea_imp > 0].sort_values(by=['fea_imp'], ascending = False)[0:30]


,cols,fea_imp
10,temperatureSurface_min,0.081606
24,salinitySurface_min,0.062848
21,salinitySurface_mean,0.040672
920,phosphateSurface_December_mean,0.033842
670,euphoticDepth_January_min,0.026538
640,chlorophyll_February_min,0.023472
836,nitrateSurface_September_mean,0.018687
119,phosphateSurface_median,0.014254
302,dissolvedOxygenSurface_December_mean,0.014143
161,temperatureSurface_May_min,0.013410


### Create Submission File

In [36]:
sample_submission = pd.read_csv('../input/engraulis-encrasicolus-prediction/sample_submission.csv')
sub_df = pd.DataFrame({"obs_id":sample_submission["obs_id"].values})
sub_df["Overall Probability"] = prediction
sub_df["Overall Probability"] = sub_df["Overall Probability"].apply(lambda x: 1 if x>1 else 0 if x<0 else x)
sub_df.to_csv("submission.csv", index=False)